# With some null constraints and 3 derivatives (Final)

In [ ]:
#THIS SECTION IS JUST TO TEST THE EXPRESSIONS FOR THE OPEN STRING AMPLITUDE
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import numpy as np
from scipy.special import legendre
from scipy import integrate
from scipy.special import hyp2f1 as scipy_hyp2f1
from scipy.special import gamma as scipy_gamma
from scipy.special import zeta
#This is all for to check formulaes using the Open String Amplitude
#Some definitions
d = 10;
alpha = (d-3)/2;
lm =146/10;
dlm =10**(-1)
lmp = lm + dlm
lmm = lm - dlm
ellmax = 20;
w00 = -(np.pi)**2/6
w10 = -zeta(3)
w01 = 7*np.pi**4/360

def hyp2f1(a, b, c, z):
    if isinstance(z, torch.Tensor):
        z_np = z.detach().numpy()
        result = scipy_hyp2f1(a,b,c,z_np)
        return torch.tensor(result, dtype=z.dtype)
    else:
        return torch.tensor(scipy_hyp2f1(a,b,c,z))

def gamma(x):
    if isinstance(x, torch.Tensor):
        x_np = x.detach().numpy()
        result = scipy_gamma(x_np)
        return torch.tensor(result, dtype=x.dtype)
    else:
        return torch.tensor(scipy_gamma(x))

def ker(n,s1,s2):
    return (1/(s1 - n) + 1/(s2 - n) + 1/(n + lm))

def geg(ell,beta, z):
    return gamma(ell + 2*beta) / (gamma(2*beta) * gamma(ell + 1)) * \
           hyp2f1(-ell, ell + 2*beta, beta + 1/2, (1-z)/2)

#ell,n tensor and Null grid tensor
ell = torch.arange(ellmax + 1, dtype = torch.float64).unsqueeze(1).repeat(1, ellmax + 1)
n = torch.arange(1, ellmax + 2, dtype = torch.float64).unsqueeze(0).repeat(ellmax + 1, 1)
mask = (n > ell) & (n <= ellmax + 1) & ((n - ell - 1) % 2 == 0)
ell_n_tensor = torch.stack([ell[mask], n[mask]], dim=1)

NullPoints = torch.stack(torch.meshgrid(
    torch.arange(-2 +1/2, 2+1/2 + 1e-6, 11/10, dtype=torch.float64),
    torch.arange(-0.1, 0.1 + 1e-6, 2/11, dtype=torch.float64),
    indexing='ij'
)).permute(1, 2, 0).reshape(-1, 2)


def w00comp(ell_n_tensor):
    ell = ell_n_tensor[:, 0].view(-1, 1)
    n = ell_n_tensor[:, 1].view(-1, 1)
    result = (-1/n)*geg(ell,alpha,1)
    return result.squeeze(-1)

def w10comp(ell_n_tensor):
    ell = ell_n_tensor[:, 0].view(-1, 1)  # Shape (k, 1)
    n = ell_n_tensor[:, 1].view(-1, 1)    # Shape (k, 1)
    result = (gamma(-3 + d + ell) * (-((2 * ell * (-3 + d + ell) * lm * (n + 2 * lm) * \
            hyp2f1(1 - ell, -2 + d + ell, d / 2, lm / (n + lm))) / ((-2 + d) * (n + lm)**2)) - \
            hyp2f1(-ell, -3 + d + ell, 1/2 * (-2 + d), lm / (n + lm)))) / (n**2 * gamma(-3 + d) * gamma(1 + ell))
    return result.squeeze(-1)


def w01comp(ell_n_tensor):
    ell = ell_n_tensor[:, 0].view(-1, 1)  # Shape (k, 1)
    n = ell_n_tensor[:, 1].view(-1, 1)    # Shape (k, 1)
    result = (gamma(-3 + d + ell) * (-(1/((d - 2) * (n + lm)**2)) *\
             2 * ell * (-3 + d + ell) * n * (n + 2*lm) *  hyp2f1(1 - ell, -2 + d + ell, d/2, lm/(n + lm)) + \
             2 * hyp2f1(-ell, -3 + d + ell, (d - 2)/2, lm/(n + lm)))) / (n**3 * gamma(-3 + d) * gamma(1 + ell))
    return result.squeeze(-1)

def ampcomp(ell_n_tensor, NullPoints):
    ell = ell_n_tensor[:, 0].view(-1, 1)
    n = ell_n_tensor[:, 1].view(-1, 1)
    s1 = NullPoints[:, 0].view(1, -1)
    s2 = NullPoints[:, 1].view(1, -1)
    result = ker(n, s1, s2) * geg(ell, alpha, 1 + 2 * (((s1 + lm) * (s2 + lm)) / (n + lm) - lm) / n)
    return result.squeeze(-1)

def ampcompD1P(ell_n_tensor, NullPoints):
    ell = ell_n_tensor[:, 0].view(-1, 1)
    n = ell_n_tensor[:, 1].view(-1, 1)
    s1 = NullPoints[:, 0].view(1, -1)
    s2 = NullPoints[:, 1].view(1, -1)
    result = ker(n,s1,s2)*geg(ell,alpha,1 + 2*(((s1 + lmp)*(s2 + lmp))/(n + lmp)-lmp)/n)
    return result.squeeze(-1)

def ampcompD1M(ell_n_tensor, NullPoints):
    ell = ell_n_tensor[:, 0].view(-1, 1)
    n = ell_n_tensor[:, 1].view(-1, 1)
    s1 = NullPoints[:, 0].view(1, -1)
    s2 = NullPoints[:, 1].view(1, -1)
    result = ker(n,s1,s2)*geg(ell,alpha,1 + 2*(((s1 + lmm)*(s2 + lmm))/(n + lmm)-lmm)/n)
    return result.squeeze(-1)


def ampcompD1(ell_n_tensor, NullPoints):
    ell = ell_n_tensor[:, 0].view(-1, 1)
    n = ell_n_tensor[:, 1].view(-1, 1)
    s1 = NullPoints[:, 0].view(1, -1)
    s2 = NullPoints[:, 1].view(1, -1)
    result = ((2 * (-3 + d) * (n**2 + 2 * n * lm - s2 * lm - s1 * (s2 + lm)) * geg(-1 + ell, 1/2 * (-1 + d),
        1 + (2 * ((-n + s2) * lm + s1 * (s2 + lm))) /  (n * (n + lm))) / n - (n + lm) * geg(ell, 1/2 * (-3 + d),
        1 + (2 * ((-n + s2) * lm + s1 * (s2 + lm))) / (n * (n + lm)))) / (n + lm)**3)
    return result.squeeze(-1)

def ampcompD2(ell_n_tensor, NullPoints):
    ell = ell_n_tensor[:, 0].view(-1, 1)
    n = ell_n_tensor[:, 1].view(-1, 1)
    s1 = NullPoints[:, 0].view(1, -1)
    s2 = NullPoints[:, 1].view(1, -1)
    result = (1 / (n + lm)**4 * 2 * (-((2 * (-3 + d) * (-1 + d) * (n - s1) * (n - s2) *
               (n**2 + 2*n*lm - s2*lm - s1*(s2 + lm)) *
               geg(-2 + ell,(d+1)/2, 1 + (2 * ((-n + s2)*lm + s1*(s2 + lm)))/(n*(n + lm)))) /
              (n**2 * (n + lm))) + ((-3 + d) * (n - s1) * (n - s2) *
             geg(-1 + ell,(d-1)/2, 1 + (2 * ((-n + s2)*lm + s1*(s2 + lm)))/(n*(n + lm)))) / n +
            ((-3 + d) * (2*n - s1 - s2) * (n + lm) *
             geg(-1 + ell, (d-1)/2, 1 + (2 * ((-n + s2)*lm + s1*(s2 + lm)))/(n*(n + lm)))) / n -
            (3 * (-3 + d) * (n**2 + 2*n*lm - s2*lm - s1*(s2 + lm)) *
             geg(-1 + ell,(d-1)/2, 1 + (2 * ((-n + s2)*lm + s1*(s2 + lm)))/(n*(n + lm)))) / n +
            (n + lm) * geg(ell,(d-3)/2, 1 + (2 * ((-n + s2)*lm + s1*(s2 + lm)))/(n*(n + lm)))))

    return result.squeeze(-1)


def ampcompD3(ell_n_tensor, NullPoints):
    ell = ell_n_tensor[:, 0].view(-1, 1)
    n = ell_n_tensor[:, 1].view(-1, 1)
    s1 = NullPoints[:, 0].view(1, -1)
    s2 = NullPoints[:, 1].view(1, -1)
    result =  (2 / (n**3 * (n + lm)**7)) * (4 * (-3 + d) * (-1 + d) * (1 + d) * (n - s1)**2 * (n - s2)**2 * (n**2 + 2 * n * lm - s2 * lm -
        s1 * (s2 + lm)) * geg(-3 + ell, (3 + d)/2, 1 + (2 * ((-n + s2) * lm + s1 * (s2 + lm)))/(n * (n + lm))) -
     2 * (-3 + d) * (-1 + d) * n * (n - s1)**2 * (n -s2)**2 * (n + lm) * geg(-2 + ell, (1 + d)/2,
       1 + (2 * ((-n + s2) * lm + s1 * (s2 + lm)))/(n * (n + lm))) -
     4 * (-3 + d) * (-1 + d) * n * (n - s1) * (n - s2) * (2 * n - s1 -s2) * (n + lm)**2 * geg(-2 + ell, (1 + d)/2,
       1 + (2 * ((-n + s2) * lm + s1 * (s2 + lm)))/(n * (n + lm))) +
     8 * (-3 + d) * (-1 + d) * n * (n - s1) * (n - s2) * (n + lm) * (n**2 +2 * n * lm - s2 * lm -
        s1 * (s2 + lm)) * geg(-2 + ell, (1 + d)/2,1 + (2 * ((-n + s2) * lm + s1 * (s2 + lm)))/(n * (n + lm))) -
     2 * (-3 + d) * n**2 * (n - s1) * (n -s2) * (n + lm)**2 * geg(-1 + ell, 1/2 * (-1 + d),
       1 + (2 * ((-n + s2) * lm + s1 * (s2 + lm)))/( n * (n + lm))) -
     2 * (-3 + d) * n**2 * (2 * n - s1 - s2) * (n + lm)**3 * geg(-1 + ell, 1/2 * (-1 + d),1 + (2 * ((-n + s2) * lm + s1 * (s2 + lm)))/(n * (n + lm))) +
     n**3 * (n + lm)**3 * geg(ell, 1/2 * (-3 + d),1 + (2 * ((-n + s2) * lm + s1 * (s2 + lm)))/( n * (n + lm))) +
     4 * n * (n + lm) * (2 * (-3 + d) * (-1 + d) * (n - s1) * (n -s2) * (n**2 + 2 * n * lm - s2 * lm -
     s1 * (s2 + lm)) * geg(-2 + ell, (1 + d)/2,1 + (2 * ((-n + s2) * lm + s1 * (s2 + lm)))/(n * (n + lm)))
     - (-3 + d) * n * (n - s1) * (n -s2) * (n + lm) * geg(-1 + ell, 1/2 * (-1 + d),
      1 + (2 * ((-n + s2) * lm + s1 * (s2 + lm)))/(n * (n + lm))) - (-3 + d) * n * (2 * n - s1 -s2) * (n + lm)**2 * geg(-1 + ell, 1/2 * (-1 + d),
      1 + (2 * ((-n + s2) * lm + s1 * (s2 + lm)))/( n * (n + lm))) +3 * (-3 + d) * n * (n + lm) * (n**2 + 2 * n * lm -
      s2 * lm - s1 * (s2 + lm)) * geg(-1 + ell,1/2 * (-1 + d),1 + (2 * ((-n + s2) * lm + s1 * (s2 + lm)))/( n * (n + lm))) -
        n**2 * (n + lm)**2 * geg(ell, 1/2 * (-3 + d),  1 + (2 * ((-n + s2) * lm + s1 * (s2 + lm)))/(n * (n + lm)))))
    return result.squeeze(-1)

#First, lets calculate cc(ell,n) for open string

def AbsOpString(n,s2):
    return (-1)**(1 + n)*gamma(-s2)/(gamma(n+1)*gamma(1 - n - s2))



def cc(ell_n_tensor):
    z = torch.linspace(-1 + 10**(-6), 1 - 10**(-6), 10**6).view(1, -1)

    # Directly use ell and n from ell_n_tensor
    ell_values = ell_n_tensor[:, 0].view(-1, 1)  # Shape (k, 1)
    n_values = ell_n_tensor[:, 1].view(-1, 1)    # Shape (k, 1)

    # Compute normfac for all combinations of ell and n
    normfac = ((np.pi * 2**(1 - 2 * alpha) * gamma(2 * alpha + ell_values)) /
                (gamma(ell_values + 1) * (ell_values + alpha) * gamma(alpha)**2))**-1

    # Compute integrand
    integrand = normfac * (1 - z**2)**(alpha - 1/2) * \
                AbsOpString(n_values, n_values * (z - 1) / 2) * \
                geg(ell_values, alpha, z)

    # Integrate across z
    return torch.trapz(integrand, z, dim=1)

NullPoints

tensor([[-1.5000, -0.1000],
        [-1.5000,  0.0818],
        [-0.4000, -0.1000],
        [-0.4000,  0.0818],
        [ 0.7000, -0.1000],
        [ 0.7000,  0.0818],
        [ 1.8000, -0.1000],
        [ 1.8000,  0.0818]], dtype=torch.float64)

In [ ]:
#Store all the values
coeff_vals = cc(ell_n_tensor)
w00comp_vals = w00comp(ell_n_tensor)
w01comp_vals = w01comp(ell_n_tensor)
w10comp_vals = w10comp(ell_n_tensor)

ampcomp_vals = ampcomp(ell_n_tensor, NullPoints)
ampcompD1P_vals = ampcompD1P(ell_n_tensor, NullPoints)
ampcompD1M_vals = ampcompD1M(ell_n_tensor, NullPoints)
ampcompD1_vals = ampcompD1(ell_n_tensor, NullPoints)
ampcompD2_vals = ampcompD2(ell_n_tensor, NullPoints)
ampcompD3_vals = ampcompD3(ell_n_tensor, NullPoints)

<ipython-input-2-740bb5865f5a>:29: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return torch.tensor(result, dtype=z.dtype)
<ipython-input-2-740bb5865f5a>:31: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return torch.tensor(scipy_hyp2f1(a,b,c,z))


In [ ]:
#CHECKS FOR OPEN STRING
def w00fullX():
    return torch.dot(coeff_vals,w00comp_vals)

def w10fullX():
    return torch.dot(coeff_vals,w10comp_vals)

def w01fullX():
    return torch.dot(coeff_vals,w01comp_vals)

def ampfullX():
    return torch.sum(coeff_vals.view(-1, 1) * ampcomp_vals, dim = 0)

def ampD1PfullX():
    return torch.sum(coeff_vals.view(-1, 1) * ampcompD1P_vals, dim = 0)

def ampD1MfullX():
    return torch.sum(coeff_vals.view(-1, 1) * ampcompD1M_vals, dim = 0)

def ampD1fullX():
    return torch.sum(coeff_vals.view(-1, 1) * ampcompD1_vals, dim = 0)

def ampD2fullX():
    return torch.sum(coeff_vals.view(-1, 1) * ampcompD2_vals, dim = 0)

def ampD3fullX():
    return torch.sum(coeff_vals.view(-1, 1) * ampcompD3_vals, dim = 0)

(w00, w00fullX().item(),w10, w10fullX().item(),w01, w01fullX().item(),ampfullX()[1].item(),
 ampD1fullX()[1].item(),ampD2fullX()[1].item(),ampD3fullX()[1].item(),
 ampD1PfullX()[1].item(),ampD1MfullX()[1].item(),(ampD1PfullX()/ampD1MfullX())[1].item())

(-1.6449340668482264,
 -1.5984308179417641,
 -1.2020569031595942,
 -1.202056902362787,
 1.8940656589944915,
 1.894065658958795,
 -0.9368392530438512,
 1.4470927928167399e-11,
 7.384788822117905e-10,
 -3.6782704044614235e-10,
 -0.9366382712809055,
 -0.9370430272245681,
 0.9995680497779686)

In [ ]:
# Same definitions for the PINN
def w00full():
    return torch.dot(ImPW(ell_n_tensor),w00comp_vals)

def w01full():
    return torch.dot(ImPW(ell_n_tensor),w01comp_vals)

def w10full():
    return torch.dot(ImPW(ell_n_tensor),w10comp_vals)

def ampfull():
    return torch.sum(ImPW(ell_n_tensor).view(-1, 1) * ampcomp_vals, dim = 0)

def ampD1Pfull():
    return torch.sum(ImPW(ell_n_tensor).view(-1, 1) * ampcompD1P_vals, dim = 0)

def ampD1Mfull():
    return torch.sum(ImPW(ell_n_tensor).view(-1, 1) * ampcompD1M_vals, dim = 0)

def ampD1full():
    return torch.sum(ImPW(ell_n_tensor).view(-1, 1) * ampcompD1_vals, dim = 0)

def ampD2full():
    return torch.sum(ImPW(ell_n_tensor).view(-1, 1) * ampcompD2_vals, dim = 0)

def ampD3full():
    return torch.sum(ImPW(ell_n_tensor).view(-1, 1) * ampcompD3_vals, dim = 0)

In [ ]:
# Define the neural network. Now lets impose null constraints as well
class SimpleNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(SimpleNN, self).__init__()
        self.input = nn.Linear(input_size, hidden_size, dtype=torch.float64)
        self.hidden1 = nn.Linear(hidden_size, hidden_size, dtype=torch.float64)
        self.hidden2 = nn.Linear(hidden_size, hidden_size, dtype=torch.float64)
        self.output = nn.Linear(hidden_size, output_size, dtype=torch.float64)

    def forward(self, x):
        x = torch.relu(self.input(x))
        x = torch.relu(self.hidden1(x))
        x = torch.relu(self.hidden2(x))
        x = F.softplus(self.output(x))
        return x.squeeze()  # Remove extra dimensions for output

# Create the neural network
input_size = 2  # Since ell_n_tensor has 2 columns
hidden_size = 64
output_size = 1  # Since coeff(ell,n) it a 1D tensor
ImPW = SimpleNN(input_size, hidden_size, output_size)

# Training loop
num_epochs = 90000
min_cons = torch.tensor([10**4], dtype = torch.float64)  # Initialize minimum cons to positive infinity
min_cons_epoch = 0
max_w00 = -torch.tensor([10**10], dtype = torch.float64)  # Track the max w00
min_cons_w00 = 0  # Save w00 for minimum loss epoch
min_cons_w10 = 0  # Save w10 difference for minimum loss epoch
min_cons_w01 = 0  # Save w01 difference for minimum loss epoch
min_cons_D1 = 0
min_cons_D2 = 0
min_cons_D3 = 0

lr = 0.0001
optimizer = optim.Adam(ImPW.parameters(), lr=lr)  # Adam optimizer with learning rate of 0.001

zero_vec = torch.zeros(len(NullPoints),dtype = torch.float64)
for epoch in range(num_epochs+1):

    w00vals = w00full()
    w01vals = w01full()
    w10vals = w10full()


    losscomp1 = (w01- w01vals)
    losscomp2 = (w10- w10vals)
    losscomp3 =  F.mse_loss(ampD1full(),zero_vec)
    losscomp4 =  F.mse_loss(ampD2full(),zero_vec)
    losscomp5 =  F.mse_loss(ampD3full(),zero_vec)
    beta1 = 10**3
    beta2 = 10**4
    beta3 = 10**3


    cons1 = (losscomp1**2 + losscomp2**2)
    cons2 =  losscomp3
    cons3 =  (losscomp4 + losscomp5)
    cons = torch.max(torch.tensor([cons1,cons2,cons3]))**(1/2)

    loss = - w00vals + (beta1**2)*cons1 + (beta2**2)*cons2 + (beta3**2)*cons3

    if cons < min_cons:
      if w00vals > max_w00:
        min_cons = cons.item()
        min_cons_epoch = epoch
        max_w00 = w00vals.item()
        min_cons_w10 = (losscomp1).item()
        min_cons_w01 = (losscomp2).item()
        min_cons_D1 = (losscomp3**(1/2)).item()
        min_cons_D2 = (losscomp4**(1/2)).item()
        min_cons_D3 = (losscomp5**(1/2)).item()
        min_cons_coeffs = ImPW(ell_n_tensor)

    # Print loss every 100 epochs
    if epoch % 5000 == 0:
        print(f"Epoch {epoch}, Loss_Tot: {loss.item()}, Const: {(losscomp1).item(), (losscomp2).item(), (losscomp3**(1/2)).item(), (losscomp4**(1/2)).item(),(losscomp5**(1/2)).item()}, W00: {w00vals.item()}")

    if epoch % 20000 == 0:
        lr = lr

    # Backward pass and optimization
    optimizer.zero_grad()  # Zero the gradients
    loss.backward()  # Backpropagation
    optimizer.step()  # Update weights

torch.set_printoptions(precision=24)
# Print final values
print("Final values:")
print(w00vals.item(),  (losscomp1).item(), (losscomp2).item(), (losscomp3**(1/2)).item(),(losscomp4**(1/2)).item(),(losscomp5**(1/2)).item())

# Print the minimum loss and the epoch it occurred
print(f"\nMinimum Cons: {min_cons:.24f} at epoch {min_cons_epoch}")
print("Values at minimum cons epoch:")
print(f"W00: {max_w00:.10f}")
print(f"W10 diff: {min_cons_w10:.10f}")
print(f"W01 diff: {min_cons_w01:.10f}")
print(f"Comp3: {min_cons_D1:.16f}")
print(f"Comp4: {min_cons_D2:.16f}")
print(f"Comp4: {min_cons_D3:.16f}")
print(f"Coeffs: {min_cons_coeffs}")

Epoch 0, Loss_Tot: 3032880943.758614, Const: (1.5600017283286223, 6.706810379854307, 5.433765075937293, 4.5648051004082735, 3.45460394646817), W00: -114008.81565555999
Epoch 5000, Loss_Tot: 9683.746061569747, Const: (2.0157613816795816e-05, 0.0006048677075822795, 0.007980506770757356, 0.018938274624205928, 0.01923553068864023), W00: -2585.867071870814
Epoch 10000, Loss_Tot: 1005.5874288836735, Const: (-3.3224187916403025e-05, 0.00016980997199067538, 0.0020254186292869543, 0.00704965418344302, 0.0068216996199992564), W00: -499.0922174125252
Epoch 15000, Loss_Tot: 22.623251916459118, Const: (0.00011848952604021079, 5.276037207369022e-05, 0.00033697343887477394, 0.0001229042439029586, 6.385920810902182e-05), W00: -11.23213518947776
Epoch 20000, Loss_Tot: 8.441014955692493, Const: (-0.0014983734133482418, 0.0010571909200212115, 0.0001268891661327194, 5.1696699540969436e-05, 3.788216323509383e-05), W00: -3.464045773267852
Epoch 25000, Loss_Tot: 2.1528507967245933, Const: (1.3487150787971913

In [ ]:
# Define the neural network. Now lets impose null constraints as well
class SimpleNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(SimpleNN, self).__init__()
        self.input = nn.Linear(input_size, hidden_size, dtype=torch.float64)
        self.hidden1 = nn.Linear(hidden_size, hidden_size, dtype=torch.float64)
        self.hidden2 = nn.Linear(hidden_size, hidden_size, dtype=torch.float64)
        self.output = nn.Linear(hidden_size, output_size, dtype=torch.float64)

    def forward(self, x):
        x = torch.relu(self.input(x))
        x = torch.relu(self.hidden1(x))
        x = torch.relu(self.hidden2(x))
        x = F.softplus(self.output(x))
        return x.squeeze()  # Remove extra dimensions for output

# Create the neural network
input_size = 2  # Since ell_n_tensor has 2 columns
hidden_size = 64
output_size = 1  # Since coeff(ell,n) it a 1D tensor
ImPW = SimpleNN(input_size, hidden_size, output_size)

# Training loop
num_epochs = 90000
min_cons = torch.tensor([10**4], dtype = torch.float64)  # Initialize minimum cons to positive infinity
min_cons_epoch = 0
max_w00 = -torch.tensor([10**10], dtype = torch.float64)  # Track the max w00
min_cons_w00 = 0  # Save w00 for minimum loss epoch
min_cons_w10 = 0  # Save w10 difference for minimum loss epoch
min_cons_w01 = 0  # Save w01 difference for minimum loss epoch
min_cons_D1 = 0
min_cons_D2 = 0
min_cons_D3 = 0

lr = 0.0001
optimizer = optim.Adam(ImPW.parameters(), lr=lr)  # Adam optimizer with learning rate of 0.001

zero_vec = torch.zeros(len(NullPoints),dtype = torch.float64)
for epoch in range(num_epochs+1):

    w00vals = w00full()
    w01vals = w01full()
    w10vals = w10full()


    losscomp1 = (w01- w01vals)
    losscomp2 = (w10- w10vals)
    losscomp3 =  F.mse_loss(ampD1full(),zero_vec)
    losscomp4 =  F.mse_loss(ampD2full(),zero_vec)
    losscomp5 =  F.mse_loss(ampD3full(),zero_vec)
    beta1 = 10**3
    beta2 = 10**4
    beta3 = 10**3


    cons1 = (losscomp1**2 + losscomp2**2)
    cons2 =  losscomp3
    cons3 =  (losscomp4 + losscomp5)
    cons = torch.max(torch.tensor([cons1,cons2,cons3]))**(1/2)

    loss = - w00vals + (beta1**2)*cons1 + (beta2**2)*cons2 + (beta3**2)*cons3

    if cons < min_cons:
      if w00vals > max_w00:
        min_cons = cons.item()
        min_cons_epoch = epoch
        max_w00 = w00vals.item()
        min_cons_w10 = (losscomp1).item()
        min_cons_w01 = (losscomp2).item()
        min_cons_D1 = (losscomp3**(1/2)).item()
        min_cons_D2 = (losscomp4**(1/2)).item()
        min_cons_D3 = (losscomp5**(1/2)).item()
        min_cons_coeffs = ImPW(ell_n_tensor)

    # Print loss every 100 epochs
    if epoch % 5000 == 0:
        print(f"Epoch {epoch}, Loss_Tot: {loss.item()}, Const: {(losscomp1).item(), (losscomp2).item(), (losscomp3**(1/2)).item(), (losscomp4**(1/2)).item(),(losscomp5**(1/2)).item()}, W00: {w00vals.item()}")

    if epoch % 20000 == 0:
        lr = lr

    # Backward pass and optimization
    optimizer.zero_grad()  # Zero the gradients
    loss.backward()  # Backpropagation
    optimizer.step()  # Update weights

torch.set_printoptions(precision=24)
# Print final values
print("Final values:")
print(w00vals.item(),  (losscomp1).item(), (losscomp2).item(), (losscomp3**(1/2)).item(),(losscomp4**(1/2)).item(),(losscomp5**(1/2)).item())

# Print the minimum loss and the epoch it occurred
print(f"\nMinimum Cons: {min_cons:.24f} at epoch {min_cons_epoch}")
print("Values at minimum cons epoch:")
print(f"W00: {max_w00:.10f}")
print(f"W10 diff: {min_cons_w10:.10f}")
print(f"W01 diff: {min_cons_w01:.10f}")
print(f"Comp3: {min_cons_D1:.16f}")
print(f"Comp4: {min_cons_D2:.16f}")
print(f"Comp4: {min_cons_D3:.16f}")
print(f"Coeffs: {min_cons_coeffs}")

Epoch 0, Loss_Tot: 6672447252.684245, Const: (1.7070715852463794, 10.291343524741169, 8.056814229510731, 6.796830708592248, 5.10366399717221), W00: -151560.98090172617
Epoch 5000, Loss_Tot: 11745.056475890831, Const: (0.002379821309393959, 0.004921376177963177, 0.009183834000379861, 0.022072911877896194, 0.021711960580590453), W00: -2322.2696172653627
Epoch 10000, Loss_Tot: 6505.084722087959, Const: (0.0005555528903644635, 0.000942948735810134, 0.006325100666942398, 0.020087877551203278, 0.017871993292239978), W00: -1780.2661173100755
Epoch 15000, Loss_Tot: 975.9246002206447, Const: (0.0001304430495527864, 0.0004791198738032243, 0.001415742000931321, 0.007420508013487918, 0.006045162020426481), W00: -683.6375646264478
Epoch 20000, Loss_Tot: 81.14986055528874, Const: (-3.863445436014246e-06, -2.1447005924324358e-05, 0.000481959315125809, 0.0007328225744465594, 0.00015894405288302354), W00: -57.35861537379573
Epoch 25000, Loss_Tot: 15.150938415522852, Const: (-0.000888815245845942, 0.000

In [ ]:
# Define the neural network. Now lets impose null constraints as well
class SimpleNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(SimpleNN, self).__init__()
        self.input = nn.Linear(input_size, hidden_size, dtype=torch.float64)
        self.hidden1 = nn.Linear(hidden_size, hidden_size, dtype=torch.float64)
        self.hidden2 = nn.Linear(hidden_size, hidden_size, dtype=torch.float64)
        self.output = nn.Linear(hidden_size, output_size, dtype=torch.float64)

    def forward(self, x):
        x = torch.relu(self.input(x))
        x = torch.relu(self.hidden1(x))
        x = torch.relu(self.hidden2(x))
        x = F.softplus(self.output(x))
        return x.squeeze()  # Remove extra dimensions for output

# Create the neural network
input_size = 2  # Since ell_n_tensor has 2 columns
hidden_size = 64
output_size = 1  # Since coeff(ell,n) it a 1D tensor
ImPW = SimpleNN(input_size, hidden_size, output_size)

# Training loop
num_epochs = 90000
min_cons = torch.tensor([10**4], dtype = torch.float64)  # Initialize minimum cons to positive infinity
min_cons_epoch = 0
max_w00 = -torch.tensor([10**10], dtype = torch.float64)  # Track the max w00
min_cons_w00 = 0  # Save w00 for minimum loss epoch
min_cons_w10 = 0  # Save w10 difference for minimum loss epoch
min_cons_w01 = 0  # Save w01 difference for minimum loss epoch
min_cons_D1 = 0
min_cons_D2 = 0
min_cons_D3 = 0

lr = 0.0001
optimizer = optim.Adam(ImPW.parameters(), lr=lr)  # Adam optimizer with learning rate of 0.001

zero_vec = torch.zeros(len(NullPoints),dtype = torch.float64)
for epoch in range(num_epochs+1):

    w00vals = w00full()
    w01vals = w01full()
    w10vals = w10full()


    losscomp1 = (w01- w01vals)
    losscomp2 = (w10- w10vals)
    losscomp3 =  F.mse_loss(ampD1full(),zero_vec)
    losscomp4 =  F.mse_loss(ampD2full(),zero_vec)
    losscomp5 =  F.mse_loss(ampD3full(),zero_vec)
    beta1 = 10**3
    beta2 = 10**4
    beta3 = 10**3


    cons1 = (losscomp1**2 + losscomp2**2)
    cons2 =  losscomp3
    cons3 =  (losscomp4 + losscomp5)
    cons = torch.max(torch.tensor([cons1,cons2,cons3]))**(1/2)

    loss = - w00vals + (beta1**2)*cons1 + (beta2**2)*cons2 + (beta3**2)*cons3

    if cons < min_cons:
      if w00vals > max_w00:
        min_cons = cons.item()
        min_cons_epoch = epoch
        max_w00 = w00vals.item()
        min_cons_w10 = (losscomp1).item()
        min_cons_w01 = (losscomp2).item()
        min_cons_D1 = (losscomp3**(1/2)).item()
        min_cons_D2 = (losscomp4**(1/2)).item()
        min_cons_D3 = (losscomp5**(1/2)).item()
        min_cons_coeffs = ImPW(ell_n_tensor)

    # Print loss every 100 epochs
    if epoch % 5000 == 0:
        print(f"Epoch {epoch}, Loss_Tot: {loss.item()}, Const: {(losscomp1).item(), (losscomp2).item(), (losscomp3**(1/2)).item(), (losscomp4**(1/2)).item(),(losscomp5**(1/2)).item()}, W00: {w00vals.item()}")

    if epoch % 20000 == 0:
        lr = lr

    # Backward pass and optimization
    optimizer.zero_grad()  # Zero the gradients
    loss.backward()  # Backpropagation
    optimizer.step()  # Update weights

torch.set_printoptions(precision=24)
# Print final values
print("Final values:")
print(w00vals.item(),  (losscomp1).item(), (losscomp2).item(), (losscomp3**(1/2)).item(),(losscomp4**(1/2)).item(),(losscomp5**(1/2)).item())

# Print the minimum loss and the epoch it occurred
print(f"\nMinimum Cons: {min_cons:.24f} at epoch {min_cons_epoch}")
print("Values at minimum cons epoch:")
print(f"W00: {max_w00:.10f}")
print(f"W10 diff: {min_cons_w10:.10f}")
print(f"W01 diff: {min_cons_w01:.10f}")
print(f"Comp3: {min_cons_D1:.16f}")
print(f"Comp4: {min_cons_D2:.16f}")
print(f"Comp4: {min_cons_D3:.16f}")
print(f"Coeffs: {min_cons_coeffs}")

Epoch 0, Loss_Tot: 768551637.4797472, Const: (1.3272933235919409, 3.0856182041727465, 2.7367375368724276, 2.2692530229285723, 1.7560684329366525), W00: -62370.015090035384
Epoch 5000, Loss_Tot: 13032.274312853548, Const: (-6.134538322566385e-05, -6.321023002464621e-05, 0.0092026657463705, 0.027314109887890337, 0.027146578657429383), W00: -3080.363538350486
Epoch 10000, Loss_Tot: 4782.507036818766, Const: (0.0004655979108625363, -0.001670882107531968, 0.005152612934648971, 0.014656881454170418, 0.012724919334826737), W00: -1747.808656916256
Epoch 15000, Loss_Tot: 407.4519211868233, Const: (4.9221057696557935e-05, 0.00016003455790425392, 0.0009937696797547885, 0.002303515532016292, 0.0010399975133889806), W00: -302.27829114048984
Epoch 20000, Loss_Tot: 32.1156297809065, Const: (-0.00022413371065832521, 8.837252845306232e-05, 0.0003324835672660296, 0.0007896645093842569, 0.0003547205518148531), W00: -20.25365519941249
Epoch 25000, Loss_Tot: 12.271117791884098, Const: (-0.00083354824017778

In [ ]:
# Define the neural network. Now lets impose null constraints as well
class SimpleNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(SimpleNN, self).__init__()
        self.input = nn.Linear(input_size, hidden_size, dtype=torch.float64)
        self.hidden1 = nn.Linear(hidden_size, hidden_size, dtype=torch.float64)
        self.hidden2 = nn.Linear(hidden_size, hidden_size, dtype=torch.float64)
        self.output = nn.Linear(hidden_size, output_size, dtype=torch.float64)

    def forward(self, x):
        x = torch.relu(self.input(x))
        x = torch.relu(self.hidden1(x))
        x = torch.relu(self.hidden2(x))
        x = F.softplus(self.output(x))
        return x.squeeze()  # Remove extra dimensions for output

# Create the neural network
input_size = 2  # Since ell_n_tensor has 2 columns
hidden_size = 64
output_size = 1  # Since coeff(ell,n) it a 1D tensor
ImPW = SimpleNN(input_size, hidden_size, output_size)

# Training loop
num_epochs = 90000
min_cons = torch.tensor([10**4], dtype = torch.float64)  # Initialize minimum cons to positive infinity
min_cons_epoch = 0
max_w00 = -torch.tensor([10**10], dtype = torch.float64)  # Track the max w00
min_cons_w00 = 0  # Save w00 for minimum loss epoch
min_cons_w10 = 0  # Save w10 difference for minimum loss epoch
min_cons_w01 = 0  # Save w01 difference for minimum loss epoch
min_cons_D1 = 0
min_cons_D2 = 0
min_cons_D3 = 0

lr = 0.0001
optimizer = optim.Adam(ImPW.parameters(), lr=lr)  # Adam optimizer with learning rate of 0.001

zero_vec = torch.zeros(len(NullPoints),dtype = torch.float64)
for epoch in range(num_epochs+1):

    w00vals = w00full()
    w01vals = w01full()
    w10vals = w10full()


    losscomp1 = (w01- w01vals)
    losscomp2 = (w10- w10vals)
    losscomp3 =  F.mse_loss(ampD1full(),zero_vec)
    losscomp4 =  F.mse_loss(ampD2full(),zero_vec)
    losscomp5 =  F.mse_loss(ampD3full(),zero_vec)
    beta1 = 10**3
    beta2 = 10**4
    beta3 = 10**3


    cons1 = (losscomp1**2 + losscomp2**2)
    cons2 =  losscomp3
    cons3 =  (losscomp4 + losscomp5)
    cons = torch.max(torch.tensor([cons1,cons2,cons3]))**(1/2)

    loss = - w00vals + (beta1**2)*cons1 + (beta2**2)*cons2 + (beta3**2)*cons3

    if cons < min_cons:
      if w00vals > max_w00:
        min_cons = cons.item()
        min_cons_epoch = epoch
        max_w00 = w00vals.item()
        min_cons_w10 = (losscomp1).item()
        min_cons_w01 = (losscomp2).item()
        min_cons_D1 = (losscomp3**(1/2)).item()
        min_cons_D2 = (losscomp4**(1/2)).item()
        min_cons_D3 = (losscomp5**(1/2)).item()
        min_cons_coeffs = ImPW(ell_n_tensor)

    # Print loss every 100 epochs
    if epoch % 5000 == 0:
        print(f"Epoch {epoch}, Loss_Tot: {loss.item()}, Const: {(losscomp1).item(), (losscomp2).item(), (losscomp3**(1/2)).item(), (losscomp4**(1/2)).item(),(losscomp5**(1/2)).item()}, W00: {w00vals.item()}")

    if epoch % 20000 == 0:
        lr = lr

    # Backward pass and optimization
    optimizer.zero_grad()  # Zero the gradients
    loss.backward()  # Backpropagation
    optimizer.step()  # Update weights

torch.set_printoptions(precision=24)
# Print final values
print("Final values:")
print(w00vals.item(),  (losscomp1).item(), (losscomp2).item(), (losscomp3**(1/2)).item(),(losscomp4**(1/2)).item(),(losscomp5**(1/2)).item())

# Print the minimum loss and the epoch it occurred
print(f"\nMinimum Cons: {min_cons:.24f} at epoch {min_cons_epoch}")
print("Values at minimum cons epoch:")
print(f"W00: {max_w00:.10f}")
print(f"W10 diff: {min_cons_w10:.10f}")
print(f"W01 diff: {min_cons_w01:.10f}")
print(f"Comp3: {min_cons_D1:.16f}")
print(f"Comp4: {min_cons_D2:.16f}")
print(f"Comp4: {min_cons_D3:.16f}")
print(f"Coeffs: {min_cons_coeffs}")

Epoch 0, Loss_Tot: 68233129.22517483, Const: (1.1674487328320668, 0.5917249049853968, 0.8110668219121712, 0.6580096732159172, 0.5299065671685811), W00: -23337.65755859312
Epoch 5000, Loss_Tot: 5442.365726379367, Const: (0.0002938426462983923, -9.49816292672434e-05, 0.005764585489332381, 0.020506633165800307, 0.013287399366147293), W00: -1522.1487892743478
Epoch 10000, Loss_Tot: 385.9998899716486, Const: (7.941014181378492e-05, 0.00014836161681475168, 0.0009059989199992557, 0.01118867514337525, 0.004244600715673072), W00: -160.68508182822453
Epoch 15000, Loss_Tot: 19.664419031718353, Const: (-2.9529890634316658e-05, 4.171397195662507e-05, 0.00020253505386771172, 0.0019594665740422837, 0.0006666271465410856), W00: -11.275861150006877
Epoch 20000, Loss_Tot: 3.8479521883993355, Const: (-5.899845013002114e-05, 5.776731337947538e-05, 7.22768648134789e-05, 0.00031928309370844894, 5.3520328623472234e-05), W00: -3.213933670555915
Epoch 25000, Loss_Tot: 5.084232279753786, Const: (-0.001226441504

In [ ]:
# Define the neural network. Lets try all betas = 4
class SimpleNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(SimpleNN, self).__init__()
        self.input = nn.Linear(input_size, hidden_size, dtype=torch.float64)
        self.hidden1 = nn.Linear(hidden_size, hidden_size, dtype=torch.float64)
        self.hidden2 = nn.Linear(hidden_size, hidden_size, dtype=torch.float64)
        self.output = nn.Linear(hidden_size, output_size, dtype=torch.float64)

    def forward(self, x):
        x = torch.relu(self.input(x))
        x = torch.relu(self.hidden1(x))
        x = torch.relu(self.hidden2(x))
        x = F.softplus(self.output(x))
        return x.squeeze()  # Remove extra dimensions for output

# Create the neural network
input_size = 2  # Since ell_n_tensor has 2 columns
hidden_size = 64
output_size = 1  # Since coeff(ell,n) it a 1D tensor
ImPW = SimpleNN(input_size, hidden_size, output_size)

# Training loop
num_epochs = 90000
min_cons = torch.tensor([10**4], dtype = torch.float64)  # Initialize minimum cons to positive infinity
min_cons_epoch = 0
max_w00 = -torch.tensor([10**10], dtype = torch.float64)  # Track the max w00
min_cons_w00 = 0  # Save w00 for minimum loss epoch
min_cons_w10 = 0  # Save w10 difference for minimum loss epoch
min_cons_w01 = 0  # Save w01 difference for minimum loss epoch
min_cons_D1 = 0
min_cons_D2 = 0
min_cons_D3 = 0

lr = 0.0001
optimizer = optim.Adam(ImPW.parameters(), lr=lr)  # Adam optimizer with learning rate of 0.001

zero_vec = torch.zeros(len(NullPoints),dtype = torch.float64)
for epoch in range(num_epochs+1):

    w00vals = w00full()
    w01vals = w01full()
    w10vals = w10full()


    losscomp1 = (w01- w01vals)
    losscomp2 = (w10- w10vals)
    losscomp3 =  F.mse_loss(ampD1full(),zero_vec)
    losscomp4 =  F.mse_loss(ampD2full(),zero_vec)
    losscomp5 =  F.mse_loss(ampD3full(),zero_vec)
    beta1 = 10**4
    beta2 = 10**4
    beta3 = 10**4


    cons1 = (losscomp1**2 + losscomp2**2)
    cons2 =  losscomp3
    cons3 =  (losscomp4 + losscomp5)
    cons = torch.max(torch.tensor([cons1,cons2,cons3]))**(1/2)

    loss = - w00vals + (beta1**2)*cons1 + (beta2**2)*cons2 + (beta3**2)*cons3

    if cons < min_cons:
      if w00vals > max_w00:
        min_cons = cons.item()
        min_cons_epoch = epoch
        max_w00 = w00vals.item()
        min_cons_w10 = (losscomp1).item()
        min_cons_w01 = (losscomp2).item()
        min_cons_D1 = (losscomp3**(1/2)).item()
        min_cons_D2 = (losscomp4**(1/2)).item()
        min_cons_D3 = (losscomp5**(1/2)).item()
        min_cons_coeffs = ImPW(ell_n_tensor)

    # Print loss every 100 epochs
    if epoch % 5000 == 0:
        print(f"Epoch {epoch}, Loss_Tot: {loss.item()}, Const: {(losscomp1).item(), (losscomp2).item(), (losscomp3**(1/2)).item(), (losscomp4**(1/2)).item(),(losscomp5**(1/2)).item()}, W00: {w00vals.item()}")

    if epoch % 20000 == 0:
        lr = lr

    # Backward pass and optimization
    optimizer.zero_grad()  # Zero the gradients
    loss.backward()  # Backpropagation
    optimizer.step()  # Update weights

torch.set_printoptions(precision=24)
# Print final values
print("Final values:")
print(w00vals.item(),  (losscomp1).item(), (losscomp2).item(), (losscomp3**(1/2)).item(),(losscomp4**(1/2)).item(),(losscomp5**(1/2)).item())

# Print the minimum loss and the epoch it occurred
print(f"\nMinimum Cons: {min_cons:.24f} at epoch {min_cons_epoch}")
print("Values at minimum cons epoch:")
print(f"W00: {max_w00:.10f}")
print(f"W10 diff: {min_cons_w10:.10f}")
print(f"W01 diff: {min_cons_w01:.10f}")
print(f"Comp3: {min_cons_D1:.16f}")
print(f"Comp4: {min_cons_D2:.16f}")
print(f"Comp4: {min_cons_D3:.16f}")
print(f"Coeffs: {min_cons_coeffs}")

Epoch 0, Loss_Tot: 546138476.3959588, Const: (1.1906247815904984, 1.0609012531350908, 1.1778325376330385, 0.9676940884148364, 0.7708685608972735), W00: -32625.087862171327
Epoch 5000, Loss_Tot: 45346.46104565066, Const: (7.249594019764771e-05, 0.00022721494670485676, 0.016562452346417633, 0.009331718466331775, 0.008838796831356523), W00: -1388.7601474924957
Epoch 10000, Loss_Tot: 9451.509373030225, Const: (8.194434054487054e-05, 8.876347186137501e-05, 0.007300177246865143, 0.005466372832349188, 0.002519376074724458), W00: -497.9424317631191
Epoch 15000, Loss_Tot: 1215.7060511412692, Const: (0.0004502728611233131, -0.000387189306626734, 0.002657998862484459, 0.0017661777472336277, 0.0007599385331177603), W00: -104.2550940873325
Epoch 20000, Loss_Tot: 293.10385349318835, Const: (0.00021544867144895896, -0.0002006830304317475, 0.0011426929125840342, 0.0007662900760603066, 0.000543767560062746), W00: -65.57159936846065
Epoch 25000, Loss_Tot: 143.45778513856843, Const: (0.000277010909135677

In [ ]:
for epoch in range(num_epochs+1):

    w00vals = w00full()
    w01vals = w01full()
    w10vals = w10full()


    losscomp1 = (w01- w01vals)
    losscomp2 = (w10- w10vals)
    losscomp3 =  F.mse_loss(ampD1full(),zero_vec)
    losscomp4 =  F.mse_loss(ampD2full(),zero_vec)
    losscomp5 =  F.mse_loss(ampD3full(),zero_vec)
    beta1 = 10**4
    beta2 = 10**4
    beta3 = 10**4


    cons1 = (losscomp1**2 + losscomp2**2)
    cons2 =  losscomp3
    cons3 =  (losscomp4 + losscomp5)
    cons = torch.max(torch.tensor([cons1,cons2,cons3]))**(1/2)

    loss = - w00vals + (beta1**2)*cons1 + (beta2**2)*cons2 + (beta3**2)*cons3

    if cons < min_cons:
      if w00vals > max_w00:
        min_cons = cons.item()
        min_cons_epoch = epoch
        max_w00 = w00vals.item()
        min_cons_w10 = (losscomp1).item()
        min_cons_w01 = (losscomp2).item()
        min_cons_D1 = (losscomp3**(1/2)).item()
        min_cons_D2 = (losscomp4**(1/2)).item()
        min_cons_D3 = (losscomp5**(1/2)).item()
        min_cons_coeffs = ImPW(ell_n_tensor)

    # Print loss every 100 epochs
    if epoch % 5000 == 0:
        print(f"Epoch {epoch}, Loss_Tot: {loss.item()}, Const: {(losscomp1).item(), (losscomp2).item(), (losscomp3**(1/2)).item(), (losscomp4**(1/2)).item(),(losscomp5**(1/2)).item()}, W00: {w00vals.item()}")

    if epoch % 20000 == 0:
        lr = lr

    # Backward pass and optimization
    optimizer.zero_grad()  # Zero the gradients
    loss.backward()  # Backpropagation
    optimizer.step()  # Update weights

torch.set_printoptions(precision=24)
# Print final values
print("Final values:")
print(w00vals.item(),  (losscomp1).item(), (losscomp2).item(), (losscomp3**(1/2)).item(),(losscomp4**(1/2)).item(),(losscomp5**(1/2)).item())

# Print the minimum loss and the epoch it occurred
print(f"\nMinimum Cons: {min_cons:.24f} at epoch {min_cons_epoch}")
print("Values at minimum cons epoch:")
print(f"W00: {max_w00:.10f}")
print(f"W10 diff: {min_cons_w10:.10f}")
print(f"W01 diff: {min_cons_w01:.10f}")
print(f"Comp3: {min_cons_D1:.16f}")
print(f"Comp4: {min_cons_D2:.16f}")
print(f"Comp4: {min_cons_D3:.16f}")
print(f"Coeffs: {min_cons_coeffs}")

Epoch 0, Loss_Tot: 2.9044372754614356, Const: (-2.3765322487712126e-07, -1.6327027840823405e-07, 6.836333437717173e-05, 4.315735123471734e-05, 1.7530966191779432e-05), W00: -2.2200852389996184
Epoch 5000, Loss_Tot: 2.657896223635761, Const: (-3.6276189963047045e-07, -9.545012114031692e-08, 5.984211734423567e-05, 3.8723498263823574e-05, 1.5687344873531558e-05), W00: -2.1252140414225424
Epoch 10000, Loss_Tot: 3.5255695403288905, Const: (-8.171555189595558e-05, 5.8884030568417955e-05, 6.0027001155346394e-05, 3.31589972658032e-05, 1.315340791767152e-05), W00: -2.0235162818419745
Epoch 15000, Loss_Tot: 2.345904237061884, Const: (-1.1739498351204247e-07, -2.6114448403724566e-07, 5.430488836243722e-05, 3.019888543249214e-05, 1.186488424666531e-05), W00: -1.9457191332986823
Epoch 20000, Loss_Tot: 2.1876849646790086, Const: (1.244287812784961e-07, -4.158684077104624e-07, 4.293656565615095e-05, 2.8311674963204598e-05, 1.1262257395435669e-05), W00: -1.9104723166527793
Epoch 25000, Loss_Tot: 2.075

In [ ]:
for epoch in range(num_epochs+1):

    w00vals = w00full()
    w01vals = w01full()
    w10vals = w10full()


    losscomp1 = (w01- w01vals)
    losscomp2 = (w10- w10vals)
    losscomp3 =  F.mse_loss(ampD1full(),zero_vec)
    losscomp4 =  F.mse_loss(ampD2full(),zero_vec)
    losscomp5 =  F.mse_loss(ampD3full(),zero_vec)
    beta1 = 10**4
    beta2 = 10**4
    beta3 = 10**4


    cons1 = (losscomp1**2 + losscomp2**2)
    cons2 =  losscomp3
    cons3 =  (losscomp4 + losscomp5)
    cons = torch.max(torch.tensor([cons1,cons2,cons3]))**(1/2)

    loss = - w00vals + (beta1**2)*cons1 + (beta2**2)*cons2 + (beta3**2)*cons3

    if cons < min_cons:
      if w00vals > max_w00:
        min_cons = cons.item()
        min_cons_epoch = epoch
        max_w00 = w00vals.item()
        min_cons_w10 = (losscomp1).item()
        min_cons_w01 = (losscomp2).item()
        min_cons_D1 = (losscomp3**(1/2)).item()
        min_cons_D2 = (losscomp4**(1/2)).item()
        min_cons_D3 = (losscomp5**(1/2)).item()
        min_cons_coeffs = ImPW(ell_n_tensor)

    # Print loss every 100 epochs
    if epoch % 5000 == 0:
        print(f"Epoch {epoch}, Loss_Tot: {loss.item()}, Const: {(losscomp1).item(), (losscomp2).item(), (losscomp3**(1/2)).item(), (losscomp4**(1/2)).item(),(losscomp5**(1/2)).item()}, W00: {w00vals.item()}")

    if epoch % 20000 == 0:
        lr = lr

    # Backward pass and optimization
    optimizer.zero_grad()  # Zero the gradients
    loss.backward()  # Backpropagation
    optimizer.step()  # Update weights

torch.set_printoptions(precision=24)
# Print final values
print("Final values:")
print(w00vals.item(),  (losscomp1).item(), (losscomp2).item(), (losscomp3**(1/2)).item(),(losscomp4**(1/2)).item(),(losscomp5**(1/2)).item())

# Print the minimum loss and the epoch it occurred
print(f"\nMinimum Cons: {min_cons:.24f} at epoch {min_cons_epoch}")
print("Values at minimum cons epoch:")
print(f"W00: {max_w00:.10f}")
print(f"W10 diff: {min_cons_w10:.10f}")
print(f"W01 diff: {min_cons_w01:.10f}")
print(f"Comp3: {min_cons_D1:.16f}")
print(f"Comp4: {min_cons_D2:.16f}")
print(f"Comp4: {min_cons_D3:.16f}")
print(f"Coeffs: {min_cons_coeffs}")

Epoch 0, Loss_Tot: 1.6353631884403284, Const: (-6.347779168969225e-08, -1.0273456196863151e-07, 8.79765899517082e-06, 1.706155164133721e-05, 5.883072051787231e-06), W00: -1.5950511415611177
Epoch 5000, Loss_Tot: 1.6245851081622178, Const: (-5.695181548581729e-08, -9.156894065220911e-08, 7.867599122434894e-06, 1.7422709432902703e-05, 5.874831363307423e-06), W00: -1.5845875889760004
Epoch 10000, Loss_Tot: 1.607445233294772, Const: (2.4991230707094303e-07, -3.1635544517349956e-07, 6.560231135406822e-06, 1.5374004188634047e-05, 5.368931724074433e-06), W00: -1.5766067730818931
Epoch 15000, Loss_Tot: 1.7270021382637615, Const: (2.648154923035051e-05, -2.096572058118973e-05, 8.434091134962738e-06, 1.6032517929290725e-05, 5.7946133313451885e-06), W00: -1.5767434425423927
Epoch 20000, Loss_Tot: 1.598310971413185, Const: (-1.124282686948419e-07, -3.1773844666460604e-08, 5.514027373529888e-06, 1.5770670239922264e-05, 5.655748781309454e-06), W00: -1.5671990032469352
Epoch 25000, Loss_Tot: 1.590027

In [ ]:
for epoch in range(30000+1):

    w00vals = w00full()
    w01vals = w01full()
    w10vals = w10full()


    losscomp1 = (w01- w01vals)
    losscomp2 = (w10- w10vals)
    losscomp3 =  F.mse_loss(ampD1full(),zero_vec)
    losscomp4 =  F.mse_loss(ampD2full(),zero_vec)
    losscomp5 =  F.mse_loss(ampD3full(),zero_vec)
    beta1 = 10**4
    beta2 = 10**4
    beta3 = 10**4


    cons1 = (losscomp1**2 + losscomp2**2)
    cons2 =  losscomp3
    cons3 =  (losscomp4 + losscomp5)
    cons = torch.max(torch.tensor([cons1,cons2,cons3]))**(1/2)

    loss = - w00vals + (beta1**2)*cons1 + (beta2**2)*cons2 + (beta3**2)*cons3

    if cons < min_cons:
      if w00vals > max_w00:
        min_cons = cons.item()
        min_cons_epoch = epoch
        max_w00 = w00vals.item()
        min_cons_w10 = (losscomp1).item()
        min_cons_w01 = (losscomp2).item()
        min_cons_D1 = (losscomp3**(1/2)).item()
        min_cons_D2 = (losscomp4**(1/2)).item()
        min_cons_D3 = (losscomp5**(1/2)).item()
        min_cons_coeffs = ImPW(ell_n_tensor)

    # Print loss every 100 epochs
    if epoch % 5000 == 0:
        print(f"Epoch {epoch}, Loss_Tot: {loss.item()}, Const: {(losscomp1).item(), (losscomp2).item(), (losscomp3**(1/2)).item(), (losscomp4**(1/2)).item(),(losscomp5**(1/2)).item()}, W00: {w00vals.item()}")

    if epoch % 20000 == 0:
        lr = lr

    # Backward pass and optimization
    optimizer.zero_grad()  # Zero the gradients
    loss.backward()  # Backpropagation
    optimizer.step()  # Update weights

torch.set_printoptions(precision=24)
# Print final values
print("Final values:")
print(w00vals.item(),  (losscomp1).item(), (losscomp2).item(), (losscomp3**(1/2)).item(),(losscomp4**(1/2)).item(),(losscomp5**(1/2)).item())

# Print the minimum loss and the epoch it occurred
print(f"\nMinimum Cons: {min_cons:.24f} at epoch {min_cons_epoch}")
print("Values at minimum cons epoch:")
print(f"W00: {max_w00:.10f}")
print(f"W10 diff: {min_cons_w10:.10f}")
print(f"W01 diff: {min_cons_w01:.10f}")
print(f"Comp3: {min_cons_D1:.16f}")
print(f"Comp4: {min_cons_D2:.16f}")
print(f"Comp4: {min_cons_D3:.16f}")
print(f"Coeffs: {min_cons_coeffs}")

Epoch 0, Loss_Tot: 1.5255999482912919, Const: (-9.941996337303749e-09, -1.125098814114267e-07, 4.573168881872401e-06, 1.4497608835855543e-05, 5.7941093688580954e-06), W00: -1.499132048663832
Epoch 5000, Loss_Tot: 1.51981244738841, Const: (-3.8937995050503105e-08, -5.292278815183238e-08, 3.786477851097411e-06, 1.383409136431575e-05, 5.613933616132475e-06), W00: -1.4960884407855706
Epoch 10000, Loss_Tot: 1.5184729837350925, Const: (-3.9261234929810485e-08, -6.76763485252252e-08, 4.48131963194703e-06, 1.375355786553402e-05, 5.590032334202444e-06), W00: -1.494423267471651
Epoch 15000, Loss_Tot: 58.244408822925884, Const: (0.000612362544751166, -0.00043844733767151034, 7.950459424584181e-06, 1.3904632464543505e-05, 5.563834622677725e-06), W00: -1.4932629239155142
Epoch 20000, Loss_Tot: 1.5109957344395082, Const: (-2.9838154169681275e-08, -5.150773074724668e-08, 4.1768589510934375e-06, 1.2583236819036555e-05, 5.264050198380217e-06), W00: -1.490645957700096
Epoch 25000, Loss_Tot: 1.5091910491

In [ ]:
# Define the neural network. Lets try betas = 3,4,4
class SimpleNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(SimpleNN, self).__init__()
        self.input = nn.Linear(input_size, hidden_size, dtype=torch.float64)
        self.hidden1 = nn.Linear(hidden_size, hidden_size, dtype=torch.float64)
        self.hidden2 = nn.Linear(hidden_size, hidden_size, dtype=torch.float64)
        self.output = nn.Linear(hidden_size, output_size, dtype=torch.float64)

    def forward(self, x):
        x = torch.relu(self.input(x))
        x = torch.relu(self.hidden1(x))
        x = torch.relu(self.hidden2(x))
        x = F.softplus(self.output(x))
        return x.squeeze()  # Remove extra dimensions for output

# Create the neural network
input_size = 2  # Since ell_n_tensor has 2 columns
hidden_size = 64
output_size = 1  # Since coeff(ell,n) it a 1D tensor
ImPW = SimpleNN(input_size, hidden_size, output_size)

# Training loop
num_epochs = 90000
min_cons = torch.tensor([10**4], dtype = torch.float64)  # Initialize minimum cons to positive infinity
min_cons_epoch = 0
max_w00 = -torch.tensor([10**10], dtype = torch.float64)  # Track the max w00
min_cons_w00 = 0  # Save w00 for minimum loss epoch
min_cons_w10 = 0  # Save w10 difference for minimum loss epoch
min_cons_w01 = 0  # Save w01 difference for minimum loss epoch
min_cons_D1 = 0
min_cons_D2 = 0
min_cons_D3 = 0

lr = 0.0001
optimizer = optim.Adam(ImPW.parameters(), lr=lr)  # Adam optimizer with learning rate of 0.001

zero_vec = torch.zeros(len(NullPoints),dtype = torch.float64)
for epoch in range(num_epochs+1):

    w00vals = w00full()
    w01vals = w01full()
    w10vals = w10full()


    losscomp1 = (w01- w01vals)
    losscomp2 = (w10- w10vals)
    losscomp3 =  F.mse_loss(ampD1full(),zero_vec)
    losscomp4 =  F.mse_loss(ampD2full(),zero_vec)
    losscomp5 =  F.mse_loss(ampD3full(),zero_vec)
    beta1 = 10**3
    beta2 = 10**4
    beta3 = 10**4


    cons1 = (losscomp1**2 + losscomp2**2)
    cons2 =  losscomp3
    cons3 =  (losscomp4 + losscomp5)
    cons = torch.max(torch.tensor([cons1,cons2,cons3]))**(1/2)

    loss = - w00vals + (beta1**2)*cons1 + (beta2**2)*cons2 + (beta3**2)*cons3

    if cons < min_cons:
      if w00vals > max_w00:
        min_cons = cons.item()
        min_cons_epoch = epoch
        max_w00 = w00vals.item()
        min_cons_w10 = (losscomp1).item()
        min_cons_w01 = (losscomp2).item()
        min_cons_D1 = (losscomp3**(1/2)).item()
        min_cons_D2 = (losscomp4**(1/2)).item()
        min_cons_D3 = (losscomp5**(1/2)).item()
        min_cons_coeffs = ImPW(ell_n_tensor)

    # Print loss every 100 epochs
    if epoch % 5000 == 0:
        print(f"Epoch {epoch}, Loss_Tot: {loss.item()}, Const: {(losscomp1).item(), (losscomp2).item(), (losscomp3**(1/2)).item(), (losscomp4**(1/2)).item(),(losscomp5**(1/2)).item()}, W00: {w00vals.item()}")

    if epoch % 20000 == 0:
        lr = lr

    # Backward pass and optimization
    optimizer.zero_grad()  # Zero the gradients
    loss.backward()  # Backpropagation
    optimizer.step()  # Update weights

torch.set_printoptions(precision=24)
# Print final values
print("Final values:")
print(w00vals.item(),  (losscomp1).item(), (losscomp2).item(), (losscomp3**(1/2)).item(),(losscomp4**(1/2)).item(),(losscomp5**(1/2)).item())

# Print the minimum loss and the epoch it occurred
print(f"\nMinimum Cons: {min_cons:.24f} at epoch {min_cons_epoch}")
print("Values at minimum cons epoch:")
print(f"W00: {max_w00:.10f}")
print(f"W10 diff: {min_cons_w10:.10f}")
print(f"W01 diff: {min_cons_w01:.10f}")
print(f"Comp3: {min_cons_D1:.16f}")
print(f"Comp4: {min_cons_D2:.16f}")
print(f"Comp5: {min_cons_D3:.16f}")
print(f"Coeffs: {min_cons_coeffs}")

Epoch 0, Loss_Tot: 2941396017.6441875, Const: (1.4342534011555697, 4.366445105599574, 3.7198591671493593, 3.1109470039432052, 2.3846511289179024), W00: -82642.82347095115
Epoch 5000, Loss_Tot: 38784.09907207144, Const: (0.0008743957773795774, 0.0021475232716710035, 0.012218998428155405, 0.009869164063193504, 0.010777598519938493), W00: -2492.6274728460958
Epoch 10000, Loss_Tot: 3064.9544264895294, Const: (3.087353856745523e-05, 0.0005276334701524998, 0.0038354063336861768, 0.0017645192658357345, 0.0013191642201610023), W00: -1108.2686538614698
Epoch 15000, Loss_Tot: 573.2697700101106, Const: (9.473901280387942e-05, 0.00018091525850461032, 0.0010550441956894184, 0.0007634475945098721, 0.00034312845245362887), W00: -391.85730226839735
Epoch 20000, Loss_Tot: 84.79332316229124, Const: (-0.0003776721584214382, 0.0002912179491370015, 0.0004894430555841283, 0.00039448450004367136, 0.0001469367022212299), W00: -42.88958701975658
Epoch 25000, Loss_Tot: 15.892790352022708, Const: (0.000380559570

In [ ]:
#Old best with betas = 4,6,4 and a lot of pain
Values at minimum cons epoch:
W00: -1.5212958279
W10 diff: 0.0000005004
W01 diff: -0.0000006783
Comp3: 0.0000005012250899
Comp4: 0.0000009682926051
Comp4: 0.0000003606152386



#New ones with betas = 3,4,3
Values at minimum cons epoch:
W00: -1.4959695484
W10 diff: 0.0000012536
W01 diff: -0.0000012741
Comp3: 0.0000042690726472
Comp4: 0.0000038353693856
Comp4: 0.0000021966492878


Values at minimum cons epoch:
W00: -1.5118181851
W10 diff: 0.0000006890
W01 diff: 0.0000004421
Comp3: 0.0000022847899544
Comp4: 0.0000020846321734
Comp4: 0.0000009366823393


Values at minimum cons epoch:
W00: -1.5311739895
W10 diff: 0.0000062286
W01 diff: -0.0000030309
Comp3: 0.0000049949470393
Comp4: 0.0000473592149168
Comp4: 0.0000211285612638


Values at minimum cons epoch:
W00: -1.4604879195
W10 diff: 0.0000033676
W01 diff: -0.0000074755
Comp3: 0.0000065669608366
Comp4: 0.0000197875246899
Comp4: 0.0000042336134230


#For all betas =4
Values at minimum cons epoch:
W00: -1.5176843829
W10 diff: 0.0000063800
W01 diff: -0.0000049019
Comp3: 0.0000066182173299
Comp4: 0.0000119047436579
Comp4: 0.0000051224995993



#For betas = 3,4,4
Values at minimum cons epoch:
W00: -1.5112291069
W10 diff: 0.0000073293
W01 diff: -0.0000054770
Comp3: 0.0000065792392911
Comp4: 0.0000108674367635
Comp4: 0.0000054321500157

In [ ]:
#THIS SECTION IS JUST TO TEST THE EXPRESSIONS FOR THE OPEN STRING AMPLITUDE
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import numpy as np
from scipy.special import legendre
from scipy import integrate
from scipy.special import hyp2f1 as scipy_hyp2f1
from scipy.special import gamma as scipy_gamma
from scipy.special import zeta
#This is all for to check formulaes using the Open String Amplitude
#Some definitions
d = 10;
alpha = (d-3)/2;
lm =146/10;
dlm =10**(-1)
lmp = lm + dlm
lmm = lm - dlm
ellmax = 20;
w00 = -(np.pi)**2/6
w10 = -zeta(3)
w01 = 7*np.pi**4/360

def hyp2f1(a, b, c, z):
    if isinstance(z, torch.Tensor):
        z_np = z.detach().numpy()
        result = scipy_hyp2f1(a,b,c,z_np)
        return torch.tensor(result, dtype=z.dtype)
    else:
        return torch.tensor(scipy_hyp2f1(a,b,c,z))

def gamma(x):
    if isinstance(x, torch.Tensor):
        x_np = x.detach().numpy()
        result = scipy_gamma(x_np)
        return torch.tensor(result, dtype=x.dtype)
    else:
        return torch.tensor(scipy_gamma(x))

def ker(n,s1,s2):
    return (1/(s1 - n) + 1/(s2 - n) + 1/(n + lm))

def geg(ell,beta, z):
    return gamma(ell + 2*beta) / (gamma(2*beta) * gamma(ell + 1)) * \
           hyp2f1(-ell, ell + 2*beta, beta + 1/2, (1-z)/2)

#ell,n tensor and Null grid tensor
ell = torch.arange(ellmax + 1, dtype = torch.float64).unsqueeze(1).repeat(1, ellmax + 1)
n = torch.arange(1, ellmax + 2, dtype = torch.float64).unsqueeze(0).repeat(ellmax + 1, 1)
mask = (n > ell) & (n <= ellmax + 1) & ((n - ell - 1) % 2 == 0)
ell_n_tensor = torch.stack([ell[mask], n[mask]], dim=1)

NullPoints = torch.stack(torch.meshgrid(
    torch.arange(-2 +1/2, 2+1/2 + 1e-6, 11/10, dtype=torch.float64),
    torch.arange(-0.1, 0.1 + 1e-6, 2/11, dtype=torch.float64),
    indexing='ij'
)).permute(1, 2, 0).reshape(-1, 2)


def w00comp(ell_n_tensor):
    ell = ell_n_tensor[:, 0].view(-1, 1)
    n = ell_n_tensor[:, 1].view(-1, 1)
    result = (-1/n)*geg(ell,alpha,1)
    return result.squeeze(-1)

def w10comp(ell_n_tensor):
    ell = ell_n_tensor[:, 0].view(-1, 1)  # Shape (k, 1)
    n = ell_n_tensor[:, 1].view(-1, 1)    # Shape (k, 1)
    result = (gamma(-3 + d + ell) * (-((2 * ell * (-3 + d + ell) * lm * (n + 2 * lm) * \
            hyp2f1(1 - ell, -2 + d + ell, d / 2, lm / (n + lm))) / ((-2 + d) * (n + lm)**2)) - \
            hyp2f1(-ell, -3 + d + ell, 1/2 * (-2 + d), lm / (n + lm)))) / (n**2 * gamma(-3 + d) * gamma(1 + ell))
    return result.squeeze(-1)


def w01comp(ell_n_tensor):
    ell = ell_n_tensor[:, 0].view(-1, 1)  # Shape (k, 1)
    n = ell_n_tensor[:, 1].view(-1, 1)    # Shape (k, 1)
    result = (gamma(-3 + d + ell) * (-(1/((d - 2) * (n + lm)**2)) *\
             2 * ell * (-3 + d + ell) * n * (n + 2*lm) *  hyp2f1(1 - ell, -2 + d + ell, d/2, lm/(n + lm)) + \
             2 * hyp2f1(-ell, -3 + d + ell, (d - 2)/2, lm/(n + lm)))) / (n**3 * gamma(-3 + d) * gamma(1 + ell))
    return result.squeeze(-1)

def ampcomp(ell_n_tensor, NullPoints):
    ell = ell_n_tensor[:, 0].view(-1, 1)
    n = ell_n_tensor[:, 1].view(-1, 1)
    s1 = NullPoints[:, 0].view(1, -1)
    s2 = NullPoints[:, 1].view(1, -1)
    result = ker(n, s1, s2) * geg(ell, alpha, 1 + 2 * (((s1 + lm) * (s2 + lm)) / (n + lm) - lm) / n)
    return result.squeeze(-1)

def ampcompD1P(ell_n_tensor, NullPoints):
    ell = ell_n_tensor[:, 0].view(-1, 1)
    n = ell_n_tensor[:, 1].view(-1, 1)
    s1 = NullPoints[:, 0].view(1, -1)
    s2 = NullPoints[:, 1].view(1, -1)
    result = ker(n,s1,s2)*geg(ell,alpha,1 + 2*(((s1 + lmp)*(s2 + lmp))/(n + lmp)-lmp)/n)
    return result.squeeze(-1)

def ampcompD1M(ell_n_tensor, NullPoints):
    ell = ell_n_tensor[:, 0].view(-1, 1)
    n = ell_n_tensor[:, 1].view(-1, 1)
    s1 = NullPoints[:, 0].view(1, -1)
    s2 = NullPoints[:, 1].view(1, -1)
    result = ker(n,s1,s2)*geg(ell,alpha,1 + 2*(((s1 + lmm)*(s2 + lmm))/(n + lmm)-lmm)/n)
    return result.squeeze(-1)


def ampcompD1(ell_n_tensor, NullPoints):
    ell = ell_n_tensor[:, 0].view(-1, 1)
    n = ell_n_tensor[:, 1].view(-1, 1)
    s1 = NullPoints[:, 0].view(1, -1)
    s2 = NullPoints[:, 1].view(1, -1)
    result = ((2 * (-3 + d) * (n**2 + 2 * n * lm - s2 * lm - s1 * (s2 + lm)) * geg(-1 + ell, 1/2 * (-1 + d),
        1 + (2 * ((-n + s2) * lm + s1 * (s2 + lm))) /  (n * (n + lm))) / n - (n + lm) * geg(ell, 1/2 * (-3 + d),
        1 + (2 * ((-n + s2) * lm + s1 * (s2 + lm))) / (n * (n + lm)))) / (n + lm)**3)
    return result.squeeze(-1)

def ampcompD2(ell_n_tensor, NullPoints):
    ell = ell_n_tensor[:, 0].view(-1, 1)
    n = ell_n_tensor[:, 1].view(-1, 1)
    s1 = NullPoints[:, 0].view(1, -1)
    s2 = NullPoints[:, 1].view(1, -1)
    result = (1 / (n + lm)**4 * 2 * (-((2 * (-3 + d) * (-1 + d) * (n - s1) * (n - s2) *
               (n**2 + 2*n*lm - s2*lm - s1*(s2 + lm)) *
               geg(-2 + ell,(d+1)/2, 1 + (2 * ((-n + s2)*lm + s1*(s2 + lm)))/(n*(n + lm)))) /
              (n**2 * (n + lm))) + ((-3 + d) * (n - s1) * (n - s2) *
             geg(-1 + ell,(d-1)/2, 1 + (2 * ((-n + s2)*lm + s1*(s2 + lm)))/(n*(n + lm)))) / n +
            ((-3 + d) * (2*n - s1 - s2) * (n + lm) *
             geg(-1 + ell, (d-1)/2, 1 + (2 * ((-n + s2)*lm + s1*(s2 + lm)))/(n*(n + lm)))) / n -
            (3 * (-3 + d) * (n**2 + 2*n*lm - s2*lm - s1*(s2 + lm)) *
             geg(-1 + ell,(d-1)/2, 1 + (2 * ((-n + s2)*lm + s1*(s2 + lm)))/(n*(n + lm)))) / n +
            (n + lm) * geg(ell,(d-3)/2, 1 + (2 * ((-n + s2)*lm + s1*(s2 + lm)))/(n*(n + lm)))))

    return result.squeeze(-1)


def ampcompD3(ell_n_tensor, NullPoints):
    ell = ell_n_tensor[:, 0].view(-1, 1)
    n = ell_n_tensor[:, 1].view(-1, 1)
    s1 = NullPoints[:, 0].view(1, -1)
    s2 = NullPoints[:, 1].view(1, -1)
    result =  (2 / (n**3 * (n + lm)**7)) * (4 * (-3 + d) * (-1 + d) * (1 + d) * (n - s1)**2 * (n - s2)**2 * (n**2 + 2 * n * lm - s2 * lm -
        s1 * (s2 + lm)) * geg(-3 + ell, (3 + d)/2, 1 + (2 * ((-n + s2) * lm + s1 * (s2 + lm)))/(n * (n + lm))) -
     2 * (-3 + d) * (-1 + d) * n * (n - s1)**2 * (n -s2)**2 * (n + lm) * geg(-2 + ell, (1 + d)/2,
       1 + (2 * ((-n + s2) * lm + s1 * (s2 + lm)))/(n * (n + lm))) -
     4 * (-3 + d) * (-1 + d) * n * (n - s1) * (n - s2) * (2 * n - s1 -s2) * (n + lm)**2 * geg(-2 + ell, (1 + d)/2,
       1 + (2 * ((-n + s2) * lm + s1 * (s2 + lm)))/(n * (n + lm))) +
     8 * (-3 + d) * (-1 + d) * n * (n - s1) * (n - s2) * (n + lm) * (n**2 +2 * n * lm - s2 * lm -
        s1 * (s2 + lm)) * geg(-2 + ell, (1 + d)/2,1 + (2 * ((-n + s2) * lm + s1 * (s2 + lm)))/(n * (n + lm))) -
     2 * (-3 + d) * n**2 * (n - s1) * (n -s2) * (n + lm)**2 * geg(-1 + ell, 1/2 * (-1 + d),
       1 + (2 * ((-n + s2) * lm + s1 * (s2 + lm)))/( n * (n + lm))) -
     2 * (-3 + d) * n**2 * (2 * n - s1 - s2) * (n + lm)**3 * geg(-1 + ell, 1/2 * (-1 + d),1 + (2 * ((-n + s2) * lm + s1 * (s2 + lm)))/(n * (n + lm))) +
     n**3 * (n + lm)**3 * geg(ell, 1/2 * (-3 + d),1 + (2 * ((-n + s2) * lm + s1 * (s2 + lm)))/( n * (n + lm))) +
     4 * n * (n + lm) * (2 * (-3 + d) * (-1 + d) * (n - s1) * (n -s2) * (n**2 + 2 * n * lm - s2 * lm -
     s1 * (s2 + lm)) * geg(-2 + ell, (1 + d)/2,1 + (2 * ((-n + s2) * lm + s1 * (s2 + lm)))/(n * (n + lm)))
     - (-3 + d) * n * (n - s1) * (n -s2) * (n + lm) * geg(-1 + ell, 1/2 * (-1 + d),
      1 + (2 * ((-n + s2) * lm + s1 * (s2 + lm)))/(n * (n + lm))) - (-3 + d) * n * (2 * n - s1 -s2) * (n + lm)**2 * geg(-1 + ell, 1/2 * (-1 + d),
      1 + (2 * ((-n + s2) * lm + s1 * (s2 + lm)))/( n * (n + lm))) +3 * (-3 + d) * n * (n + lm) * (n**2 + 2 * n * lm -
      s2 * lm - s1 * (s2 + lm)) * geg(-1 + ell,1/2 * (-1 + d),1 + (2 * ((-n + s2) * lm + s1 * (s2 + lm)))/( n * (n + lm))) -
        n**2 * (n + lm)**2 * geg(ell, 1/2 * (-3 + d),  1 + (2 * ((-n + s2) * lm + s1 * (s2 + lm)))/(n * (n + lm)))))
    return result.squeeze(-1)

#First, lets calculate cc(ell,n) for open string

def AbsOpString(n,s2):
    return (-1)**(1 + n)*gamma(-s2)/(gamma(n+1)*gamma(1 - n - s2))



def cc(ell_n_tensor):
    z = torch.linspace(-1 + 10**(-6), 1 - 10**(-6), 10**6).view(1, -1)

    # Directly use ell and n from ell_n_tensor
    ell_values = ell_n_tensor[:, 0].view(-1, 1)  # Shape (k, 1)
    n_values = ell_n_tensor[:, 1].view(-1, 1)    # Shape (k, 1)

    # Compute normfac for all combinations of ell and n
    normfac = ((np.pi * 2**(1 - 2 * alpha) * gamma(2 * alpha + ell_values)) /
                (gamma(ell_values + 1) * (ell_values + alpha) * gamma(alpha)**2))**-1

    # Compute integrand
    integrand = normfac * (1 - z**2)**(alpha - 1/2) * \
                AbsOpString(n_values, n_values * (z - 1) / 2) * \
                geg(ell_values, alpha, z)

    # Integrate across z
    return torch.trapz(integrand, z, dim=1)

NullPoints

tensor([[-1.5000, -0.1000],
        [-1.5000,  0.0818],
        [-0.4000, -0.1000],
        [-0.4000,  0.0818],
        [ 0.7000, -0.1000],
        [ 0.7000,  0.0818],
        [ 1.8000, -0.1000],
        [ 1.8000,  0.0818]], dtype=torch.float64)

In [ ]:
#Store all the values
coeff_vals = cc(ell_n_tensor)
w00comp_vals = w00comp(ell_n_tensor)
w01comp_vals = w01comp(ell_n_tensor)
w10comp_vals = w10comp(ell_n_tensor)

ampcomp_vals = ampcomp(ell_n_tensor, NullPoints)
ampcompD1P_vals = ampcompD1P(ell_n_tensor, NullPoints)
ampcompD1M_vals = ampcompD1M(ell_n_tensor, NullPoints)
ampcompD1_vals = ampcompD1(ell_n_tensor, NullPoints)
ampcompD2_vals = ampcompD2(ell_n_tensor, NullPoints)
ampcompD3_vals = ampcompD3(ell_n_tensor, NullPoints)

<ipython-input-2-740bb5865f5a>:29: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return torch.tensor(result, dtype=z.dtype)
<ipython-input-2-740bb5865f5a>:31: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return torch.tensor(scipy_hyp2f1(a,b,c,z))


In [ ]:
#CHECKS FOR OPEN STRING
def w00fullX():
    return torch.dot(coeff_vals,w00comp_vals)

def w10fullX():
    return torch.dot(coeff_vals,w10comp_vals)

def w01fullX():
    return torch.dot(coeff_vals,w01comp_vals)

def ampfullX():
    return torch.sum(coeff_vals.view(-1, 1) * ampcomp_vals, dim = 0)

def ampD1PfullX():
    return torch.sum(coeff_vals.view(-1, 1) * ampcompD1P_vals, dim = 0)

def ampD1MfullX():
    return torch.sum(coeff_vals.view(-1, 1) * ampcompD1M_vals, dim = 0)

def ampD1fullX():
    return torch.sum(coeff_vals.view(-1, 1) * ampcompD1_vals, dim = 0)

def ampD2fullX():
    return torch.sum(coeff_vals.view(-1, 1) * ampcompD2_vals, dim = 0)

def ampD3fullX():
    return torch.sum(coeff_vals.view(-1, 1) * ampcompD3_vals, dim = 0)

(w00, w00fullX().item(),w10, w10fullX().item(),w01, w01fullX().item(),ampfullX()[1].item(),
 ampD1fullX()[1].item(),ampD2fullX()[1].item(),ampD3fullX()[1].item(),
 ampD1PfullX()[1].item(),ampD1MfullX()[1].item(),(ampD1PfullX()/ampD1MfullX())[1].item())

(-1.6449340668482264,
 -1.5984308179417641,
 -1.2020569031595942,
 -1.202056902362787,
 1.8940656589944915,
 1.894065658958795,
 -0.9368392530438512,
 1.4470927928167399e-11,
 7.384788822117905e-10,
 -3.6782704044614235e-10,
 -0.9366382712809055,
 -0.9370430272245681,
 0.9995680497779686)

In [ ]:
# Same definitions for the PINN
def w00full():
    return torch.dot(ImPW(ell_n_tensor),w00comp_vals)

def w01full():
    return torch.dot(ImPW(ell_n_tensor),w01comp_vals)

def w10full():
    return torch.dot(ImPW(ell_n_tensor),w10comp_vals)

def ampfull():
    return torch.sum(ImPW(ell_n_tensor).view(-1, 1) * ampcomp_vals, dim = 0)

def ampD1Pfull():
    return torch.sum(ImPW(ell_n_tensor).view(-1, 1) * ampcompD1P_vals, dim = 0)

def ampD1Mfull():
    return torch.sum(ImPW(ell_n_tensor).view(-1, 1) * ampcompD1M_vals, dim = 0)

def ampD1full():
    return torch.sum(ImPW(ell_n_tensor).view(-1, 1) * ampcompD1_vals, dim = 0)

def ampD2full():
    return torch.sum(ImPW(ell_n_tensor).view(-1, 1) * ampcompD2_vals, dim = 0)

def ampD3full():
    return torch.sum(ImPW(ell_n_tensor).view(-1, 1) * ampcompD3_vals, dim = 0)

In [ ]:
# Define the neural network. Now lets impose null constraints as well
class SimpleNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(SimpleNN, self).__init__()
        self.input = nn.Linear(input_size, hidden_size, dtype=torch.float64)
        self.hidden1 = nn.Linear(hidden_size, hidden_size, dtype=torch.float64)
        self.hidden2 = nn.Linear(hidden_size, hidden_size, dtype=torch.float64)
        self.output = nn.Linear(hidden_size, output_size, dtype=torch.float64)

    def forward(self, x):
        x = torch.relu(self.input(x))
        x = torch.relu(self.hidden1(x))
        x = torch.relu(self.hidden2(x))
        x = F.softplus(self.output(x))
        return x.squeeze()  # Remove extra dimensions for output

# Create the neural network
input_size = 2  # Since ell_n_tensor has 2 columns
hidden_size = 64
output_size = 1  # Since coeff(ell,n) it a 1D tensor
ImPW = SimpleNN(input_size, hidden_size, output_size)

# Training loop
num_epochs = 90000
min_cons = torch.tensor([10**4], dtype = torch.float64)  # Initialize minimum cons to positive infinity
min_cons_epoch = 0
max_w00 = -torch.tensor([10**10], dtype = torch.float64)  # Track the max w00
min_cons_w00 = 0  # Save w00 for minimum loss epoch
min_cons_w10 = 0  # Save w10 difference for minimum loss epoch
min_cons_w01 = 0  # Save w01 difference for minimum loss epoch
min_cons_D1 = 0
min_cons_D2 = 0
min_cons_D3 = 0

lr = 0.0001
optimizer = optim.Adam(ImPW.parameters(), lr=lr)  # Adam optimizer with learning rate of 0.001

zero_vec = torch.zeros(len(NullPoints),dtype = torch.float64)
for epoch in range(num_epochs+1):

    w00vals = w00full()
    w01vals = w01full()
    w10vals = w10full()


    losscomp1 = (w01- w01vals)
    losscomp2 = (w10- w10vals)
    losscomp3 =  F.mse_loss(ampD1full(),zero_vec)
    losscomp4 =  F.mse_loss(ampD2full(),zero_vec)
    losscomp5 =  F.mse_loss(ampD3full(),zero_vec)
    beta1 = 10**3
    beta2 = 10**4
    beta3 = 10**3


    cons1 = (losscomp1**2 + losscomp2**2)
    cons2 =  losscomp3
    cons3 =  (losscomp4 + losscomp5)
    cons = torch.max(torch.tensor([cons1,cons2,cons3]))**(1/2)

    loss = - w00vals + (beta1**2)*cons1 + (beta2**2)*cons2 + (beta3**2)*cons3

    if cons < min_cons:
      if w00vals > max_w00:
        min_cons = cons.item()
        min_cons_epoch = epoch
        max_w00 = w00vals.item()
        min_cons_w10 = (losscomp1).item()
        min_cons_w01 = (losscomp2).item()
        min_cons_D1 = (losscomp3**(1/2)).item()
        min_cons_D2 = (losscomp4**(1/2)).item()
        min_cons_D3 = (losscomp5**(1/2)).item()
        min_cons_coeffs = ImPW(ell_n_tensor)

    # Print loss every 100 epochs
    if epoch % 5000 == 0:
        print(f"Epoch {epoch}, Loss_Tot: {loss.item()}, Const: {(losscomp1).item(), (losscomp2).item(), (losscomp3**(1/2)).item(), (losscomp4**(1/2)).item(),(losscomp5**(1/2)).item()}, W00: {w00vals.item()}")

    if epoch % 20000 == 0:
        lr = lr

    # Backward pass and optimization
    optimizer.zero_grad()  # Zero the gradients
    loss.backward()  # Backpropagation
    optimizer.step()  # Update weights

torch.set_printoptions(precision=24)
# Print final values
print("Final values:")
print(w00vals.item(),  (losscomp1).item(), (losscomp2).item(), (losscomp3**(1/2)).item(),(losscomp4**(1/2)).item(),(losscomp5**(1/2)).item())

# Print the minimum loss and the epoch it occurred
print(f"\nMinimum Cons: {min_cons:.24f} at epoch {min_cons_epoch}")
print("Values at minimum cons epoch:")
print(f"W00: {max_w00:.10f}")
print(f"W10 diff: {min_cons_w10:.10f}")
print(f"W01 diff: {min_cons_w01:.10f}")
print(f"Comp3: {min_cons_D1:.16f}")
print(f"Comp4: {min_cons_D2:.16f}")
print(f"Comp4: {min_cons_D3:.16f}")
print(f"Coeffs: {min_cons_coeffs}")

Epoch 0, Loss_Tot: 3032880943.758614, Const: (1.5600017283286223, 6.706810379854307, 5.433765075937293, 4.5648051004082735, 3.45460394646817), W00: -114008.81565555999
Epoch 5000, Loss_Tot: 9683.746061569747, Const: (2.0157613816795816e-05, 0.0006048677075822795, 0.007980506770757356, 0.018938274624205928, 0.01923553068864023), W00: -2585.867071870814
Epoch 10000, Loss_Tot: 1005.5874288836735, Const: (-3.3224187916403025e-05, 0.00016980997199067538, 0.0020254186292869543, 0.00704965418344302, 0.0068216996199992564), W00: -499.0922174125252
Epoch 15000, Loss_Tot: 22.623251916459118, Const: (0.00011848952604021079, 5.276037207369022e-05, 0.00033697343887477394, 0.0001229042439029586, 6.385920810902182e-05), W00: -11.23213518947776
Epoch 20000, Loss_Tot: 8.441014955692493, Const: (-0.0014983734133482418, 0.0010571909200212115, 0.0001268891661327194, 5.1696699540969436e-05, 3.788216323509383e-05), W00: -3.464045773267852
Epoch 25000, Loss_Tot: 2.1528507967245933, Const: (1.3487150787971913

# With some null constraints, and multi-lambda equality constraints (Using product of cons as the strategy), FINAL

In [ ]:
#THIS SECTION IS JUST TO TEST THE EXPRESSIONS FOR THE OPEN STRING AMPLITUDE
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import numpy as np
from scipy.special import legendre
from scipy import integrate
from scipy.special import hyp2f1 as scipy_hyp2f1
from scipy.special import gamma as scipy_gamma
from scipy.special import zeta
#This is all for to check formulaes using the Open String Amplitude
#Some definitions
d = 10;
alpha = (d-3)/2;
lm = 56/10;
dlm = 1/2
lmp = lm + dlm
lmm = lm - dlm
ellmax = 12;
w00 = -(np.pi)**2/6
w10 = -zeta(3)
w01 = 7*np.pi**4/360

torch.set_printoptions(precision=12)

def hyp2f1(a, b, c, z):
    if isinstance(z, torch.Tensor):
        z_np = z.detach().numpy()
        result = scipy_hyp2f1(a,b,c,z_np)
        return torch.tensor(result, dtype=z.dtype)
    else:
        return torch.tensor(scipy_hyp2f1(a,b,c,z))

def gamma(x):
    if isinstance(x, torch.Tensor):
        x_np = x.detach().numpy()
        result = scipy_gamma(x_np)
        return torch.tensor(result, dtype=x.dtype)
    else:
        return torch.tensor(scipy_gamma(x))

def ker(lm,n,s1,s2):
    return (1/(s1 - n) + 1/(s2 - n) + 1/(n + lm))

def geg(ell,beta, z):
    return gamma(ell + 2*beta) / (gamma(2*beta) * gamma(ell + 1)) * \
           hyp2f1(-ell, ell + 2*beta, beta + 1/2, (1-z)/2)

#ell,n tensor and Null grid tensor
ell = torch.arange(ellmax + 1, dtype = torch.float64).unsqueeze(1).repeat(1, ellmax + 1)
n = torch.arange(1, ellmax + 2, dtype = torch.float64).unsqueeze(0).repeat(ellmax + 1, 1)
mask = (n > ell) & (n <= ellmax + 1) & ((n - ell - 1) % 2 == 0)
ell_n_tensor = torch.stack([ell[mask], n[mask]], dim=1)

NullPoints = torch.stack(torch.meshgrid(
    torch.arange(-6 +1/2, 5+1/2 + 1e-6, 101/100, dtype=torch.float64),
    torch.arange(-0.2, 0.2 + 1e-6, 4/11, dtype=torch.float64),
    indexing='ij'
)).permute(1, 2, 0).reshape(-1, 2)


def w00comp(ell_n_tensor):
    ell = ell_n_tensor[:, 0].view(-1, 1)
    n = ell_n_tensor[:, 1].view(-1, 1)
    result = (-1/n)*geg(ell,alpha,1)
    return result.squeeze(-1)

def w10comp(ell_n_tensor):
    ell = ell_n_tensor[:, 0].view(-1, 1)  # Shape (k, 1)
    n = ell_n_tensor[:, 1].view(-1, 1)    # Shape (k, 1)
    result = (gamma(-3 + d + ell) * (-((2 * ell * (-3 + d + ell) * lm * (n + 2 * lm) * \
            hyp2f1(1 - ell, -2 + d + ell, d / 2, lm / (n + lm))) / ((-2 + d) * (n + lm)**2)) - \
            hyp2f1(-ell, -3 + d + ell, 1/2 * (-2 + d), lm / (n + lm)))) / (n**2 * gamma(-3 + d) * gamma(1 + ell))
    return result.squeeze(-1)


def w01comp(ell_n_tensor):
    ell = ell_n_tensor[:, 0].view(-1, 1)  # Shape (k, 1)
    n = ell_n_tensor[:, 1].view(-1, 1)    # Shape (k, 1)
    result = (gamma(-3 + d + ell) * (-(1/((d - 2) * (n + lm)**2)) *\
             2 * ell * (-3 + d + ell) * n * (n + 2*lm) *  hyp2f1(1 - ell, -2 + d + ell, d/2, lm/(n + lm)) + \
             2 * hyp2f1(-ell, -3 + d + ell, (d - 2)/2, lm/(n + lm)))) / (n**3 * gamma(-3 + d) * gamma(1 + ell))
    return result.squeeze(-1)

def ampcomp(ell_n_tensor, NullPoints):
    ell = ell_n_tensor[:, 0].view(-1, 1)
    n = ell_n_tensor[:, 1].view(-1, 1)
    s1 = NullPoints[:, 0].view(1, -1)
    s2 = NullPoints[:, 1].view(1, -1)
    result = ker(lm,n, s1, s2) * geg(ell, alpha, 1 + 2 * (((s1 + lm) * (s2 + lm)) / (n + lm) - lm) / n)
    return result.squeeze(-1)

def ampcompD1P(ell_n_tensor, NullPoints):
    ell = ell_n_tensor[:, 0].view(-1, 1)
    n = ell_n_tensor[:, 1].view(-1, 1)
    s1 = NullPoints[:, 0].view(1, -1)
    s2 = NullPoints[:, 1].view(1, -1)
    result = ker(lmp,n,s1,s2)*geg(ell,alpha,1 + 2*(((s1 + lmp)*(s2 + lmp))/(n + lmp)-lmp)/n)
    return result.squeeze(-1)

def ampcompD1M(ell_n_tensor, NullPoints):
    ell = ell_n_tensor[:, 0].view(-1, 1)
    n = ell_n_tensor[:, 1].view(-1, 1)
    s1 = NullPoints[:, 0].view(1, -1)
    s2 = NullPoints[:, 1].view(1, -1)
    result = ker(lmm,n,s1,s2)*geg(ell,alpha,1 + 2*(((s1 + lmm)*(s2 + lmm))/(n + lmm)-lmm)/n)
    return result.squeeze(-1)


def ampcompD1(ell_n_tensor, NullPoints):
    ell = ell_n_tensor[:, 0].view(-1, 1)
    n = ell_n_tensor[:, 1].view(-1, 1)
    s1 = NullPoints[:, 0].view(1, -1)
    s2 = NullPoints[:, 1].view(1, -1)
    result = ((2 * (-3 + d) * (n**2 + 2 * n * lm - s2 * lm - s1 * (s2 + lm)) * geg(-1 + ell, 1/2 * (-1 + d),
        1 + (2 * ((-n + s2) * lm + s1 * (s2 + lm))) /  (n * (n + lm))) / n - (n + lm) * geg(ell, 1/2 * (-3 + d),
        1 + (2 * ((-n + s2) * lm + s1 * (s2 + lm))) / (n * (n + lm)))) / (n + lm)**3)
    return result.squeeze(-1)

def ampcompD2(ell_n_tensor, NullPoints):
    ell = ell_n_tensor[:, 0].view(-1, 1)
    n = ell_n_tensor[:, 1].view(-1, 1)
    s1 = NullPoints[:, 0].view(1, -1)
    s2 = NullPoints[:, 1].view(1, -1)
    result = (1 / (n + lm)**4 * 2 * (-((2 * (-3 + d) * (-1 + d) * (n - s1) * (n - s2) *
               (n**2 + 2*n*lm - s2*lm - s1*(s2 + lm)) *
               geg(-2 + ell,(d+1)/2, 1 + (2 * ((-n + s2)*lm + s1*(s2 + lm)))/(n*(n + lm)))) /
              (n**2 * (n + lm))) + ((-3 + d) * (n - s1) * (n - s2) *
             geg(-1 + ell,(d-1)/2, 1 + (2 * ((-n + s2)*lm + s1*(s2 + lm)))/(n*(n + lm)))) / n +
            ((-3 + d) * (2*n - s1 - s2) * (n + lm) *
             geg(-1 + ell, (d-1)/2, 1 + (2 * ((-n + s2)*lm + s1*(s2 + lm)))/(n*(n + lm)))) / n -
            (3 * (-3 + d) * (n**2 + 2*n*lm - s2*lm - s1*(s2 + lm)) *
             geg(-1 + ell,(d-1)/2, 1 + (2 * ((-n + s2)*lm + s1*(s2 + lm)))/(n*(n + lm)))) / n +
            (n + lm) * geg(ell,(d-3)/2, 1 + (2 * ((-n + s2)*lm + s1*(s2 + lm)))/(n*(n + lm)))))

    return result.squeeze(-1)


def ampcompD3(ell_n_tensor, NullPoints):
    ell = ell_n_tensor[:, 0].view(-1, 1)
    n = ell_n_tensor[:, 1].view(-1, 1)
    s1 = NullPoints[:, 0].view(1, -1)
    s2 = NullPoints[:, 1].view(1, -1)
    result =  (2 / (n**3 * (n + lm)**7)) * (4 * (-3 + d) * (-1 + d) * (1 + d) * (n - s1)**2 * (n - s2)**2 * (n**2 + 2 * n * lm - s2 * lm -
        s1 * (s2 + lm)) * geg(-3 + ell, (3 + d)/2, 1 + (2 * ((-n + s2) * lm + s1 * (s2 + lm)))/(n * (n + lm))) -
     2 * (-3 + d) * (-1 + d) * n * (n - s1)**2 * (n -s2)**2 * (n + lm) * geg(-2 + ell, (1 + d)/2,
       1 + (2 * ((-n + s2) * lm + s1 * (s2 + lm)))/(n * (n + lm))) -
     4 * (-3 + d) * (-1 + d) * n * (n - s1) * (n - s2) * (2 * n - s1 -s2) * (n + lm)**2 * geg(-2 + ell, (1 + d)/2,
       1 + (2 * ((-n + s2) * lm + s1 * (s2 + lm)))/(n * (n + lm))) +
     8 * (-3 + d) * (-1 + d) * n * (n - s1) * (n - s2) * (n + lm) * (n**2 +2 * n * lm - s2 * lm -
        s1 * (s2 + lm)) * geg(-2 + ell, (1 + d)/2,1 + (2 * ((-n + s2) * lm + s1 * (s2 + lm)))/(n * (n + lm))) -
     2 * (-3 + d) * n**2 * (n - s1) * (n -s2) * (n + lm)**2 * geg(-1 + ell, 1/2 * (-1 + d),
       1 + (2 * ((-n + s2) * lm + s1 * (s2 + lm)))/( n * (n + lm))) -
     2 * (-3 + d) * n**2 * (2 * n - s1 - s2) * (n + lm)**3 * geg(-1 + ell, 1/2 * (-1 + d),1 + (2 * ((-n + s2) * lm + s1 * (s2 + lm)))/(n * (n + lm))) +
     n**3 * (n + lm)**3 * geg(ell, 1/2 * (-3 + d),1 + (2 * ((-n + s2) * lm + s1 * (s2 + lm)))/( n * (n + lm))) +
     4 * n * (n + lm) * (2 * (-3 + d) * (-1 + d) * (n - s1) * (n -s2) * (n**2 + 2 * n * lm - s2 * lm -
     s1 * (s2 + lm)) * geg(-2 + ell, (1 + d)/2,1 + (2 * ((-n + s2) * lm + s1 * (s2 + lm)))/(n * (n + lm)))
     - (-3 + d) * n * (n - s1) * (n -s2) * (n + lm) * geg(-1 + ell, 1/2 * (-1 + d),
      1 + (2 * ((-n + s2) * lm + s1 * (s2 + lm)))/(n * (n + lm))) - (-3 + d) * n * (2 * n - s1 -s2) * (n + lm)**2 * geg(-1 + ell, 1/2 * (-1 + d),
      1 + (2 * ((-n + s2) * lm + s1 * (s2 + lm)))/( n * (n + lm))) +3 * (-3 + d) * n * (n + lm) * (n**2 + 2 * n * lm -
      s2 * lm - s1 * (s2 + lm)) * geg(-1 + ell,1/2 * (-1 + d),1 + (2 * ((-n + s2) * lm + s1 * (s2 + lm)))/( n * (n + lm))) -
        n**2 * (n + lm)**2 * geg(ell, 1/2 * (-3 + d),  1 + (2 * ((-n + s2) * lm + s1 * (s2 + lm)))/(n * (n + lm)))))
    return result.squeeze(-1)

#First, lets calculate cc(ell,n) for open string

def AbsOpString(n,s2):
    return (-1)**(1 + n)*gamma(-s2)/(gamma(n+1)*gamma(1 - n - s2))



def cc(ell_n_tensor):
    z = torch.linspace(-1 + 10**(-6), 1 - 10**(-6), 10**6).view(1, -1)

    # Directly use ell and n from ell_n_tensor
    ell_values = ell_n_tensor[:, 0].view(-1, 1)  # Shape (k, 1)
    n_values = ell_n_tensor[:, 1].view(-1, 1)    # Shape (k, 1)

    # Compute normfac for all combinations of ell and n
    normfac = ((np.pi * 2**(1 - 2 * alpha) * gamma(2 * alpha + ell_values)) /
                (gamma(ell_values + 1) * (ell_values + alpha) * gamma(alpha)**2))**-1

    # Compute integrand
    integrand = normfac * (1 - z**2)**(alpha - 1/2) * \
                AbsOpString(n_values, n_values * (z - 1) / 2) * \
                geg(ell_values, alpha, z)

    # Integrate across z
    return torch.trapz(integrand, z, dim=1)

NullPoints

tensor([[-5.500000000000, -0.200000000000],
        [-5.500000000000,  0.163636363636],
        [-4.490000000000, -0.200000000000],
        [-4.490000000000,  0.163636363636],
        [-3.480000000000, -0.200000000000],
        [-3.480000000000,  0.163636363636],
        [-2.470000000000, -0.200000000000],
        [-2.470000000000,  0.163636363636],
        [-1.460000000000, -0.200000000000],
        [-1.460000000000,  0.163636363636],
        [-0.450000000000, -0.200000000000],
        [-0.450000000000,  0.163636363636],
        [ 0.560000000000, -0.200000000000],
        [ 0.560000000000,  0.163636363636],
        [ 1.570000000000, -0.200000000000],
        [ 1.570000000000,  0.163636363636],
        [ 2.580000000000, -0.200000000000],
        [ 2.580000000000,  0.163636363636],
        [ 3.590000000000, -0.200000000000],
        [ 3.590000000000,  0.163636363636],
        [ 4.600000000000, -0.200000000000],
        [ 4.600000000000,  0.163636363636]], dtype=torch.float64)

In [ ]:
#Store all the values
coeff_vals = cc(ell_n_tensor)
w00comp_vals = w00comp(ell_n_tensor)
w01comp_vals = w01comp(ell_n_tensor)
w10comp_vals = w10comp(ell_n_tensor)

ampcomp_vals = ampcomp(ell_n_tensor, NullPoints)
ampcompD1P_vals = ampcompD1P(ell_n_tensor, NullPoints)
ampcompD1M_vals = ampcompD1M(ell_n_tensor, NullPoints)
ampcompD1_vals = ampcompD1(ell_n_tensor, NullPoints)
ampcompD2_vals = ampcompD2(ell_n_tensor, NullPoints)
ampcompD3_vals = ampcompD3(ell_n_tensor, NullPoints)

<ipython-input-28-1ea80a7cb81f>:31: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return torch.tensor(result, dtype=z.dtype)
<ipython-input-28-1ea80a7cb81f>:33: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return torch.tensor(scipy_hyp2f1(a,b,c,z))


In [ ]:
#CHECKS FOR OPEN STRING
def w00fullX():
    return torch.dot(coeff_vals,w00comp_vals)

def w10fullX():
    return torch.dot(coeff_vals,w10comp_vals)

def w01fullX():
    return torch.dot(coeff_vals,w01comp_vals)

def ampfullX():
    return torch.sum(coeff_vals.view(-1, 1) * ampcomp_vals, dim = 0)

def ampD1PfullX():
    return torch.sum(coeff_vals.view(-1, 1) * ampcompD1P_vals, dim = 0)

def ampD1MfullX():
    return torch.sum(coeff_vals.view(-1, 1) * ampcompD1M_vals, dim = 0)

def ampD1fullX():
    return torch.sum(coeff_vals.view(-1, 1) * ampcompD1_vals, dim = 0)

def ampD2fullX():
    return torch.sum(coeff_vals.view(-1, 1) * ampcompD2_vals, dim = 0)

def ampD3fullX():
    return torch.sum(coeff_vals.view(-1, 1) * ampcompD3_vals, dim = 0)

(w00, w00fullX().item(),w10, w10fullX().item(),w01, w01fullX().item(),ampfullX()[1].item(),
 ampD1fullX()[1].item(),ampD2fullX()[1].item(),ampD3fullX()[1].item(),
 ampD1PfullX()[1].item(),ampD1MfullX()[1].item(),(ampD1PfullX()/ampD1MfullX())[1].item())

(-1.6449340668482264,
 -1.5708937983031839,
 -1.2020569031595942,
 -1.2020580619524968,
 1.8940656589944915,
 1.8940654448676988,
 -0.5624502228752933,
 3.839619282262798e-07,
 1.7739561382799387e-06,
 -3.90493232622108e-06,
 -0.5624498928853159,
 -0.5624501257869979,
 0.9999995859158504)

In [ ]:
# Same definitions for the PINN
def w00full():
    return torch.dot(ImPW(ell_n_tensor),w00comp_vals)

def w01full():
    return torch.dot(ImPW(ell_n_tensor),w01comp_vals)

def w10full():
    return torch.dot(ImPW(ell_n_tensor),w10comp_vals)

def ampfull():
    return torch.sum(ImPW(ell_n_tensor).view(-1, 1) * ampcomp_vals, dim = 0)

def ampD1Pfull():
    return torch.sum(ImPW(ell_n_tensor).view(-1, 1) * ampcompD1P_vals, dim = 0)

def ampD1Mfull():
    return torch.sum(ImPW(ell_n_tensor).view(-1, 1) * ampcompD1M_vals, dim = 0)

def ampD1full():
    return torch.sum(ImPW(ell_n_tensor).view(-1, 1) * ampcompD1_vals, dim = 0)

def ampD2full():
    return torch.sum(ImPW(ell_n_tensor).view(-1, 1) * ampcompD2_vals, dim = 0)

def ampD3full():
    return torch.sum(ImPW(ell_n_tensor).view(-1, 1) * ampcompD3_vals, dim = 0)

In [ ]:
# Define the neural network. We will impose multi-lambda equality. But lets first check how well
# this is satisfied with three derivatives imposed.
class SimpleNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(SimpleNN, self).__init__()
        self.input = nn.Linear(input_size, hidden_size, dtype=torch.float64)
        self.hidden1 = nn.Linear(hidden_size, hidden_size, dtype=torch.float64)
        self.hidden2 = nn.Linear(hidden_size, hidden_size, dtype=torch.float64)
        self.output = nn.Linear(hidden_size, output_size, dtype=torch.float64)

    def forward(self, x):
        x = torch.relu(self.input(x))
        x = torch.relu(self.hidden1(x))
        x = torch.relu(self.hidden2(x))
        x = F.softplus(self.output(x))
        return x.squeeze()  # Remove extra dimensions for output

# Create the neural network
input_size = 2  # Since ell_n_tensor has 2 columns
hidden_size = 64
output_size = 1  # Since coeff(ell,n) it a 1D tensor
ImPW = SimpleNN(input_size, hidden_size, output_size)
lr = 0.0001
optimizer = optim.Adam(ImPW.parameters(), lr=lr)  # Adam optimizer with learning rate of 0.0001

zero_vec = torch.zeros(len(NullPoints),dtype = torch.float64)
one_vec = torch.ones(len(NullPoints),dtype = torch.float64)

# Training loop
epoch_part = 20000
num_epochs = 4*epoch_part
min_cons = torch.tensor([10**4], dtype = torch.float64)  # Initialize minimum cons to positive infinity
min_cons_epoch = 0
max_w00 = -torch.tensor([10**10], dtype = torch.float64)  # Track the max w00
min_cons_w00 = 0  # Save w00 for minimum loss epoch
min_cons_w10 = 0  # Save w10 difference for minimum loss epoch
min_cons_w01 = 0  # Save w01 difference for minimum loss epoch
min_cons_D1 = 0
min_cons_D2 = 0
min_cons_D3 = 0
min_cons_L6 = 0


zero_vec = torch.zeros(len(NullPoints),dtype = torch.float64)
one_vec = torch.ones(len(NullPoints),dtype = torch.float64)
for epoch in range(num_epochs+1):

    w00vals = w00full()
    w01vals = w01full()
    w10vals = w10full()


    losscomp1 = (w01- w01vals)
    losscomp2 = (w10- w10vals)
    losscomp3 =  F.mse_loss(ampD1full(),zero_vec)
    losscomp4 =  F.mse_loss(ampD2full(),zero_vec)
    losscomp5 =  F.mse_loss(ampD3full(),zero_vec)
    losscomp6 =  F.mse_loss(ampD1Pfull()/ampD1Mfull(),one_vec)
    beta1 = 10**2
    beta4 = 10**2


    cons1 = (losscomp1**2 + losscomp2**2)
    cons2 =  losscomp3
    cons3 =  (losscomp4 + losscomp5)
    cons4 = losscomp6
    cons = (cons1**(1/2))*(cons4**(1/2))

    loss = - w00vals + (beta1**2)*cons1 + (beta4**2)*cons4

    if cons < min_cons:
      if w00vals > max_w00:
        min_cons = cons.item()
        min_cons_epoch = epoch
        max_w00 = w00vals.item()
        min_cons_w10 = (losscomp1).item()
        min_cons_w01 = (losscomp2).item()
        min_cons_D1 = (losscomp3**(1/2)).item()
        min_cons_D2 = (losscomp4**(1/2)).item()
        min_cons_D3 = (losscomp5**(1/2)).item()
        min_cons_L6 = (losscomp6**(1/2)).item()
        min_cons_coeffs = ImPW(ell_n_tensor)

    # Print loss every 5000 epochs
    if epoch % 5000 == 0:
        print(f"Epoch {epoch}, Loss_Tot: {loss.item()}, Const: {(losscomp1).item(), (losscomp2).item(), (losscomp3**(1/2)).item(), (losscomp4**(1/2)).item(),(losscomp5**(1/2)).item(),(losscomp6**(1/2)).item()}, W00: {w00vals.item()}")
     # Print best solution every 100000 epochs
    if (epoch+1) % (epoch_part) == 0:
      print(f"\nMinimum Cons: {min_cons:.24f} at epoch {min_cons_epoch}")
      print("Values at minimum cons epoch:")
      print(f"W00: {max_w00:.10f}")
      print(f"W10 diff: {min_cons_w10:.10f}")
      print(f"W01 diff: {min_cons_w01:.10f}")
      print(f"Comp3: {min_cons_D1:.16f}")
      print(f"Comp4: {min_cons_D2:.16f}")
      print(f"Comp5: {min_cons_D3:.16f}")
      print(f"Comp6: {min_cons_L6:.16f}")
      print(f"Coeffs: {min_cons_coeffs}")

      min_cons = torch.tensor([10**4], dtype = torch.float64)  # Initialize minimum cons to positive infinity
      min_cons_epoch = 0
      max_w00 = -torch.tensor([10**10], dtype = torch.float64)  # Track the max w00
      min_cons_w00 = 0  # Save w00 for minimum loss epoch
      min_cons_w10 = 0  # Save w10 difference for minimum loss epoch
      min_cons_w01 = 0  # Save w01 difference for minimum loss epoch
      min_cons_D1 = 0
      min_cons_D2 = 0
      min_cons_D3 = 0
      min_cons_L6 = 0

    # Backward pass and optimization
    optimizer.zero_grad()  # Zero the gradients
    loss.backward()  # Backpropagation
    optimizer.step()  # Update weights

Epoch 0, Loss_Tot: 341834.0335534883, Const: (0.22679816104075412, -4.990971892851955, 14.89791886876163, 9.916554164021845, 10.598252882842647, 2.9450139159638673), W00: -5490.58549038305
Epoch 5000, Loss_Tot: 52800.41271418599, Const: (1.776044296713453, -1.198060700082214, 0.18094685140837283, 0.051474251257841865, 0.022703352463995165, 0.8304963737081668), W00: -6.342597063143899
Epoch 10000, Loss_Tot: 46673.023786144746, Const: (1.760393392897039, -1.1981046291599193, 0.22370780409556054, 0.06453945665339879, 0.028678682138519734, 0.3633116594688823), W00: -8.674165384645763
Epoch 15000, Loss_Tot: 25512.846579283083, Const: (1.3358172997715696, -0.8125491321644088, 0.6368299114479896, 0.19006545865669758, 0.08520202541119537, 0.32085997317559056), W00: -36.89584991926194

Minimum Cons: 0.660597258573675572002060 at epoch 4647
Values at minimum cons epoch:
W00: -10.4930245297
W10 diff: 1.7682111945
W01 diff: -1.2135229421
Comp3: 0.2496805220549490
Comp4: 0.0737482991100804
Comp5: 0

In [ ]:
# Training loop
epoch_part = 20000
num_epochs = 2*epoch_part
min_cons = torch.tensor([10**4], dtype = torch.float64)  # Initialize minimum cons to positive infinity
min_cons_epoch = 0
max_w00 = -torch.tensor([10**10], dtype = torch.float64)  # Track the max w00
min_cons_w00 = 0  # Save w00 for minimum loss epoch
min_cons_w10 = 0  # Save w10 difference for minimum loss epoch
min_cons_w01 = 0  # Save w01 difference for minimum loss epoch
min_cons_D1 = 0
min_cons_D2 = 0
min_cons_D3 = 0
min_cons_L6 = 0


zero_vec = torch.zeros(len(NullPoints),dtype = torch.float64)
one_vec = torch.ones(len(NullPoints),dtype = torch.float64)
for epoch in range(num_epochs+1):

    w00vals = w00full()
    w01vals = w01full()
    w10vals = w10full()


    losscomp1 = (w01- w01vals)
    losscomp2 = (w10- w10vals)
    losscomp3 =  F.mse_loss(ampD1full(),zero_vec)
    losscomp4 =  F.mse_loss(ampD2full(),zero_vec)
    losscomp5 =  F.mse_loss(ampD3full(),zero_vec)
    losscomp6 =  F.mse_loss(ampD1Pfull()/ampD1Mfull(),one_vec)
    beta1 = 10**2
    beta4 = 10**2


    cons1 = (losscomp1**2 + losscomp2**2)
    cons2 =  losscomp3
    cons3 =  (losscomp4 + losscomp5)
    cons4 = losscomp6
    cons = (cons1**(1/2))*(cons4**(1/2))

    loss = - w00vals + (beta1**2)*cons1 + (beta4**2)*cons4

    if cons < min_cons:
      if w00vals > max_w00:
        min_cons = cons.item()
        min_cons_epoch = epoch
        max_w00 = w00vals.item()
        min_cons_w10 = (losscomp1).item()
        min_cons_w01 = (losscomp2).item()
        min_cons_D1 = (losscomp3**(1/2)).item()
        min_cons_D2 = (losscomp4**(1/2)).item()
        min_cons_D3 = (losscomp5**(1/2)).item()
        min_cons_L6 = (losscomp6**(1/2)).item()
        min_cons_coeffs = ImPW(ell_n_tensor)

    # Print loss every 5000 epochs
    if epoch % 5000 == 0:
        print(f"Epoch {epoch}, Loss_Tot: {loss.item()}, Const: {(losscomp1).item(), (losscomp2).item(), (losscomp3**(1/2)).item(), (losscomp4**(1/2)).item(),(losscomp5**(1/2)).item(),(losscomp6**(1/2)).item()}, W00: {w00vals.item()}")
     # Print best solution every 100000 epochs
    if (epoch+1) % (epoch_part) == 0:
      print(f"\nMinimum Cons: {min_cons:.24f} at epoch {min_cons_epoch}")
      print("Values at minimum cons epoch:")
      print(f"W00: {max_w00:.10f}")
      print(f"W10 diff: {min_cons_w10:.10f}")
      print(f"W01 diff: {min_cons_w01:.10f}")
      print(f"Comp3: {min_cons_D1:.16f}")
      print(f"Comp4: {min_cons_D2:.16f}")
      print(f"Comp5: {min_cons_D3:.16f}")
      print(f"Comp6: {min_cons_L6:.16f}")
      print(f"Coeffs: {min_cons_coeffs}")

      min_cons = torch.tensor([10**4], dtype = torch.float64)  # Initialize minimum cons to positive infinity
      min_cons_epoch = 0
      max_w00 = -torch.tensor([10**10], dtype = torch.float64)  # Track the max w00
      min_cons_w00 = 0  # Save w00 for minimum loss epoch
      min_cons_w10 = 0  # Save w10 difference for minimum loss epoch
      min_cons_w01 = 0  # Save w01 difference for minimum loss epoch
      min_cons_D1 = 0
      min_cons_D2 = 0
      min_cons_D3 = 0
      min_cons_L6 = 0

    # Backward pass and optimization
    optimizer.zero_grad()  # Zero the gradients
    loss.backward()  # Backpropagation
    optimizer.step()  # Update weights

Epoch 0, Loss_Tot: 1.5492168815899061, Const: (0.0002749127028447429, -0.0003767118823196913, 0.0003284241872289632, 0.0005215938616544739, 0.000620932793352143, 0.0006533818313309842), W00: -1.5427729150501095
Epoch 5000, Loss_Tot: 1.551239505337353, Const: (0.00048113463009236135, -0.0005662474536110018, 0.00033981447701311095, 0.0005750269624871248, 0.0006603907002091663, 0.000667816419364008), W00: -1.541258450527681
Epoch 10000, Loss_Tot: 1.546118847902478, Const: (-8.228296329182427e-05, -3.328869339314089e-05, 0.0003706741784292797, 0.0005935079135301533, 0.0006925614730263987, 0.0006839796480041044), W00: -1.541361780082081
Epoch 15000, Loss_Tot: 1.5450670773388095, Const: (-0.00017804064299031808, 5.2571607609186444e-05, 0.00039282780478444676, 0.0006362299765821588, 0.0007411037978903055, 0.0007096387847567669), W00: -1.5396865828456687

Minimum Cons: 0.000000002503893295150918 at epoch 18513
Values at minimum cons epoch:
W00: -1.5399421222
W10 diff: 0.0000034630
W01 diff: 0.

In [ ]:
# Training loop
epoch_part = 20000
num_epochs = 2*epoch_part
min_cons = torch.tensor([10**4], dtype = torch.float64)  # Initialize minimum cons to positive infinity
min_cons_epoch = 0
max_w00 = -torch.tensor([10**10], dtype = torch.float64)  # Track the max w00
min_cons_w00 = 0  # Save w00 for minimum loss epoch
min_cons_w10 = 0  # Save w10 difference for minimum loss epoch
min_cons_w01 = 0  # Save w01 difference for minimum loss epoch
min_cons_D1 = 0
min_cons_D2 = 0
min_cons_D3 = 0
min_cons_L6 = 0


zero_vec = torch.zeros(len(NullPoints),dtype = torch.float64)
one_vec = torch.ones(len(NullPoints),dtype = torch.float64)
for epoch in range(num_epochs+1):

    w00vals = w00full()
    w01vals = w01full()
    w10vals = w10full()


    losscomp1 = (w01- w01vals)
    losscomp2 = (w10- w10vals)
    losscomp3 =  F.mse_loss(ampD1full(),zero_vec)
    losscomp4 =  F.mse_loss(ampD2full(),zero_vec)
    losscomp5 =  F.mse_loss(ampD3full(),zero_vec)
    losscomp6 =  F.mse_loss(ampD1Pfull()/ampD1Mfull(),one_vec)
    beta1 = 10**2
    beta4 = 10**2


    cons1 = (losscomp1**2 + losscomp2**2)
    cons2 =  losscomp3
    cons3 =  (losscomp4 + losscomp5)
    cons4 = losscomp6
    cons = (cons1**(1/2))*(cons4**(1/2))

    loss = - w00vals + (beta1**2)*cons1 + (beta4**2)*cons4

    if cons < min_cons:
      if w00vals > max_w00:
        min_cons = cons.item()
        min_cons_epoch = epoch
        max_w00 = w00vals.item()
        min_cons_w10 = (losscomp1).item()
        min_cons_w01 = (losscomp2).item()
        min_cons_D1 = (losscomp3**(1/2)).item()
        min_cons_D2 = (losscomp4**(1/2)).item()
        min_cons_D3 = (losscomp5**(1/2)).item()
        min_cons_L6 = (losscomp6**(1/2)).item()
        min_cons_coeffs = ImPW(ell_n_tensor)

    # Print loss every 5000 epochs
    if epoch % 5000 == 0:
        print(f"Epoch {epoch}, Loss_Tot: {loss.item()}, Const: {(losscomp1).item(), (losscomp2).item(), (losscomp3**(1/2)).item(), (losscomp4**(1/2)).item(),(losscomp5**(1/2)).item(),(losscomp6**(1/2)).item()}, W00: {w00vals.item()}")
     # Print best solution every 100000 epochs
    if (epoch+1) % (epoch_part) == 0:
      print(f"\nMinimum Cons: {min_cons:.24f} at epoch {min_cons_epoch}")
      print("Values at minimum cons epoch:")
      print(f"W00: {max_w00:.10f}")
      print(f"W10 diff: {min_cons_w10:.10f}")
      print(f"W01 diff: {min_cons_w01:.10f}")
      print(f"Comp3: {min_cons_D1:.16f}")
      print(f"Comp4: {min_cons_D2:.16f}")
      print(f"Comp5: {min_cons_D3:.16f}")
      print(f"Comp6: {min_cons_L6:.16f}")
      print(f"Coeffs: {min_cons_coeffs}")

      min_cons = torch.tensor([10**4], dtype = torch.float64)  # Initialize minimum cons to positive infinity
      min_cons_epoch = 0
      max_w00 = -torch.tensor([10**10], dtype = torch.float64)  # Track the max w00
      min_cons_w00 = 0  # Save w00 for minimum loss epoch
      min_cons_w10 = 0  # Save w10 difference for minimum loss epoch
      min_cons_w01 = 0  # Save w01 difference for minimum loss epoch
      min_cons_D1 = 0
      min_cons_D2 = 0
      min_cons_D3 = 0
      min_cons_L6 = 0

    # Backward pass and optimization
    optimizer.zero_grad()  # Zero the gradients
    loss.backward()  # Backpropagation
    optimizer.step()  # Update weights

Epoch 0, Loss_Tot: 1.5498305205866723, Const: (-0.0006702324052028086, 0.00048158839522716157, 0.00045418775370276187, 0.0007841381520005983, 0.0009133735561466511, 0.0008071493786082759), W00: -1.5365042307987808
Epoch 5000, Loss_Tot: 1.5569483352814342, Const: (-0.0009100440782778829, 0.0007506890579946912, 0.00048449500791958945, 0.0008345487937472614, 0.0009701743192856769, 0.0008111886421651941), W00: -1.5364509222876401
Epoch 10000, Loss_Tot: 1.8231990711288457, Const: (-4.9628095678988515e-05, -0.0001373302320211156, 0.0020623886042948456, 0.0015770777124773326, 0.0016027583150132845, 0.005724190679839145), W00: -1.4953222563321955
Epoch 15000, Loss_Tot: 1.7827462603017277, Const: (-4.241554295991712e-05, -0.00012777266132912857, 0.0017073403047778825, 0.0014598777736435974, 0.0012876185626245706, 0.005357779238838889), W00: -1.4955070272677202

Minimum Cons: 0.000000050270080597097319 at epoch 19638
Values at minimum cons epoch:
W00: -1.5156175985
W10 diff: -0.0000272379
W01 di

In [ ]:
# Training loop
epoch_part = 20000
num_epochs = 2*epoch_part
min_cons = torch.tensor([10**4], dtype = torch.float64)  # Initialize minimum cons to positive infinity
min_cons_epoch = 0
max_w00 = -torch.tensor([10**10], dtype = torch.float64)  # Track the max w00
min_cons_w00 = 0  # Save w00 for minimum loss epoch
min_cons_w10 = 0  # Save w10 difference for minimum loss epoch
min_cons_w01 = 0  # Save w01 difference for minimum loss epoch
min_cons_D1 = 0
min_cons_D2 = 0
min_cons_D3 = 0
min_cons_L6 = 0


zero_vec = torch.zeros(len(NullPoints),dtype = torch.float64)
one_vec = torch.ones(len(NullPoints),dtype = torch.float64)
for epoch in range(num_epochs+1):

    w00vals = w00full()
    w01vals = w01full()
    w10vals = w10full()


    losscomp1 = (w01- w01vals)
    losscomp2 = (w10- w10vals)
    losscomp3 =  F.mse_loss(ampD1full(),zero_vec)
    losscomp4 =  F.mse_loss(ampD2full(),zero_vec)
    losscomp5 =  F.mse_loss(ampD3full(),zero_vec)
    losscomp6 =  F.mse_loss(ampD1Pfull()/ampD1Mfull(),one_vec)
    beta1 = 10**2
    beta4 = 10**2


    cons1 = (losscomp1**2 + losscomp2**2)
    cons2 =  losscomp3
    cons3 =  (losscomp4 + losscomp5)
    cons4 = losscomp6
    cons = (cons1**(1/2))*(cons4**(1/2))

    loss = - w00vals + (beta1**2)*cons1 + (beta4**2)*cons4

    if cons < min_cons:
      if w00vals > max_w00:
        min_cons = cons.item()
        min_cons_epoch = epoch
        max_w00 = w00vals.item()
        min_cons_w10 = (losscomp1).item()
        min_cons_w01 = (losscomp2).item()
        min_cons_D1 = (losscomp3**(1/2)).item()
        min_cons_D2 = (losscomp4**(1/2)).item()
        min_cons_D3 = (losscomp5**(1/2)).item()
        min_cons_L6 = (losscomp6**(1/2)).item()
        min_cons_coeffs = ImPW(ell_n_tensor)

    # Print loss every 5000 epochs
    if epoch % 5000 == 0:
        print(f"Epoch {epoch}, Loss_Tot: {loss.item()}, Const: {(losscomp1).item(), (losscomp2).item(), (losscomp3**(1/2)).item(), (losscomp4**(1/2)).item(),(losscomp5**(1/2)).item(),(losscomp6**(1/2)).item()}, W00: {w00vals.item()}")
     # Print best solution every 100000 epochs
    if (epoch+1) % (epoch_part) == 0:
      print(f"\nMinimum Cons: {min_cons:.24f} at epoch {min_cons_epoch}")
      print("Values at minimum cons epoch:")
      print(f"W00: {max_w00:.10f}")
      print(f"W10 diff: {min_cons_w10:.10f}")
      print(f"W01 diff: {min_cons_w01:.10f}")
      print(f"Comp3: {min_cons_D1:.16f}")
      print(f"Comp4: {min_cons_D2:.16f}")
      print(f"Comp5: {min_cons_D3:.16f}")
      print(f"Comp6: {min_cons_L6:.16f}")
      print(f"Coeffs: {min_cons_coeffs}")

      min_cons = torch.tensor([10**4], dtype = torch.float64)  # Initialize minimum cons to positive infinity
      min_cons_epoch = 0
      max_w00 = -torch.tensor([10**10], dtype = torch.float64)  # Track the max w00
      min_cons_w00 = 0  # Save w00 for minimum loss epoch
      min_cons_w10 = 0  # Save w10 difference for minimum loss epoch
      min_cons_w01 = 0  # Save w01 difference for minimum loss epoch
      min_cons_D1 = 0
      min_cons_D2 = 0
      min_cons_D3 = 0
      min_cons_L6 = 0

    # Backward pass and optimization
    optimizer.zero_grad()  # Zero the gradients
    loss.backward()  # Backpropagation
    optimizer.step()  # Update weights

Epoch 0, Loss_Tot: 1.5181678110290284, Const: (-0.0003989440875591388, 0.0003119786153116255, 0.0003326123265898206, 0.0004469424472744337, 0.0006145542443029353, 0.0008926947766853837), W00: -1.5076339009717132
Epoch 5000, Loss_Tot: 1.5426086121671359, Const: (-0.0010941706612088709, 0.0008372349732261863, 0.0003150029629509764, 0.0004136957651763728, 0.0005962707908549695, 0.0012346969837197586), W00: -1.5083821273886362
Epoch 10000, Loss_Tot: 1.5264134111596583, Const: (0.0007549073720320987, -0.0006514260895003066, 0.0003268598460253454, 0.00038387546096520534, 0.0005749518553034035, 0.0010677151351602312), W00: -1.5050708441568554
Epoch 15000, Loss_Tot: 1.5151652313071682, Const: (-0.0001228999105817863, 0.00011527815049561241, 0.00034126944153142985, 0.00042855771901023594, 0.000569793728655396, 0.0008514729609372689), W00: -1.5076312348750684

Minimum Cons: 0.000000119465702296387439 at epoch 852
Values at minimum cons epoch:
W00: -1.5058554152
W10 diff: 0.0000659313
W01 diff: -

In [ ]:
solTensor = torch.tensor([9.993667472672e-01, 1.346706541218e-02, 6.401434284304e-09,
        1.875733941401e-14, 6.377081992302e-20, 3.057888592934e-25,
        1.472855700503e-30, 7.145293988770e-02, 4.324204708594e-05,
        2.271398388500e-10, 9.276835460277e-16, 4.687418072893e-21,
        2.876423055655e-26, 1.207753591305e-02, 1.524076702881e-06,
        1.957959273852e-11, 2.118198457604e-16, 1.050828347060e-21,
        5.595759864956e-27, 3.002819915860e-03, 1.842276191040e-07,
        2.225452592778e-12, 2.830674500808e-17, 2.986760413211e-22,
        9.220474891228e-04, 1.888464034004e-08, 3.708219112511e-13,
        3.500283284190e-18, 4.443037398398e-23, 3.319760219835e-04,
        2.462606791682e-09, 3.528472557117e-14, 6.574658560685e-19,
        8.203064522224e-05, 1.465015544226e-10, 2.935151492456e-15,
        7.183060352062e-20, 4.219570932112e-06, 1.074704582353e-11,
        1.318289890497e-16, 2.170423228737e-07, 8.729138761279e-13,
        8.012748103926e-18, 1.116399777401e-08, 7.090121765973e-14,
        5.742420762826e-10, 5.758852967178e-15, 2.953726498269e-11,
        1.519307028284e-12], dtype=torch.float64)

def CoeffFuncSol(solTensor, ell, n):
    solTensor = solTensor.unsqueeze(1)  # Shape [49, 1]
    combined_tensor = torch.cat((ell_n_tensor, solTensor), dim=1)
    mask = (combined_tensor[:, 0] == ell) & (combined_tensor[:, 1] == n)
    # Check if there is an entry
    if mask.any():
        return combined_tensor[mask, 2].item()
    else:
        return None

CoeffFuncSol(solTensor,0,1),CoeffFuncSol(solTensor,1,2),CoeffFuncSol(solTensor,2,3),CoeffFuncSol(solTensor,3,4),CoeffFuncSol(solTensor,4,5),CoeffFuncSol(solTensor,5,6)

0.9993667472672

In [ ]:
CoeffFuncSol(solTensor,0,1),CoeffFuncSol(solTensor,1,2),CoeffFuncSol(solTensor,2,3),CoeffFuncSol(solTensor,3,4),CoeffFuncSol(solTensor,4,5),CoeffFuncSol(solTensor,5,6)

(0.9993667472672,
 0.0714529398877,
 0.01207753591305,
 0.00300281991586,
 0.0009220474891228,
 0.0003319760219835)

# New section With some null constraints, and multi-lambda equality constraints (Using product of cons as the strategy), FINAL IMPROVEMENTS

In [ ]:
#THIS SECTION IS JUST TO TEST THE EXPRESSIONS FOR THE OPEN STRING AMPLITUDE
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import numpy as np
from scipy.special import legendre
from scipy import integrate
from scipy.special import hyp2f1 as scipy_hyp2f1
from scipy.special import gamma as scipy_gamma
from scipy.special import zeta
#This is all for to check formulaes using the Open String Amplitude
#Some definitions
d = 10;
alpha = (d-3)/2;
lm = 56/10;
dlm = 1/2
lmp = lm + dlm
lmm = lm - dlm
ellmax = 12;
w00 = -(np.pi)**2/6
w10 = -zeta(3)
w01 = 7*np.pi**4/360

torch.set_printoptions(precision=12)

def hyp2f1(a, b, c, z):
    if isinstance(z, torch.Tensor):
        z_np = z.detach().numpy()
        result = scipy_hyp2f1(a,b,c,z_np)
        return torch.tensor(result, dtype=z.dtype)
    else:
        return torch.tensor(scipy_hyp2f1(a,b,c,z))

def gamma(x):
    if isinstance(x, torch.Tensor):
        x_np = x.detach().numpy()
        result = scipy_gamma(x_np)
        return torch.tensor(result, dtype=x.dtype)
    else:
        return torch.tensor(scipy_gamma(x))

def ker(lm,n,s1,s2):
    return (1/(s1 - n) + 1/(s2 - n) + 1/(n + lm))

def geg(ell,beta, z):
    return gamma(ell + 2*beta) / (gamma(2*beta) * gamma(ell + 1)) * \
           hyp2f1(-ell, ell + 2*beta, beta + 1/2, (1-z)/2)

#ell,n tensor and Null grid tensor
ell = torch.arange(ellmax + 1, dtype = torch.float64).unsqueeze(1).repeat(1, ellmax + 1)
n = torch.arange(1, ellmax + 2, dtype = torch.float64).unsqueeze(0).repeat(ellmax + 1, 1)
mask = (n > ell) & (n <= ellmax + 1) & ((n - ell - 1) % 2 == 0)
ell_n_tensor = torch.stack([ell[mask], n[mask]], dim=1)

NullPoints = torch.stack(torch.meshgrid(
    torch.arange(-6 +1/2, 5+1/2 + 1e-6, 101/100, dtype=torch.float64),
    torch.arange(-0.2, 0.2 + 1e-6, 4/11, dtype=torch.float64),
    indexing='ij'
)).permute(1, 2, 0).reshape(-1, 2)


def w00comp(ell_n_tensor):
    ell = ell_n_tensor[:, 0].view(-1, 1)
    n = ell_n_tensor[:, 1].view(-1, 1)
    result = (-1/n)*geg(ell,alpha,1)
    return result.squeeze(-1)

def w10comp(ell_n_tensor):
    ell = ell_n_tensor[:, 0].view(-1, 1)  # Shape (k, 1)
    n = ell_n_tensor[:, 1].view(-1, 1)    # Shape (k, 1)
    result = (gamma(-3 + d + ell) * (-((2 * ell * (-3 + d + ell) * lm * (n + 2 * lm) * \
            hyp2f1(1 - ell, -2 + d + ell, d / 2, lm / (n + lm))) / ((-2 + d) * (n + lm)**2)) - \
            hyp2f1(-ell, -3 + d + ell, 1/2 * (-2 + d), lm / (n + lm)))) / (n**2 * gamma(-3 + d) * gamma(1 + ell))
    return result.squeeze(-1)


def w01comp(ell_n_tensor):
    ell = ell_n_tensor[:, 0].view(-1, 1)  # Shape (k, 1)
    n = ell_n_tensor[:, 1].view(-1, 1)    # Shape (k, 1)
    result = (gamma(-3 + d + ell) * (-(1/((d - 2) * (n + lm)**2)) *\
             2 * ell * (-3 + d + ell) * n * (n + 2*lm) *  hyp2f1(1 - ell, -2 + d + ell, d/2, lm/(n + lm)) + \
             2 * hyp2f1(-ell, -3 + d + ell, (d - 2)/2, lm/(n + lm)))) / (n**3 * gamma(-3 + d) * gamma(1 + ell))
    return result.squeeze(-1)

def ampcomp(ell_n_tensor, NullPoints):
    ell = ell_n_tensor[:, 0].view(-1, 1)
    n = ell_n_tensor[:, 1].view(-1, 1)
    s1 = NullPoints[:, 0].view(1, -1)
    s2 = NullPoints[:, 1].view(1, -1)
    result = ker(lm,n, s1, s2) * geg(ell, alpha, 1 + 2 * (((s1 + lm) * (s2 + lm)) / (n + lm) - lm) / n)
    return result.squeeze(-1)

def ampcompD1P(ell_n_tensor, NullPoints):
    ell = ell_n_tensor[:, 0].view(-1, 1)
    n = ell_n_tensor[:, 1].view(-1, 1)
    s1 = NullPoints[:, 0].view(1, -1)
    s2 = NullPoints[:, 1].view(1, -1)
    result = ker(lmp,n,s1,s2)*geg(ell,alpha,1 + 2*(((s1 + lmp)*(s2 + lmp))/(n + lmp)-lmp)/n)
    return result.squeeze(-1)

def ampcompD1M(ell_n_tensor, NullPoints):
    ell = ell_n_tensor[:, 0].view(-1, 1)
    n = ell_n_tensor[:, 1].view(-1, 1)
    s1 = NullPoints[:, 0].view(1, -1)
    s2 = NullPoints[:, 1].view(1, -1)
    result = ker(lmm,n,s1,s2)*geg(ell,alpha,1 + 2*(((s1 + lmm)*(s2 + lmm))/(n + lmm)-lmm)/n)
    return result.squeeze(-1)


def ampcompD1(ell_n_tensor, NullPoints):
    ell = ell_n_tensor[:, 0].view(-1, 1)
    n = ell_n_tensor[:, 1].view(-1, 1)
    s1 = NullPoints[:, 0].view(1, -1)
    s2 = NullPoints[:, 1].view(1, -1)
    result = ((2 * (-3 + d) * (n**2 + 2 * n * lm - s2 * lm - s1 * (s2 + lm)) * geg(-1 + ell, 1/2 * (-1 + d),
        1 + (2 * ((-n + s2) * lm + s1 * (s2 + lm))) /  (n * (n + lm))) / n - (n + lm) * geg(ell, 1/2 * (-3 + d),
        1 + (2 * ((-n + s2) * lm + s1 * (s2 + lm))) / (n * (n + lm)))) / (n + lm)**3)
    return result.squeeze(-1)

def ampcompD2(ell_n_tensor, NullPoints):
    ell = ell_n_tensor[:, 0].view(-1, 1)
    n = ell_n_tensor[:, 1].view(-1, 1)
    s1 = NullPoints[:, 0].view(1, -1)
    s2 = NullPoints[:, 1].view(1, -1)
    result = (1 / (n + lm)**4 * 2 * (-((2 * (-3 + d) * (-1 + d) * (n - s1) * (n - s2) *
               (n**2 + 2*n*lm - s2*lm - s1*(s2 + lm)) *
               geg(-2 + ell,(d+1)/2, 1 + (2 * ((-n + s2)*lm + s1*(s2 + lm)))/(n*(n + lm)))) /
              (n**2 * (n + lm))) + ((-3 + d) * (n - s1) * (n - s2) *
             geg(-1 + ell,(d-1)/2, 1 + (2 * ((-n + s2)*lm + s1*(s2 + lm)))/(n*(n + lm)))) / n +
            ((-3 + d) * (2*n - s1 - s2) * (n + lm) *
             geg(-1 + ell, (d-1)/2, 1 + (2 * ((-n + s2)*lm + s1*(s2 + lm)))/(n*(n + lm)))) / n -
            (3 * (-3 + d) * (n**2 + 2*n*lm - s2*lm - s1*(s2 + lm)) *
             geg(-1 + ell,(d-1)/2, 1 + (2 * ((-n + s2)*lm + s1*(s2 + lm)))/(n*(n + lm)))) / n +
            (n + lm) * geg(ell,(d-3)/2, 1 + (2 * ((-n + s2)*lm + s1*(s2 + lm)))/(n*(n + lm)))))

    return result.squeeze(-1)


def ampcompD3(ell_n_tensor, NullPoints):
    ell = ell_n_tensor[:, 0].view(-1, 1)
    n = ell_n_tensor[:, 1].view(-1, 1)
    s1 = NullPoints[:, 0].view(1, -1)
    s2 = NullPoints[:, 1].view(1, -1)
    result =  (2 / (n**3 * (n + lm)**7)) * (4 * (-3 + d) * (-1 + d) * (1 + d) * (n - s1)**2 * (n - s2)**2 * (n**2 + 2 * n * lm - s2 * lm -
        s1 * (s2 + lm)) * geg(-3 + ell, (3 + d)/2, 1 + (2 * ((-n + s2) * lm + s1 * (s2 + lm)))/(n * (n + lm))) -
     2 * (-3 + d) * (-1 + d) * n * (n - s1)**2 * (n -s2)**2 * (n + lm) * geg(-2 + ell, (1 + d)/2,
       1 + (2 * ((-n + s2) * lm + s1 * (s2 + lm)))/(n * (n + lm))) -
     4 * (-3 + d) * (-1 + d) * n * (n - s1) * (n - s2) * (2 * n - s1 -s2) * (n + lm)**2 * geg(-2 + ell, (1 + d)/2,
       1 + (2 * ((-n + s2) * lm + s1 * (s2 + lm)))/(n * (n + lm))) +
     8 * (-3 + d) * (-1 + d) * n * (n - s1) * (n - s2) * (n + lm) * (n**2 +2 * n * lm - s2 * lm -
        s1 * (s2 + lm)) * geg(-2 + ell, (1 + d)/2,1 + (2 * ((-n + s2) * lm + s1 * (s2 + lm)))/(n * (n + lm))) -
     2 * (-3 + d) * n**2 * (n - s1) * (n -s2) * (n + lm)**2 * geg(-1 + ell, 1/2 * (-1 + d),
       1 + (2 * ((-n + s2) * lm + s1 * (s2 + lm)))/( n * (n + lm))) -
     2 * (-3 + d) * n**2 * (2 * n - s1 - s2) * (n + lm)**3 * geg(-1 + ell, 1/2 * (-1 + d),1 + (2 * ((-n + s2) * lm + s1 * (s2 + lm)))/(n * (n + lm))) +
     n**3 * (n + lm)**3 * geg(ell, 1/2 * (-3 + d),1 + (2 * ((-n + s2) * lm + s1 * (s2 + lm)))/( n * (n + lm))) +
     4 * n * (n + lm) * (2 * (-3 + d) * (-1 + d) * (n - s1) * (n -s2) * (n**2 + 2 * n * lm - s2 * lm -
     s1 * (s2 + lm)) * geg(-2 + ell, (1 + d)/2,1 + (2 * ((-n + s2) * lm + s1 * (s2 + lm)))/(n * (n + lm)))
     - (-3 + d) * n * (n - s1) * (n -s2) * (n + lm) * geg(-1 + ell, 1/2 * (-1 + d),
      1 + (2 * ((-n + s2) * lm + s1 * (s2 + lm)))/(n * (n + lm))) - (-3 + d) * n * (2 * n - s1 -s2) * (n + lm)**2 * geg(-1 + ell, 1/2 * (-1 + d),
      1 + (2 * ((-n + s2) * lm + s1 * (s2 + lm)))/( n * (n + lm))) +3 * (-3 + d) * n * (n + lm) * (n**2 + 2 * n * lm -
      s2 * lm - s1 * (s2 + lm)) * geg(-1 + ell,1/2 * (-1 + d),1 + (2 * ((-n + s2) * lm + s1 * (s2 + lm)))/( n * (n + lm))) -
        n**2 * (n + lm)**2 * geg(ell, 1/2 * (-3 + d),  1 + (2 * ((-n + s2) * lm + s1 * (s2 + lm)))/(n * (n + lm)))))
    return result.squeeze(-1)

#First, lets calculate cc(ell,n) for open string

def AbsOpString(n,s2):
    return (-1)**(1 + n)*gamma(-s2)/(gamma(n+1)*gamma(1 - n - s2))



def cc(ell_n_tensor):
    z = torch.linspace(-1 + 10**(-6), 1 - 10**(-6), 10**6).view(1, -1)

    # Directly use ell and n from ell_n_tensor
    ell_values = ell_n_tensor[:, 0].view(-1, 1)  # Shape (k, 1)
    n_values = ell_n_tensor[:, 1].view(-1, 1)    # Shape (k, 1)

    # Compute normfac for all combinations of ell and n
    normfac = ((np.pi * 2**(1 - 2 * alpha) * gamma(2 * alpha + ell_values)) /
                (gamma(ell_values + 1) * (ell_values + alpha) * gamma(alpha)**2))**-1

    # Compute integrand
    integrand = normfac * (1 - z**2)**(alpha - 1/2) * \
                AbsOpString(n_values, n_values * (z - 1) / 2) * \
                geg(ell_values, alpha, z)

    # Integrate across z
    return torch.trapz(integrand, z, dim=1)

NullPoints

tensor([[-5.500000000000, -0.200000000000],
        [-5.500000000000,  0.163636363636],
        [-4.490000000000, -0.200000000000],
        [-4.490000000000,  0.163636363636],
        [-3.480000000000, -0.200000000000],
        [-3.480000000000,  0.163636363636],
        [-2.470000000000, -0.200000000000],
        [-2.470000000000,  0.163636363636],
        [-1.460000000000, -0.200000000000],
        [-1.460000000000,  0.163636363636],
        [-0.450000000000, -0.200000000000],
        [-0.450000000000,  0.163636363636],
        [ 0.560000000000, -0.200000000000],
        [ 0.560000000000,  0.163636363636],
        [ 1.570000000000, -0.200000000000],
        [ 1.570000000000,  0.163636363636],
        [ 2.580000000000, -0.200000000000],
        [ 2.580000000000,  0.163636363636],
        [ 3.590000000000, -0.200000000000],
        [ 3.590000000000,  0.163636363636],
        [ 4.600000000000, -0.200000000000],
        [ 4.600000000000,  0.163636363636]], dtype=torch.float64)

In [ ]:
#Store all the values
coeff_vals = cc(ell_n_tensor)
w00comp_vals = w00comp(ell_n_tensor)
w01comp_vals = w01comp(ell_n_tensor)
w10comp_vals = w10comp(ell_n_tensor)

ampcomp_vals = ampcomp(ell_n_tensor, NullPoints)
ampcompD1P_vals = ampcompD1P(ell_n_tensor, NullPoints)
ampcompD1M_vals = ampcompD1M(ell_n_tensor, NullPoints)
ampcompD1_vals = ampcompD1(ell_n_tensor, NullPoints)
ampcompD2_vals = ampcompD2(ell_n_tensor, NullPoints)
ampcompD3_vals = ampcompD3(ell_n_tensor, NullPoints)

<ipython-input-1-1ea80a7cb81f>:31: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return torch.tensor(result, dtype=z.dtype)
<ipython-input-1-1ea80a7cb81f>:33: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return torch.tensor(scipy_hyp2f1(a,b,c,z))


In [ ]:
#CHECKS FOR OPEN STRING
def w00fullX():
    return torch.dot(coeff_vals,w00comp_vals)

def w10fullX():
    return torch.dot(coeff_vals,w10comp_vals)

def w01fullX():
    return torch.dot(coeff_vals,w01comp_vals)

def ampfullX():
    return torch.sum(coeff_vals.view(-1, 1) * ampcomp_vals, dim = 0)

def ampD1PfullX():
    return torch.sum(coeff_vals.view(-1, 1) * ampcompD1P_vals, dim = 0)

def ampD1MfullX():
    return torch.sum(coeff_vals.view(-1, 1) * ampcompD1M_vals, dim = 0)

def ampD1fullX():
    return torch.sum(coeff_vals.view(-1, 1) * ampcompD1_vals, dim = 0)

def ampD2fullX():
    return torch.sum(coeff_vals.view(-1, 1) * ampcompD2_vals, dim = 0)

def ampD3fullX():
    return torch.sum(coeff_vals.view(-1, 1) * ampcompD3_vals, dim = 0)

(w00, w00fullX().item(),w10, w10fullX().item(),w01, w01fullX().item(),ampfullX()[1].item(),
 ampD1fullX()[1].item(),ampD2fullX()[1].item(),ampD3fullX()[1].item(),
 ampD1PfullX()[1].item(),ampD1MfullX()[1].item(),(ampD1PfullX()/ampD1MfullX())[1].item())

(-1.6449340668482264,
 -1.5708937983031839,
 -1.2020569031595942,
 -1.2020580619524968,
 1.8940656589944915,
 1.8940654448676988,
 -0.5624502228752933,
 3.839619282262798e-07,
 1.7739561382799387e-06,
 -3.90493232622108e-06,
 -0.5624498928853159,
 -0.5624501257869979,
 0.9999995859158504)

In [ ]:
# Same definitions for the PINN
def w00full():
    return torch.dot(ImPW(ell_n_tensor),w00comp_vals)

def w01full():
    return torch.dot(ImPW(ell_n_tensor),w01comp_vals)

def w10full():
    return torch.dot(ImPW(ell_n_tensor),w10comp_vals)

def ampfull():
    return torch.sum(ImPW(ell_n_tensor).view(-1, 1) * ampcomp_vals, dim = 0)

def ampD1Pfull():
    return torch.sum(ImPW(ell_n_tensor).view(-1, 1) * ampcompD1P_vals, dim = 0)

def ampD1Mfull():
    return torch.sum(ImPW(ell_n_tensor).view(-1, 1) * ampcompD1M_vals, dim = 0)

def ampD1full():
    return torch.sum(ImPW(ell_n_tensor).view(-1, 1) * ampcompD1_vals, dim = 0)

def ampD2full():
    return torch.sum(ImPW(ell_n_tensor).view(-1, 1) * ampcompD2_vals, dim = 0)

def ampD3full():
    return torch.sum(ImPW(ell_n_tensor).view(-1, 1) * ampcompD3_vals, dim = 0)

In [ ]:
# Define the neural network. We will impose multi-lambda equality. But lets first check how well
# this is satisfied with three derivatives imposed.
class SimpleNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(SimpleNN, self).__init__()
        self.input = nn.Linear(input_size, hidden_size, dtype=torch.float64)
        self.hidden1 = nn.Linear(hidden_size, hidden_size, dtype=torch.float64)
        self.hidden2 = nn.Linear(hidden_size, hidden_size, dtype=torch.float64)
        self.output = nn.Linear(hidden_size, output_size, dtype=torch.float64)

    def forward(self, x):
        x = torch.relu(self.input(x))
        x = torch.relu(self.hidden1(x))
        x = torch.relu(self.hidden2(x))
        x = F.softplus(self.output(x))
        return x.squeeze()  # Remove extra dimensions for output

# Create the neural network
input_size = 2  # Since ell_n_tensor has 2 columns
hidden_size = 64
output_size = 1  # Since coeff(ell,n) it a 1D tensor
ImPW = SimpleNN(input_size, hidden_size, output_size)
lr = 0.0001
optimizer = optim.Adam(ImPW.parameters(), lr=lr)  # Adam optimizer with learning rate of 0.0001

zero_vec = torch.zeros(len(NullPoints),dtype = torch.float64)
one_vec = torch.ones(len(NullPoints),dtype = torch.float64)

# Training loop
epoch_part = 20000
num_epochs = epoch_part
min_cons = torch.tensor([10**4], dtype = torch.float64)  # Initialize minimum cons to positive infinity
min_cons_epoch = 0
max_w00 = -torch.tensor([10**10], dtype = torch.float64)  # Track the max w00
min_cons_w00 = 0  # Save w00 for minimum loss epoch
min_cons_w10 = 0  # Save w10 difference for minimum loss epoch
min_cons_w01 = 0  # Save w01 difference for minimum loss epoch
min_cons_D1 = 0
min_cons_D2 = 0
min_cons_D3 = 0
min_cons_L6 = 0


zero_vec = torch.zeros(len(NullPoints),dtype = torch.float64)
one_vec = torch.ones(len(NullPoints),dtype = torch.float64)
for epoch in range(num_epochs+1):

    w00vals = w00full()
    w01vals = w01full()
    w10vals = w10full()


    losscomp1 = (w01- w01vals)
    losscomp2 = (w10- w10vals)
    losscomp3 =  F.mse_loss(ampD1full(),zero_vec)
    losscomp4 =  F.mse_loss(ampD2full(),zero_vec)
    losscomp5 =  F.mse_loss(ampD3full(),zero_vec)
    losscomp6 =  F.mse_loss(ampD1Pfull()/ampD1Mfull(),one_vec)
    beta1 = 10**2
    beta4 = 10**2


    cons1 = (losscomp1**2 + losscomp2**2)
    cons2 =  losscomp3
    cons3 =  (losscomp4 + losscomp5)
    cons4 = losscomp6
    cons = (cons1**(1/2))*(cons4**(1/2))

    loss = - w00vals + (beta1**2)*cons1 + (beta4**2)*cons4

    if cons < min_cons:
      if w00vals > max_w00:
        min_cons = cons.item()
        min_cons_epoch = epoch
        max_w00 = w00vals.item()
        min_cons_w10 = (losscomp1).item()
        min_cons_w01 = (losscomp2).item()
        min_cons_D1 = (losscomp3**(1/2)).item()
        min_cons_D2 = (losscomp4**(1/2)).item()
        min_cons_D3 = (losscomp5**(1/2)).item()
        min_cons_L6 = (losscomp6**(1/2)).item()
        min_cons_coeffs = ImPW(ell_n_tensor)

    # Print loss every 5000 epochs
    if epoch % 5000 == 0:
        print(f"Epoch {epoch}, Loss_Tot: {loss.item()}, Const: {(losscomp1).item(), (losscomp2).item(), (losscomp3**(1/2)).item(), (losscomp4**(1/2)).item(),(losscomp5**(1/2)).item(),(losscomp6**(1/2)).item()}, W00: {w00vals.item()}")
     # Print best solution every 100000 epochs
    if (epoch+1) % (epoch_part) == 0:
      print(f"\nMinimum Cons: {min_cons:.24f} at epoch {min_cons_epoch}")
      print("Values at minimum cons epoch:")
      print(f"W00: {max_w00:.10f}")
      print(f"W10 diff: {min_cons_w10:.10f}")
      print(f"W01 diff: {min_cons_w01:.10f}")
      print(f"Comp3: {min_cons_D1:.16f}")
      print(f"Comp4: {min_cons_D2:.16f}")
      print(f"Comp5: {min_cons_D3:.16f}")
      print(f"Comp6: {min_cons_L6:.16f}")
      print(f"Coeffs: {min_cons_coeffs}")

      min_cons = torch.tensor([10**4], dtype = torch.float64)  # Initialize minimum cons to positive infinity
      min_cons_epoch = 0
      max_w00 = -torch.tensor([10**10], dtype = torch.float64)  # Track the max w00
      min_cons_w00 = 0  # Save w00 for minimum loss epoch
      min_cons_w10 = 0  # Save w10 difference for minimum loss epoch
      min_cons_w01 = 0  # Save w01 difference for minimum loss epoch
      min_cons_D1 = 0
      min_cons_D2 = 0
      min_cons_D3 = 0
      min_cons_L6 = 0

    # Backward pass and optimization
    optimizer.zero_grad()  # Zero the gradients
    loss.backward()  # Backpropagation
    optimizer.step()  # Update weights

Epoch 0, Loss_Tot: 256051.46406119806, Const: (0.5501998547284257, -3.330277994707946, 11.531696177996201, 6.641633703692179, 6.846140015201417, 3.718215594078753), W00: -3865.47799890233
Epoch 5000, Loss_Tot: 115461.14346657263, Const: (1.1077462973768828, -2.1293593164188716, 4.9850537418645064, 3.1946200801028137, 3.1111240331200585, 2.377507358527765), W00: -1323.0014903155447
Epoch 10000, Loss_Tot: 79670.71874190436, Const: (1.169346255045514, -2.0491072583485224, 4.902542271434544, 2.661720646191805, 2.456857629849814, 1.509204586337057), W00: -1231.6217036406229
Epoch 15000, Loss_Tot: 78422.08982242925, Const: (1.1700610120608235, -2.034091710996785, 4.8728251845364605, 2.6360227504281473, 2.427846909229025, 1.4878560322292216), W00: -1219.215489114151

Minimum Cons: 4.393467870442067102487727 at epoch 5055
Values at minimum cons epoch:
W00: -1102.1174269535
W10 diff: 1.2026196524
W01 diff: -1.8668261360
Comp3: 4.5572307133983543
Comp4: 2.3865053290437213
Comp5: 2.16302880889150

In [ ]:
# Training loop
epoch_part = 20000
num_epochs = 4*epoch_part
min_cons = torch.tensor([10**4], dtype = torch.float64)  # Initialize minimum cons to positive infinity
min_cons_epoch = 0
max_w00 = -torch.tensor([10**10], dtype = torch.float64)  # Track the max w00
min_cons_w00 = 0  # Save w00 for minimum loss epoch
min_cons_w10 = 0  # Save w10 difference for minimum loss epoch
min_cons_w01 = 0  # Save w01 difference for minimum loss epoch
min_cons_D1 = 0
min_cons_D2 = 0
min_cons_D3 = 0
min_cons_L6 = 0


zero_vec = torch.zeros(len(NullPoints),dtype = torch.float64)
one_vec = torch.ones(len(NullPoints),dtype = torch.float64)
for epoch in range(num_epochs+1):

    w00vals = w00full()
    w01vals = w01full()
    w10vals = w10full()


    losscomp1 = (w01- w01vals)
    losscomp2 = (w10- w10vals)
    losscomp3 =  F.mse_loss(ampD1full(),zero_vec)
    losscomp4 =  F.mse_loss(ampD2full(),zero_vec)
    losscomp5 =  F.mse_loss(ampD3full(),zero_vec)
    losscomp6 =  F.mse_loss(ampD1Pfull()/ampD1Mfull(),one_vec)
    beta1 = 10**2
    beta4 = 10**2


    cons1 = (losscomp1**2 + losscomp2**2)
    cons2 =  losscomp3
    cons3 =  (losscomp4 + losscomp5)
    cons4 = losscomp6
    cons = (cons1**(1/2))*(cons4**(1/2))

    loss = - w00vals + (beta1**2)*cons1 + (beta4**2)*cons4

    if cons < min_cons:
      if w00vals > max_w00:
        min_cons = cons.item()
        min_cons_epoch = epoch
        max_w00 = w00vals.item()
        min_cons_w10 = (losscomp1).item()
        min_cons_w01 = (losscomp2).item()
        min_cons_D1 = (losscomp3**(1/2)).item()
        min_cons_D2 = (losscomp4**(1/2)).item()
        min_cons_D3 = (losscomp5**(1/2)).item()
        min_cons_L6 = (losscomp6**(1/2)).item()
        min_cons_coeffs = ImPW(ell_n_tensor)

    # Print loss every 5000 epochs
    if epoch % 5000 == 0:
        print(f"Epoch {epoch}, Loss_Tot: {loss.item()}, Const: {(losscomp1).item(), (losscomp2).item(), (losscomp3**(1/2)).item(), (losscomp4**(1/2)).item(),(losscomp5**(1/2)).item(),(losscomp6**(1/2)).item()}, W00: {w00vals.item()}")
     # Print best solution every 100000 epochs
    if (epoch) % (epoch_part) == 0:
      print(f"\nMinimum Cons: {min_cons:.24f} at epoch {min_cons_epoch}")
      print("Values at minimum cons epoch:")
      print(f"W00: {max_w00:.10f}")
      print(f"W10 diff: {min_cons_w10:.10f}")
      print(f"W01 diff: {min_cons_w01:.10f}")
      print(f"Comp3: {min_cons_D1:.16f}")
      print(f"Comp4: {min_cons_D2:.16f}")
      print(f"Comp5: {min_cons_D3:.16f}")
      print(f"Comp6: {min_cons_L6:.16f}")
      print(f"Coeffs: {min_cons_coeffs}")

      min_cons = torch.tensor([10**4], dtype = torch.float64)  # Initialize minimum cons to positive infinity
      min_cons_epoch = 0
      max_w00 = -torch.tensor([10**10], dtype = torch.float64)  # Track the max w00
      min_cons_w00 = 0  # Save w00 for minimum loss epoch
      min_cons_w10 = 0  # Save w10 difference for minimum loss epoch
      min_cons_w01 = 0  # Save w01 difference for minimum loss epoch
      min_cons_D1 = 0
      min_cons_D2 = 0
      min_cons_D3 = 0
      min_cons_L6 = 0

    # Backward pass and optimization
    optimizer.zero_grad()  # Zero the gradients
    loss.backward()  # Backpropagation
    optimizer.step()  # Update weights

Epoch 0, Loss_Tot: 73325.46503601906, Const: (1.1819139373720078, -1.9544728691559712, 4.706734509363361, 2.4887885245929917, 2.262873072276772, 1.4144471063681452), W00: -1150.0113526770997

Minimum Cons: 3.230668021400498357564857 at epoch 0
Values at minimum cons epoch:
W00: -1150.0113526771
W10 diff: 1.1819139374
W01 diff: -1.9544728692
Comp3: 4.7067345093633612
Comp4: 2.4887885245929917
Comp5: 2.2628730722767720
Comp6: 1.4144471063681452
Coeffs: tensor([0.396068259120, 0.265860195554, 0.184093066041, 0.117528285561,
        0.074655085764, 0.046877151903, 0.029004053770, 0.331534267659,
        0.319293587394, 0.266127098330, 0.186128371730, 0.119711479237,
        0.076508776901, 0.315687462502, 0.314591522011, 0.350208638978,
        0.299874968711, 0.202366433586, 0.130890745405, 0.284789056989,
        0.259426583112, 0.301286930510, 0.345638535799, 0.314886516900,
        0.262210319437, 0.224095406893, 0.245058500373, 0.287403074999,
        0.332508774739, 0.242758042277, 0

In [ ]:
# Training loop
epoch_part = 20000
num_epochs = 2*epoch_part
min_cons = torch.tensor([10**4], dtype = torch.float64)  # Initialize minimum cons to positive infinity
min_cons_epoch = 0
max_w00 = -torch.tensor([10**10], dtype = torch.float64)  # Track the max w00
min_cons_w00 = 0  # Save w00 for minimum loss epoch
min_cons_w10 = 0  # Save w10 difference for minimum loss epoch
min_cons_w01 = 0  # Save w01 difference for minimum loss epoch
min_cons_D1 = 0
min_cons_D2 = 0
min_cons_D3 = 0
min_cons_L6 = 0


zero_vec = torch.zeros(len(NullPoints),dtype = torch.float64)
one_vec = torch.ones(len(NullPoints),dtype = torch.float64)
for epoch in range(num_epochs+1):

    w00vals = w00full()
    w01vals = w01full()
    w10vals = w10full()


    losscomp1 = (w01- w01vals)
    losscomp2 = (w10- w10vals)
    losscomp3 =  F.mse_loss(ampD1full(),zero_vec)
    losscomp4 =  F.mse_loss(ampD2full(),zero_vec)
    losscomp5 =  F.mse_loss(ampD3full(),zero_vec)
    losscomp6 =  F.mse_loss(ampD1Pfull()/ampD1Mfull(),one_vec)
    beta1 = 10**2
    beta4 = 10**2


    cons1 = (losscomp1**2 + losscomp2**2)
    cons2 =  losscomp3
    cons3 =  (losscomp4 + losscomp5)
    cons4 = losscomp6
    cons = (cons1**(1/2))*(cons4**(1/2))

    loss = - w00vals + (beta1**2)*cons1 + (beta4**2)*cons4

    if cons < min_cons:
      if w00vals > max_w00:
        min_cons = cons.item()
        min_cons_epoch = epoch
        max_w00 = w00vals.item()
        min_cons_w10 = (losscomp1).item()
        min_cons_w01 = (losscomp2).item()
        min_cons_D1 = (losscomp3**(1/2)).item()
        min_cons_D2 = (losscomp4**(1/2)).item()
        min_cons_D3 = (losscomp5**(1/2)).item()
        min_cons_L6 = (losscomp6**(1/2)).item()
        min_cons_coeffs = ImPW(ell_n_tensor)

    # Print loss every 5000 epochs
    if epoch % 5000 == 0:
        print(f"Epoch {epoch}, Loss_Tot: {loss.item()}, Const: {(losscomp1).item(), (losscomp2).item(), (losscomp3**(1/2)).item(), (losscomp4**(1/2)).item(),(losscomp5**(1/2)).item(),(losscomp6**(1/2)).item()}, W00: {w00vals.item()}")
     # Print best solution every 100000 epochs
    if (epoch) % (epoch_part) == 0:
      print(f"\nMinimum Cons: {min_cons:.24f} at epoch {min_cons_epoch}")
      print("Values at minimum cons epoch:")
      print(f"W00: {max_w00:.10f}")
      print(f"W10 diff: {min_cons_w10:.10f}")
      print(f"W01 diff: {min_cons_w01:.10f}")
      print(f"Comp3: {min_cons_D1:.16f}")
      print(f"Comp4: {min_cons_D2:.16f}")
      print(f"Comp5: {min_cons_D3:.16f}")
      print(f"Comp6: {min_cons_L6:.16f}")
      print(f"Coeffs: {min_cons_coeffs}")

      min_cons = torch.tensor([10**4], dtype = torch.float64)  # Initialize minimum cons to positive infinity
      min_cons_epoch = 0
      max_w00 = -torch.tensor([10**10], dtype = torch.float64)  # Track the max w00
      min_cons_w00 = 0  # Save w00 for minimum loss epoch
      min_cons_w10 = 0  # Save w10 difference for minimum loss epoch
      min_cons_w01 = 0  # Save w01 difference for minimum loss epoch
      min_cons_D1 = 0
      min_cons_D2 = 0
      min_cons_D3 = 0
      min_cons_L6 = 0

    # Backward pass and optimization
    optimizer.zero_grad()  # Zero the gradients
    loss.backward()  # Backpropagation
    optimizer.step()  # Update weights

Epoch 0, Loss_Tot: 1.536410400506526, Const: (0.0007513443333646563, -0.000839175472490794, 0.0004913423844400277, 0.004124272679380109, 0.0018352223152830872, 0.0005753642848931888), W00: -1.5204126220941263

Minimum Cons: 0.000000648079302486109005 at epoch 0
Values at minimum cons epoch:
W00: -1.5204126221
W10 diff: 0.0007513443
W01 diff: -0.0008391755
Comp3: 0.0004913423844400
Comp4: 0.0041242726793801
Comp5: 0.0018352223152831
Comp6: 0.0005753642848932
Coeffs: tensor([9.854712507732e-01, 2.008082825890e-01, 5.931666332485e-03,
        1.336595274161e-04, 4.847336084723e-06, 1.835057158533e-07,
        6.837828880809e-09, 5.707823014287e-02, 1.342639951155e-02,
        7.193554198774e-04, 3.613680857593e-05, 1.508621868305e-06,
        6.020545218254e-08, 2.776132615220e-03, 2.226944718195e-02,
        8.401590083742e-04, 3.191715194418e-05, 1.324124365714e-06,
        9.492252735702e-08, 1.165407766779e-03, 1.973561369897e-03,
        3.035986100936e-04, 2.086331502226e-05, 1.1599

# New section With some null constraints, and multi-lambda equality constraints (Using product of cons as the strategy), Diff Null grid

In [ ]:
#THIS SECTION IS JUST TO TEST THE EXPRESSIONS FOR THE OPEN STRING AMPLITUDE
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import numpy as np
from scipy.special import legendre
from scipy import integrate
from scipy.special import hyp2f1 as scipy_hyp2f1
from scipy.special import gamma as scipy_gamma
from scipy.special import zeta
#This is all for to check formulaes using the Open String Amplitude
#Some definitions
d = 10;
alpha = (d-3)/2;
lm = 56/10;
dlm = 1/2
lmp = lm + dlm
lmm = lm - dlm
ellmax = 12;
w00 = -(np.pi)**2/6
w10 = -zeta(3)
w01 = 7*np.pi**4/360

torch.set_printoptions(precision=12)

def hyp2f1(a, b, c, z):
    if isinstance(z, torch.Tensor):
        z_np = z.detach().numpy()
        result = scipy_hyp2f1(a,b,c,z_np)
        return torch.tensor(result, dtype=z.dtype)
    else:
        return torch.tensor(scipy_hyp2f1(a,b,c,z))

def gamma(x):
    if isinstance(x, torch.Tensor):
        x_np = x.detach().numpy()
        result = scipy_gamma(x_np)
        return torch.tensor(result, dtype=x.dtype)
    else:
        return torch.tensor(scipy_gamma(x))

def ker(lm,n,s1,s2):
    return (1/(s1 - n) + 1/(s2 - n) + 1/(n + lm))

def geg(ell,beta, z):
    return gamma(ell + 2*beta) / (gamma(2*beta) * gamma(ell + 1)) * \
           hyp2f1(-ell, ell + 2*beta, beta + 1/2, (1-z)/2)

#ell,n tensor and Null grid tensor
ell = torch.arange(ellmax + 1, dtype = torch.float64).unsqueeze(1).repeat(1, ellmax + 1)
n = torch.arange(1, ellmax + 2, dtype = torch.float64).unsqueeze(0).repeat(ellmax + 1, 1)
mask = (n > ell) & (n <= ellmax + 1) & ((n - ell - 1) % 2 == 0)
ell_n_tensor = torch.stack([ell[mask], n[mask]], dim=1)

NullPoints = torch.stack(torch.meshgrid(
    torch.arange(-6 +1/2, 5+1/2 + 1e-6, 1, dtype=torch.float64),
    torch.arange(-0.2, 0.2 + 1e-6, 0.4, dtype=torch.float64),
    indexing='ij'
)).permute(1, 2, 0).reshape(-1, 2)


def w00comp(ell_n_tensor):
    ell = ell_n_tensor[:, 0].view(-1, 1)
    n = ell_n_tensor[:, 1].view(-1, 1)
    result = (-1/n)*geg(ell,alpha,1)
    return result.squeeze(-1)

def w10comp(ell_n_tensor):
    ell = ell_n_tensor[:, 0].view(-1, 1)  # Shape (k, 1)
    n = ell_n_tensor[:, 1].view(-1, 1)    # Shape (k, 1)
    result = (gamma(-3 + d + ell) * (-((2 * ell * (-3 + d + ell) * lm * (n + 2 * lm) * \
            hyp2f1(1 - ell, -2 + d + ell, d / 2, lm / (n + lm))) / ((-2 + d) * (n + lm)**2)) - \
            hyp2f1(-ell, -3 + d + ell, 1/2 * (-2 + d), lm / (n + lm)))) / (n**2 * gamma(-3 + d) * gamma(1 + ell))
    return result.squeeze(-1)


def w01comp(ell_n_tensor):
    ell = ell_n_tensor[:, 0].view(-1, 1)  # Shape (k, 1)
    n = ell_n_tensor[:, 1].view(-1, 1)    # Shape (k, 1)
    result = (gamma(-3 + d + ell) * (-(1/((d - 2) * (n + lm)**2)) *\
             2 * ell * (-3 + d + ell) * n * (n + 2*lm) *  hyp2f1(1 - ell, -2 + d + ell, d/2, lm/(n + lm)) + \
             2 * hyp2f1(-ell, -3 + d + ell, (d - 2)/2, lm/(n + lm)))) / (n**3 * gamma(-3 + d) * gamma(1 + ell))
    return result.squeeze(-1)

def ampcomp(ell_n_tensor, NullPoints):
    ell = ell_n_tensor[:, 0].view(-1, 1)
    n = ell_n_tensor[:, 1].view(-1, 1)
    s1 = NullPoints[:, 0].view(1, -1)
    s2 = NullPoints[:, 1].view(1, -1)
    result = ker(lm,n, s1, s2) * geg(ell, alpha, 1 + 2 * (((s1 + lm) * (s2 + lm)) / (n + lm) - lm) / n)
    return result.squeeze(-1)

def ampcompD1P(ell_n_tensor, NullPoints):
    ell = ell_n_tensor[:, 0].view(-1, 1)
    n = ell_n_tensor[:, 1].view(-1, 1)
    s1 = NullPoints[:, 0].view(1, -1)
    s2 = NullPoints[:, 1].view(1, -1)
    result = ker(lmp,n,s1,s2)*geg(ell,alpha,1 + 2*(((s1 + lmp)*(s2 + lmp))/(n + lmp)-lmp)/n)
    return result.squeeze(-1)

def ampcompD1M(ell_n_tensor, NullPoints):
    ell = ell_n_tensor[:, 0].view(-1, 1)
    n = ell_n_tensor[:, 1].view(-1, 1)
    s1 = NullPoints[:, 0].view(1, -1)
    s2 = NullPoints[:, 1].view(1, -1)
    result = ker(lmm,n,s1,s2)*geg(ell,alpha,1 + 2*(((s1 + lmm)*(s2 + lmm))/(n + lmm)-lmm)/n)
    return result.squeeze(-1)


def ampcompD1(ell_n_tensor, NullPoints):
    ell = ell_n_tensor[:, 0].view(-1, 1)
    n = ell_n_tensor[:, 1].view(-1, 1)
    s1 = NullPoints[:, 0].view(1, -1)
    s2 = NullPoints[:, 1].view(1, -1)
    result = ((2 * (-3 + d) * (n**2 + 2 * n * lm - s2 * lm - s1 * (s2 + lm)) * geg(-1 + ell, 1/2 * (-1 + d),
        1 + (2 * ((-n + s2) * lm + s1 * (s2 + lm))) /  (n * (n + lm))) / n - (n + lm) * geg(ell, 1/2 * (-3 + d),
        1 + (2 * ((-n + s2) * lm + s1 * (s2 + lm))) / (n * (n + lm)))) / (n + lm)**3)
    return result.squeeze(-1)

def ampcompD2(ell_n_tensor, NullPoints):
    ell = ell_n_tensor[:, 0].view(-1, 1)
    n = ell_n_tensor[:, 1].view(-1, 1)
    s1 = NullPoints[:, 0].view(1, -1)
    s2 = NullPoints[:, 1].view(1, -1)
    result = (1 / (n + lm)**4 * 2 * (-((2 * (-3 + d) * (-1 + d) * (n - s1) * (n - s2) *
               (n**2 + 2*n*lm - s2*lm - s1*(s2 + lm)) *
               geg(-2 + ell,(d+1)/2, 1 + (2 * ((-n + s2)*lm + s1*(s2 + lm)))/(n*(n + lm)))) /
              (n**2 * (n + lm))) + ((-3 + d) * (n - s1) * (n - s2) *
             geg(-1 + ell,(d-1)/2, 1 + (2 * ((-n + s2)*lm + s1*(s2 + lm)))/(n*(n + lm)))) / n +
            ((-3 + d) * (2*n - s1 - s2) * (n + lm) *
             geg(-1 + ell, (d-1)/2, 1 + (2 * ((-n + s2)*lm + s1*(s2 + lm)))/(n*(n + lm)))) / n -
            (3 * (-3 + d) * (n**2 + 2*n*lm - s2*lm - s1*(s2 + lm)) *
             geg(-1 + ell,(d-1)/2, 1 + (2 * ((-n + s2)*lm + s1*(s2 + lm)))/(n*(n + lm)))) / n +
            (n + lm) * geg(ell,(d-3)/2, 1 + (2 * ((-n + s2)*lm + s1*(s2 + lm)))/(n*(n + lm)))))

    return result.squeeze(-1)


def ampcompD3(ell_n_tensor, NullPoints):
    ell = ell_n_tensor[:, 0].view(-1, 1)
    n = ell_n_tensor[:, 1].view(-1, 1)
    s1 = NullPoints[:, 0].view(1, -1)
    s2 = NullPoints[:, 1].view(1, -1)
    result =  (2 / (n**3 * (n + lm)**7)) * (4 * (-3 + d) * (-1 + d) * (1 + d) * (n - s1)**2 * (n - s2)**2 * (n**2 + 2 * n * lm - s2 * lm -
        s1 * (s2 + lm)) * geg(-3 + ell, (3 + d)/2, 1 + (2 * ((-n + s2) * lm + s1 * (s2 + lm)))/(n * (n + lm))) -
     2 * (-3 + d) * (-1 + d) * n * (n - s1)**2 * (n -s2)**2 * (n + lm) * geg(-2 + ell, (1 + d)/2,
       1 + (2 * ((-n + s2) * lm + s1 * (s2 + lm)))/(n * (n + lm))) -
     4 * (-3 + d) * (-1 + d) * n * (n - s1) * (n - s2) * (2 * n - s1 -s2) * (n + lm)**2 * geg(-2 + ell, (1 + d)/2,
       1 + (2 * ((-n + s2) * lm + s1 * (s2 + lm)))/(n * (n + lm))) +
     8 * (-3 + d) * (-1 + d) * n * (n - s1) * (n - s2) * (n + lm) * (n**2 +2 * n * lm - s2 * lm -
        s1 * (s2 + lm)) * geg(-2 + ell, (1 + d)/2,1 + (2 * ((-n + s2) * lm + s1 * (s2 + lm)))/(n * (n + lm))) -
     2 * (-3 + d) * n**2 * (n - s1) * (n -s2) * (n + lm)**2 * geg(-1 + ell, 1/2 * (-1 + d),
       1 + (2 * ((-n + s2) * lm + s1 * (s2 + lm)))/( n * (n + lm))) -
     2 * (-3 + d) * n**2 * (2 * n - s1 - s2) * (n + lm)**3 * geg(-1 + ell, 1/2 * (-1 + d),1 + (2 * ((-n + s2) * lm + s1 * (s2 + lm)))/(n * (n + lm))) +
     n**3 * (n + lm)**3 * geg(ell, 1/2 * (-3 + d),1 + (2 * ((-n + s2) * lm + s1 * (s2 + lm)))/( n * (n + lm))) +
     4 * n * (n + lm) * (2 * (-3 + d) * (-1 + d) * (n - s1) * (n -s2) * (n**2 + 2 * n * lm - s2 * lm -
     s1 * (s2 + lm)) * geg(-2 + ell, (1 + d)/2,1 + (2 * ((-n + s2) * lm + s1 * (s2 + lm)))/(n * (n + lm)))
     - (-3 + d) * n * (n - s1) * (n -s2) * (n + lm) * geg(-1 + ell, 1/2 * (-1 + d),
      1 + (2 * ((-n + s2) * lm + s1 * (s2 + lm)))/(n * (n + lm))) - (-3 + d) * n * (2 * n - s1 -s2) * (n + lm)**2 * geg(-1 + ell, 1/2 * (-1 + d),
      1 + (2 * ((-n + s2) * lm + s1 * (s2 + lm)))/( n * (n + lm))) +3 * (-3 + d) * n * (n + lm) * (n**2 + 2 * n * lm -
      s2 * lm - s1 * (s2 + lm)) * geg(-1 + ell,1/2 * (-1 + d),1 + (2 * ((-n + s2) * lm + s1 * (s2 + lm)))/( n * (n + lm))) -
        n**2 * (n + lm)**2 * geg(ell, 1/2 * (-3 + d),  1 + (2 * ((-n + s2) * lm + s1 * (s2 + lm)))/(n * (n + lm)))))
    return result.squeeze(-1)

#First, lets calculate cc(ell,n) for open string

def AbsOpString(n,s2):
    return (-1)**(1 + n)*gamma(-s2)/(gamma(n+1)*gamma(1 - n - s2))



def cc(ell_n_tensor):
    z = torch.linspace(-1 + 10**(-6), 1 - 10**(-6), 10**6).view(1, -1)

    # Directly use ell and n from ell_n_tensor
    ell_values = ell_n_tensor[:, 0].view(-1, 1)  # Shape (k, 1)
    n_values = ell_n_tensor[:, 1].view(-1, 1)    # Shape (k, 1)

    # Compute normfac for all combinations of ell and n
    normfac = ((np.pi * 2**(1 - 2 * alpha) * gamma(2 * alpha + ell_values)) /
                (gamma(ell_values + 1) * (ell_values + alpha) * gamma(alpha)**2))**-1

    # Compute integrand
    integrand = normfac * (1 - z**2)**(alpha - 1/2) * \
                AbsOpString(n_values, n_values * (z - 1) / 2) * \
                geg(ell_values, alpha, z)

    # Integrate across z
    return torch.trapz(integrand, z, dim=1)

NullPoints

tensor([[-5.500000000000, -0.200000000000],
        [-5.500000000000,  0.200000000000],
        [-4.500000000000, -0.200000000000],
        [-4.500000000000,  0.200000000000],
        [-3.500000000000, -0.200000000000],
        [-3.500000000000,  0.200000000000],
        [-2.500000000000, -0.200000000000],
        [-2.500000000000,  0.200000000000],
        [-1.500000000000, -0.200000000000],
        [-1.500000000000,  0.200000000000],
        [-0.500000000000, -0.200000000000],
        [-0.500000000000,  0.200000000000],
        [ 0.500000000000, -0.200000000000],
        [ 0.500000000000,  0.200000000000],
        [ 1.500000000000, -0.200000000000],
        [ 1.500000000000,  0.200000000000],
        [ 2.500000000000, -0.200000000000],
        [ 2.500000000000,  0.200000000000],
        [ 3.500000000000, -0.200000000000],
        [ 3.500000000000,  0.200000000000],
        [ 4.500000000000, -0.200000000000],
        [ 4.500000000000,  0.200000000000],
        [ 5.500000000000, -0.200

In [ ]:
#Store all the values
coeff_vals = cc(ell_n_tensor)
w00comp_vals = w00comp(ell_n_tensor)
w01comp_vals = w01comp(ell_n_tensor)
w10comp_vals = w10comp(ell_n_tensor)

ampcomp_vals = ampcomp(ell_n_tensor, NullPoints)
ampcompD1P_vals = ampcompD1P(ell_n_tensor, NullPoints)
ampcompD1M_vals = ampcompD1M(ell_n_tensor, NullPoints)
ampcompD1_vals = ampcompD1(ell_n_tensor, NullPoints)
ampcompD2_vals = ampcompD2(ell_n_tensor, NullPoints)
ampcompD3_vals = ampcompD3(ell_n_tensor, NullPoints)

<ipython-input-24-33f1b916b2d8>:31: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return torch.tensor(result, dtype=z.dtype)
<ipython-input-24-33f1b916b2d8>:33: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return torch.tensor(scipy_hyp2f1(a,b,c,z))


In [ ]:
#CHECKS FOR OPEN STRING
def w00fullX():
    return torch.dot(coeff_vals,w00comp_vals)

def w10fullX():
    return torch.dot(coeff_vals,w10comp_vals)

def w01fullX():
    return torch.dot(coeff_vals,w01comp_vals)

def ampfullX():
    return torch.sum(coeff_vals.view(-1, 1) * ampcomp_vals, dim = 0)

def ampD1PfullX():
    return torch.sum(coeff_vals.view(-1, 1) * ampcompD1P_vals, dim = 0)

def ampD1MfullX():
    return torch.sum(coeff_vals.view(-1, 1) * ampcompD1M_vals, dim = 0)

def ampD1fullX():
    return torch.sum(coeff_vals.view(-1, 1) * ampcompD1_vals, dim = 0)

def ampD2fullX():
    return torch.sum(coeff_vals.view(-1, 1) * ampcompD2_vals, dim = 0)

def ampD3fullX():
    return torch.sum(coeff_vals.view(-1, 1) * ampcompD3_vals, dim = 0)

(w00, w00fullX().item(),w10, w10fullX().item(),w01, w01fullX().item(),ampfullX()[1].item(),
 ampD1fullX()[1].item(),ampD2fullX()[1].item(),ampD3fullX()[1].item(),
 ampD1PfullX()[1].item(),ampD1MfullX()[1].item(),(ampD1PfullX()/ampD1MfullX())[1].item())

(-1.6449340668482264,
 -1.5708937983031839,
 -1.2020569031595942,
 -1.2020580619524968,
 1.8940656589944915,
 1.8940654448676988,
 -0.6006966395237955,
 3.8352236174573874e-07,
 1.770410565962509e-06,
 -3.886982374550674e-06,
 -0.6006963098081171,
 -0.6006965429156984,
 0.9999996119378677)

In [ ]:
# Same definitions for the PINN
def w00full():
    return torch.dot(ImPW(ell_n_tensor),w00comp_vals)

def w01full():
    return torch.dot(ImPW(ell_n_tensor),w01comp_vals)

def w10full():
    return torch.dot(ImPW(ell_n_tensor),w10comp_vals)

def ampfull():
    return torch.sum(ImPW(ell_n_tensor).view(-1, 1) * ampcomp_vals, dim = 0)

def ampD1Pfull():
    return torch.sum(ImPW(ell_n_tensor).view(-1, 1) * ampcompD1P_vals, dim = 0)

def ampD1Mfull():
    return torch.sum(ImPW(ell_n_tensor).view(-1, 1) * ampcompD1M_vals, dim = 0)

def ampD1full():
    return torch.sum(ImPW(ell_n_tensor).view(-1, 1) * ampcompD1_vals, dim = 0)

def ampD2full():
    return torch.sum(ImPW(ell_n_tensor).view(-1, 1) * ampcompD2_vals, dim = 0)

def ampD3full():
    return torch.sum(ImPW(ell_n_tensor).view(-1, 1) * ampcompD3_vals, dim = 0)

In [ ]:
# Define the neural network. We will impose multi-lambda equality. But lets first check how well
# this is satisfied with three derivatives imposed.
class SimpleNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(SimpleNN, self).__init__()
        self.input = nn.Linear(input_size, hidden_size, dtype=torch.float64)
        self.hidden1 = nn.Linear(hidden_size, hidden_size, dtype=torch.float64)
        self.hidden2 = nn.Linear(hidden_size, hidden_size, dtype=torch.float64)
        self.output = nn.Linear(hidden_size, output_size, dtype=torch.float64)

    def forward(self, x):
        x = torch.relu(self.input(x))
        x = torch.relu(self.hidden1(x))
        x = torch.relu(self.hidden2(x))
        x = F.softplus(self.output(x))
        return x.squeeze()  # Remove extra dimensions for output

# Create the neural network
input_size = 2  # Since ell_n_tensor has 2 columns
hidden_size = 64
output_size = 1  # Since coeff(ell,n) it a 1D tensor
ImPW = SimpleNN(input_size, hidden_size, output_size)
lr = 0.0001
optimizer = optim.Adam(ImPW.parameters(), lr=lr)  # Adam optimizer with learning rate of 0.0001

zero_vec = torch.zeros(len(NullPoints),dtype = torch.float64)
one_vec = torch.ones(len(NullPoints),dtype = torch.float64)

# Training loop
epoch_part = 20000
num_epochs = epoch_part
min_cons = torch.tensor([10**4], dtype = torch.float64)  # Initialize minimum cons to positive infinity
min_cons_epoch = 0
max_w00 = -torch.tensor([10**10], dtype = torch.float64)  # Track the max w00
min_cons_w00 = 0  # Save w00 for minimum loss epoch
min_cons_w10 = 0  # Save w10 difference for minimum loss epoch
min_cons_w01 = 0  # Save w01 difference for minimum loss epoch
min_cons_D1 = 0
min_cons_D2 = 0
min_cons_D3 = 0
min_cons_L6 = 0


zero_vec = torch.zeros(len(NullPoints),dtype = torch.float64)
one_vec = torch.ones(len(NullPoints),dtype = torch.float64)
for epoch in range(num_epochs+1):

    w00vals = w00full()
    w01vals = w01full()
    w10vals = w10full()


    losscomp1 = (w01- w01vals)
    losscomp2 = (w10- w10vals)
    losscomp3 =  F.mse_loss(ampD1full(),zero_vec)
    losscomp4 =  F.mse_loss(ampD2full(),zero_vec)
    losscomp5 =  F.mse_loss(ampD3full(),zero_vec)
    losscomp6 =  F.mse_loss(ampD1Pfull()/ampD1Mfull(),one_vec)
    beta1 = 10**2
    beta4 = 10**2


    cons1 = (losscomp1**2 + losscomp2**2)
    cons2 =  losscomp3
    cons3 =  (losscomp4 + losscomp5)
    cons4 = losscomp6
    cons = (cons1**(1/2))*(cons4**(1/2))

    loss = - w00vals + (beta1**2)*cons1 + (beta4**2)*cons4

    if cons < min_cons:
      if w00vals > max_w00:
        min_cons = cons.item()
        min_cons_epoch = epoch
        max_w00 = w00vals.item()
        min_cons_w10 = (losscomp1).item()
        min_cons_w01 = (losscomp2).item()
        min_cons_D1 = (losscomp3**(1/2)).item()
        min_cons_D2 = (losscomp4**(1/2)).item()
        min_cons_D3 = (losscomp5**(1/2)).item()
        min_cons_L6 = (losscomp6**(1/2)).item()
        min_cons_coeffs = ImPW(ell_n_tensor)

    # Print loss every 5000 epochs
    if epoch % 5000 == 0:
        print(f"Epoch {epoch}, Loss_Tot: {loss.item()}, Const: {(losscomp1).item(), (losscomp2).item(), (losscomp3**(1/2)).item(), (losscomp4**(1/2)).item(),(losscomp5**(1/2)).item(),(losscomp6**(1/2)).item()}, W00: {w00vals.item()}")
     # Print best solution every 100000 epochs
    if (epoch+1) % (epoch_part) == 0:
      print(f"\nMinimum Cons: {min_cons:.24f} at epoch {min_cons_epoch}")
      print("Values at minimum cons epoch:")
      print(f"W00: {max_w00:.10f}")
      print(f"W10 diff: {min_cons_w10:.10f}")
      print(f"W01 diff: {min_cons_w01:.10f}")
      print(f"Comp3: {min_cons_D1:.16f}")
      print(f"Comp4: {min_cons_D2:.16f}")
      print(f"Comp5: {min_cons_D3:.16f}")
      print(f"Comp6: {min_cons_L6:.16f}")
      print(f"Coeffs: {min_cons_coeffs}")

      min_cons = torch.tensor([10**4], dtype = torch.float64)  # Initialize minimum cons to positive infinity
      min_cons_epoch = 0
      max_w00 = -torch.tensor([10**10], dtype = torch.float64)  # Track the max w00
      min_cons_w00 = 0  # Save w00 for minimum loss epoch
      min_cons_w10 = 0  # Save w10 difference for minimum loss epoch
      min_cons_w01 = 0  # Save w01 difference for minimum loss epoch
      min_cons_D1 = 0
      min_cons_D2 = 0
      min_cons_D3 = 0
      min_cons_L6 = 0

    # Backward pass and optimization
    optimizer.zero_grad()  # Zero the gradients
    loss.backward()  # Backpropagation
    optimizer.step()  # Update weights

Epoch 0, Loss_Tot: 189980.284108349, Const: (0.6191000483218319, -2.877804170995457, 21.948242580185813, 6.779894033145742, 5.717897197004856, 3.1616399141430804), W00: -3370.197477012916
Epoch 5000, Loss_Tot: 32572.106268427233, Const: (1.395905127589047, -1.0285955033970475, 5.279119027821399, 1.5849906577282786, 0.7678880740883774, 0.4749649951609432), W00: -250.59045376469362
Epoch 10000, Loss_Tot: 27485.205933561127, Const: (1.3869602401243903, -0.8085609613877025, 5.519884395062325, 1.6547477011638747, 0.7942621001224516, 0.3810039304134601), W00: -259.27062399496134
Epoch 15000, Loss_Tot: 5951.751090871003, Const: (0.6473528582409411, -0.11399074069011861, 8.640242874567436, 2.647858757290833, 1.3033898698483461, 0.34460570603676616), W00: -443.6240441820438

Minimum Cons: 0.739904374782945795097078 at epoch 6361
Values at minimum cons epoch:
W00: -253.8943417368
W10 diff: 1.3930827722
W01 diff: -0.9844350696
Comp3: 5.3210734990924813
Comp4: 1.5972162787765296
Comp5: 0.773190049

In [ ]:
# Training loop
epoch_part = 20000
num_epochs = 4*epoch_part
min_cons = torch.tensor([10**4], dtype = torch.float64)  # Initialize minimum cons to positive infinity
min_cons_epoch = 0
max_w00 = -torch.tensor([10**10], dtype = torch.float64)  # Track the max w00
min_cons_w00 = 0  # Save w00 for minimum loss epoch
min_cons_w10 = 0  # Save w10 difference for minimum loss epoch
min_cons_w01 = 0  # Save w01 difference for minimum loss epoch
min_cons_D1 = 0
min_cons_D2 = 0
min_cons_D3 = 0
min_cons_L6 = 0


zero_vec = torch.zeros(len(NullPoints),dtype = torch.float64)
one_vec = torch.ones(len(NullPoints),dtype = torch.float64)
for epoch in range(num_epochs+1):

    w00vals = w00full()
    w01vals = w01full()
    w10vals = w10full()


    losscomp1 = (w01- w01vals)
    losscomp2 = (w10- w10vals)
    losscomp3 =  F.mse_loss(ampD1full(),zero_vec)
    losscomp4 =  F.mse_loss(ampD2full(),zero_vec)
    losscomp5 =  F.mse_loss(ampD3full(),zero_vec)
    losscomp6 =  F.mse_loss(ampD1Pfull()/ampD1Mfull(),one_vec)
    beta1 = 10**2
    beta4 = 10**2


    cons1 = (losscomp1**2 + losscomp2**2)
    cons2 =  losscomp3
    cons3 =  (losscomp4 + losscomp5)
    cons4 = losscomp6
    cons = (cons1**(1/2))*(cons4**(1/2))

    loss = - w00vals + (beta1**2)*cons1 + (beta4**2)*cons4

    if cons < min_cons:
      if w00vals > max_w00:
        min_cons = cons.item()
        min_cons_epoch = epoch
        max_w00 = w00vals.item()
        min_cons_w10 = (losscomp1).item()
        min_cons_w01 = (losscomp2).item()
        min_cons_D1 = (losscomp3**(1/2)).item()
        min_cons_D2 = (losscomp4**(1/2)).item()
        min_cons_D3 = (losscomp5**(1/2)).item()
        min_cons_L6 = (losscomp6**(1/2)).item()
        min_cons_coeffs = ImPW(ell_n_tensor)

    # Print loss every 5000 epochs
    if epoch % 5000 == 0:
        print(f"Epoch {epoch}, Loss_Tot: {loss.item()}, Const: {(losscomp1).item(), (losscomp2).item(), (losscomp3**(1/2)).item(), (losscomp4**(1/2)).item(),(losscomp5**(1/2)).item(),(losscomp6**(1/2)).item()}, W00: {w00vals.item()}")
     # Print best solution every 100000 epochs
    if (epoch) % (epoch_part) == 0:
      print(f"\nMinimum Cons: {min_cons:.24f} at epoch {min_cons_epoch}")
      print("Values at minimum cons epoch:")
      print(f"W00: {max_w00:.10f}")
      print(f"W10 diff: {min_cons_w10:.10f}")
      print(f"W01 diff: {min_cons_w01:.10f}")
      print(f"Comp3: {min_cons_D1:.16f}")
      print(f"Comp4: {min_cons_D2:.16f}")
      print(f"Comp5: {min_cons_D3:.16f}")
      print(f"Comp6: {min_cons_L6:.16f}")
      print(f"Coeffs: {min_cons_coeffs}")

      min_cons = torch.tensor([10**4], dtype = torch.float64)  # Initialize minimum cons to positive infinity
      min_cons_epoch = 0
      max_w00 = -torch.tensor([10**10], dtype = torch.float64)  # Track the max w00
      min_cons_w00 = 0  # Save w00 for minimum loss epoch
      min_cons_w10 = 0  # Save w10 difference for minimum loss epoch
      min_cons_w01 = 0  # Save w01 difference for minimum loss epoch
      min_cons_D1 = 0
      min_cons_D2 = 0
      min_cons_D3 = 0
      min_cons_L6 = 0

    # Backward pass and optimization
    optimizer.zero_grad()  # Zero the gradients
    loss.backward()  # Backpropagation
    optimizer.step()  # Update weights

Epoch 0, Loss_Tot: 1222.6977131297722, Const: (0.014797582530841291, 0.01580974702324589, 7.5629614121893445, 2.282618234584476, 1.0973146778165375, 0.298176744665952), W00: -328.9148370369678

Minimum Cons: 0.006456862667961107277170 at epoch 0
Values at minimum cons epoch:
W00: -328.9148370370
W10 diff: 0.0147975825
W01 diff: 0.0158097470
Comp3: 7.5629614121893445
Comp4: 2.2826182345844761
Comp5: 1.0973146778165375
Comp6: 0.2981767446659520
Coeffs: tensor([1.085689969219, 0.624362224378, 0.742575126561, 0.957486214642,
        1.212250967408, 1.506990392009, 1.828955908956, 0.376812068204,
        0.535718113340, 0.740467035332, 0.932777165430, 1.175767647729,
        1.445523906654, 0.386397262322, 0.431371833866, 0.523143535459,
        0.648721610471, 0.795204756944, 0.964304684474, 0.291390542622,
        0.316493346875, 0.365582972089, 0.448686762202, 0.575468092677,
        0.206423763687, 0.221323473177, 0.268287656707, 0.311328499998,
        0.393374169722, 0.140680060336, 0

In [ ]:
# Training loop
epoch_part = 20000
num_epochs = 4*epoch_part
min_cons = torch.tensor([10**4], dtype = torch.float64)  # Initialize minimum cons to positive infinity
min_cons_epoch = 0
max_w00 = -torch.tensor([10**10], dtype = torch.float64)  # Track the max w00
min_cons_w00 = 0  # Save w00 for minimum loss epoch
min_cons_w10 = 0  # Save w10 difference for minimum loss epoch
min_cons_w01 = 0  # Save w01 difference for minimum loss epoch
min_cons_D1 = 0
min_cons_D2 = 0
min_cons_D3 = 0
min_cons_L6 = 0


zero_vec = torch.zeros(len(NullPoints),dtype = torch.float64)
one_vec = torch.ones(len(NullPoints),dtype = torch.float64)
for epoch in range(num_epochs+1):

    w00vals = w00full()
    w01vals = w01full()
    w10vals = w10full()


    losscomp1 = (w01- w01vals)
    losscomp2 = (w10- w10vals)
    losscomp3 =  F.mse_loss(ampD1full(),zero_vec)
    losscomp4 =  F.mse_loss(ampD2full(),zero_vec)
    losscomp5 =  F.mse_loss(ampD3full(),zero_vec)
    losscomp6 =  F.mse_loss(ampD1Pfull()/ampD1Mfull(),one_vec)
    beta1 = 10**2
    beta4 = 10**2


    cons1 = (losscomp1**2 + losscomp2**2)
    cons2 =  losscomp3
    cons3 =  (losscomp4 + losscomp5)
    cons4 = losscomp6
    cons = (cons1**(1/2))*(cons4**(1/2))

    loss = - w00vals + (beta1**2)*cons1 + (beta4**2)*cons4

    if cons < min_cons:
      if w00vals > max_w00:
        min_cons = cons.item()
        min_cons_epoch = epoch
        max_w00 = w00vals.item()
        min_cons_w10 = (losscomp1).item()
        min_cons_w01 = (losscomp2).item()
        min_cons_D1 = (losscomp3**(1/2)).item()
        min_cons_D2 = (losscomp4**(1/2)).item()
        min_cons_D3 = (losscomp5**(1/2)).item()
        min_cons_L6 = (losscomp6**(1/2)).item()
        min_cons_coeffs = ImPW(ell_n_tensor)

    # Print loss every 5000 epochs
    if epoch % 5000 == 0:
        print(f"Epoch {epoch}, Loss_Tot: {loss.item()}, Const: {(losscomp1).item(), (losscomp2).item(), (losscomp3**(1/2)).item(), (losscomp4**(1/2)).item(),(losscomp5**(1/2)).item(),(losscomp6**(1/2)).item()}, W00: {w00vals.item()}")
     # Print best solution every 100000 epochs
    if (epoch) % (epoch_part) == 0:
      print(f"\nMinimum Cons: {min_cons:.24f} at epoch {min_cons_epoch}")
      print("Values at minimum cons epoch:")
      print(f"W00: {max_w00:.10f}")
      print(f"W10 diff: {min_cons_w10:.10f}")
      print(f"W01 diff: {min_cons_w01:.10f}")
      print(f"Comp3: {min_cons_D1:.16f}")
      print(f"Comp4: {min_cons_D2:.16f}")
      print(f"Comp5: {min_cons_D3:.16f}")
      print(f"Comp6: {min_cons_L6:.16f}")
      print(f"Coeffs: {min_cons_coeffs}")

      min_cons = torch.tensor([10**4], dtype = torch.float64)  # Initialize minimum cons to positive infinity
      min_cons_epoch = 0
      max_w00 = -torch.tensor([10**10], dtype = torch.float64)  # Track the max w00
      min_cons_w00 = 0  # Save w00 for minimum loss epoch
      min_cons_w10 = 0  # Save w10 difference for minimum loss epoch
      min_cons_w01 = 0  # Save w01 difference for minimum loss epoch
      min_cons_D1 = 0
      min_cons_D2 = 0
      min_cons_D3 = 0
      min_cons_L6 = 0

    # Backward pass and optimization
    optimizer.zero_grad()  # Zero the gradients
    loss.backward()  # Backpropagation
    optimizer.step()  # Update weights

Epoch 0, Loss_Tot: 1.6119347593011568, Const: (7.704423030219232e-06, -0.00022476130117476956, 0.0008452040577020881, 0.0011548938032536259, 0.0006715728214232341, 0.0009891565649714378), W00: -1.6016446821944959

Minimum Cons: 0.000000222454693470804007 at epoch 0
Values at minimum cons epoch:
W00: -1.6016446822
W10 diff: 0.0000077044
W01 diff: -0.0002247613
Comp3: 0.0008452040577021
Comp4: 0.0011548938032536
Comp5: 0.0006715728214232
Comp6: 0.0009891565649714
Coeffs: tensor([9.845771819594e-01, 1.563807423563e-01, 2.637292492286e-01,
        3.740555394670e-02, 3.743908808155e-03, 4.037221569589e-04,
        4.781827386587e-05, 5.661135937951e-02, 3.347563771332e-02,
        7.087496179205e-03, 7.599301510998e-04, 7.643678121759e-05,
        8.461522818067e-06, 8.352364878893e-03, 7.134101393795e-03,
        1.216958013017e-03, 1.167829427876e-04, 1.315188355287e-05,
        1.495329408729e-06, 2.199201771930e-03, 7.098580536852e-04,
        1.313883945019e-04, 2.028877288424e-05, 2.

In [ ]:
# Training loop
epoch_part = 20000
num_epochs = 4*epoch_part
min_cons = torch.tensor([10**4], dtype = torch.float64)  # Initialize minimum cons to positive infinity
min_cons_epoch = 0
max_w00 = -torch.tensor([10**10], dtype = torch.float64)  # Track the max w00
min_cons_w00 = 0  # Save w00 for minimum loss epoch
min_cons_w10 = 0  # Save w10 difference for minimum loss epoch
min_cons_w01 = 0  # Save w01 difference for minimum loss epoch
min_cons_D1 = 0
min_cons_D2 = 0
min_cons_D3 = 0
min_cons_L6 = 0


zero_vec = torch.zeros(len(NullPoints),dtype = torch.float64)
one_vec = torch.ones(len(NullPoints),dtype = torch.float64)
for epoch in range(num_epochs+1):

    w00vals = w00full()
    w01vals = w01full()
    w10vals = w10full()


    losscomp1 = (w01- w01vals)
    losscomp2 = (w10- w10vals)
    losscomp3 =  F.mse_loss(ampD1full(),zero_vec)
    losscomp4 =  F.mse_loss(ampD2full(),zero_vec)
    losscomp5 =  F.mse_loss(ampD3full(),zero_vec)
    losscomp6 =  F.mse_loss(ampD1Pfull()/ampD1Mfull(),one_vec)
    beta1 = 10**2
    beta4 = 10**2


    cons1 = (losscomp1**2 + losscomp2**2)
    cons2 =  losscomp3
    cons3 =  (losscomp4 + losscomp5)
    cons4 = losscomp6
    cons = (cons1**(1/2))*(cons4**(1/2))

    loss = - w00vals + (beta1**2)*cons1 + (beta4**2)*cons4

    if cons < min_cons:
      if w00vals > max_w00:
        min_cons = cons.item()
        min_cons_epoch = epoch
        max_w00 = w00vals.item()
        min_cons_w10 = (losscomp1).item()
        min_cons_w01 = (losscomp2).item()
        min_cons_D1 = (losscomp3**(1/2)).item()
        min_cons_D2 = (losscomp4**(1/2)).item()
        min_cons_D3 = (losscomp5**(1/2)).item()
        min_cons_L6 = (losscomp6**(1/2)).item()
        min_cons_coeffs = ImPW(ell_n_tensor)

    # Print loss every 5000 epochs
    if epoch % 5000 == 0:
        print(f"Epoch {epoch}, Loss_Tot: {loss.item()}, Const: {(losscomp1).item(), (losscomp2).item(), (losscomp3**(1/2)).item(), (losscomp4**(1/2)).item(),(losscomp5**(1/2)).item(),(losscomp6**(1/2)).item()}, W00: {w00vals.item()}")
     # Print best solution every 100000 epochs
    if (epoch) % (epoch_part) == 0:
      print(f"\nMinimum Cons: {min_cons:.24f} at epoch {min_cons_epoch}")
      print("Values at minimum cons epoch:")
      print(f"W00: {max_w00:.10f}")
      print(f"W10 diff: {min_cons_w10:.10f}")
      print(f"W01 diff: {min_cons_w01:.10f}")
      print(f"Comp3: {min_cons_D1:.16f}")
      print(f"Comp4: {min_cons_D2:.16f}")
      print(f"Comp5: {min_cons_D3:.16f}")
      print(f"Comp6: {min_cons_L6:.16f}")
      print(f"Coeffs: {min_cons_coeffs}")

      min_cons = torch.tensor([10**4], dtype = torch.float64)  # Initialize minimum cons to positive infinity
      min_cons_epoch = 0
      max_w00 = -torch.tensor([10**10], dtype = torch.float64)  # Track the max w00
      min_cons_w00 = 0  # Save w00 for minimum loss epoch
      min_cons_w10 = 0  # Save w10 difference for minimum loss epoch
      min_cons_w01 = 0  # Save w01 difference for minimum loss epoch
      min_cons_D1 = 0
      min_cons_D2 = 0
      min_cons_D3 = 0
      min_cons_L6 = 0

    # Backward pass and optimization
    optimizer.zero_grad()  # Zero the gradients
    loss.backward()  # Backpropagation
    optimizer.step()  # Update weights

Epoch 0, Loss_Tot: 1.567781829746423, Const: (-0.0004270458733175442, 6.710731808312609e-05, 0.0008077332099939884, 0.0033524507050305434, 0.002101680779283418, 0.000661732824934782), W00: -1.5615342107298829

Minimum Cons: 0.000000286058130068924872 at epoch 0
Values at minimum cons epoch:
W00: -1.5615342107
W10 diff: -0.0004270459
W01 diff: 0.0000671073
Comp3: 0.0008077332099940
Comp4: 0.0033524507050305
Comp5: 0.0021016807792834
Comp6: 0.0006617328249348
Coeffs: tensor([9.817302281532e-01, 2.299627139655e-01, 1.594512643255e-01,
        1.070166183329e-03, 1.091407218780e-05, 1.859035800269e-07,
        3.490046036914e-09, 5.326044142693e-02, 3.029398723400e-02,
        1.655955432510e-03, 1.790453654329e-05, 3.157220513870e-07,
        5.671291206118e-09, 4.057232861634e-03, 1.966139447323e-02,
        2.393082639361e-04, 2.808709500950e-06, 3.860636616529e-08,
        5.346233354295e-10, 1.605499823952e-03, 5.670834179939e-04,
        3.579559492991e-05, 5.527441132187e-07, 6.7919

# Checking dM/M constraints for 1 derivative

In [ ]:
#THIS SECTION IS JUST TO TEST THE EXPRESSIONS FOR THE OPEN STRING AMPLITUDE
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import numpy as np
from scipy.special import legendre
from scipy import integrate
from scipy.special import hyp2f1 as scipy_hyp2f1
from scipy.special import gamma as scipy_gamma
from scipy.special import zeta


#This is all for to check formulaes using the Open String Amplitude
#Some definitions
d = 10;
alpha = (d-3)/2;
lm = 146/10;
dlm = 1/2
lmp = lm + dlm
lmm = lm - dlm
ellmax = 19;
w00 = -(np.pi)**2/6
w10 = -zeta(3)
w01 = 7*np.pi**4/360

torch.set_printoptions(precision=12)

def hyp2f1(a, b, c, z):
    if isinstance(z, torch.Tensor):
        z_np = z.detach().numpy()
        result = scipy_hyp2f1(a,b,c,z_np)
        return torch.tensor(result, dtype=z.dtype)
    else:
        return torch.tensor(scipy_hyp2f1(a,b,c,z))

def gamma(x):
    if isinstance(x, torch.Tensor):
        x_np = x.detach().numpy()
        result = scipy_gamma(x_np)
        return torch.tensor(result, dtype=x.dtype)
    else:
        return torch.tensor(scipy_gamma(x))

def ker(lm,n,s1,s2):
    return (1/(s1 - n) + 1/(s2 - n) + 1/(n + lm))

def geg(ell,beta, z):
    return gamma(ell + 2*beta) / (gamma(2*beta) * gamma(ell + 1)) * \
           hyp2f1(-ell, ell + 2*beta, beta + 1/2, (1-z)/2)

#ell,n tensor and Null grid tensor
ell = torch.arange(ellmax + 1, dtype = torch.float64).unsqueeze(1).repeat(1, ellmax + 1)
n = torch.arange(1, ellmax + 2, dtype = torch.float64).unsqueeze(0).repeat(ellmax + 1, 1)
mask = (n > ell) & (n <= ellmax + 1) & ((n - ell - 1) % 2 == 0)
ell_n_tensor = torch.stack([ell[mask], n[mask]], dim=1)

NullPoints = torch.stack(torch.meshgrid(
    torch.arange(4/10, 104/10 + 1e-6, 1, dtype=torch.float64),
    torch.arange(101/10, 101/10 + 1e-6, 4/11, dtype=torch.float64),
    indexing='ij'
)).permute(1, 2, 0).reshape(-1, 2)


def w00comp(ell_n_tensor):
    ell = ell_n_tensor[:, 0].view(-1, 1)
    n = ell_n_tensor[:, 1].view(-1, 1)
    result = (-1/n)*geg(ell,alpha,1)
    return result.squeeze(-1)

def w10comp(ell_n_tensor):
    ell = ell_n_tensor[:, 0].view(-1, 1)  # Shape (k, 1)
    n = ell_n_tensor[:, 1].view(-1, 1)    # Shape (k, 1)
    result = (gamma(-3 + d + ell) * (-((2 * ell * (-3 + d + ell) * lm * (n + 2 * lm) * \
            hyp2f1(1 - ell, -2 + d + ell, d / 2, lm / (n + lm))) / ((-2 + d) * (n + lm)**2)) - \
            hyp2f1(-ell, -3 + d + ell, 1/2 * (-2 + d), lm / (n + lm)))) / (n**2 * gamma(-3 + d) * gamma(1 + ell))
    return result.squeeze(-1)


def w01comp(ell_n_tensor):
    ell = ell_n_tensor[:, 0].view(-1, 1)  # Shape (k, 1)
    n = ell_n_tensor[:, 1].view(-1, 1)    # Shape (k, 1)
    result = (gamma(-3 + d + ell) * (-(1/((d - 2) * (n + lm)**2)) *\
             2 * ell * (-3 + d + ell) * n * (n + 2*lm) *  hyp2f1(1 - ell, -2 + d + ell, d/2, lm/(n + lm)) + \
             2 * hyp2f1(-ell, -3 + d + ell, (d - 2)/2, lm/(n + lm)))) / (n**3 * gamma(-3 + d) * gamma(1 + ell))
    return result.squeeze(-1)

def ampcomp(ell_n_tensor, NullPoints):
    ell = ell_n_tensor[:, 0].view(-1, 1)
    n = ell_n_tensor[:, 1].view(-1, 1)
    s1 = NullPoints[:, 0].view(1, -1)
    s2 = NullPoints[:, 1].view(1, -1)
    result = ker(lm,n, s1, s2) * geg(ell, alpha, 1 + 2 * (((s1 + lm) * (s2 + lm)) / (n + lm) - lm) / n)
    return result.squeeze(-1)

def ampcompD1P(ell_n_tensor, NullPoints):
    ell = ell_n_tensor[:, 0].view(-1, 1)
    n = ell_n_tensor[:, 1].view(-1, 1)
    s1 = NullPoints[:, 0].view(1, -1)
    s2 = NullPoints[:, 1].view(1, -1)
    result = ker(lmp,n,s1,s2)*geg(ell,alpha,1 + 2*(((s1 + lmp)*(s2 + lmp))/(n + lmp)-lmp)/n)
    return result.squeeze(-1)

def ampcompD1M(ell_n_tensor, NullPoints):
    ell = ell_n_tensor[:, 0].view(-1, 1)
    n = ell_n_tensor[:, 1].view(-1, 1)
    s1 = NullPoints[:, 0].view(1, -1)
    s2 = NullPoints[:, 1].view(1, -1)
    result = ker(lmm,n,s1,s2)*geg(ell,alpha,1 + 2*(((s1 + lmm)*(s2 + lmm))/(n + lmm)-lmm)/n)
    return result.squeeze(-1)


def ampcompD1(ell_n_tensor, NullPoints):
    ell = ell_n_tensor[:, 0].view(-1, 1)
    n = ell_n_tensor[:, 1].view(-1, 1)
    s1 = NullPoints[:, 0].view(1, -1)
    s2 = NullPoints[:, 1].view(1, -1)
    result = ((2 * (-3 + d) * (n**2 + 2 * n * lm - s2 * lm - s1 * (s2 + lm)) * geg(-1 + ell, 1/2 * (-1 + d),
        1 + (2 * ((-n + s2) * lm + s1 * (s2 + lm))) /  (n * (n + lm))) / n - (n + lm) * geg(ell, 1/2 * (-3 + d),
        1 + (2 * ((-n + s2) * lm + s1 * (s2 + lm))) / (n * (n + lm)))) / (n + lm)**3)
    return result.squeeze(-1)

def ampcompD2(ell_n_tensor, NullPoints):
    ell = ell_n_tensor[:, 0].view(-1, 1)
    n = ell_n_tensor[:, 1].view(-1, 1)
    s1 = NullPoints[:, 0].view(1, -1)
    s2 = NullPoints[:, 1].view(1, -1)
    result = (1 / (n + lm)**4 * 2 * (-((2 * (-3 + d) * (-1 + d) * (n - s1) * (n - s2) *
               (n**2 + 2*n*lm - s2*lm - s1*(s2 + lm)) *
               geg(-2 + ell,(d+1)/2, 1 + (2 * ((-n + s2)*lm + s1*(s2 + lm)))/(n*(n + lm)))) /
              (n**2 * (n + lm))) + ((-3 + d) * (n - s1) * (n - s2) *
             geg(-1 + ell,(d-1)/2, 1 + (2 * ((-n + s2)*lm + s1*(s2 + lm)))/(n*(n + lm)))) / n +
            ((-3 + d) * (2*n - s1 - s2) * (n + lm) *
             geg(-1 + ell, (d-1)/2, 1 + (2 * ((-n + s2)*lm + s1*(s2 + lm)))/(n*(n + lm)))) / n -
            (3 * (-3 + d) * (n**2 + 2*n*lm - s2*lm - s1*(s2 + lm)) *
             geg(-1 + ell,(d-1)/2, 1 + (2 * ((-n + s2)*lm + s1*(s2 + lm)))/(n*(n + lm)))) / n +
            (n + lm) * geg(ell,(d-3)/2, 1 + (2 * ((-n + s2)*lm + s1*(s2 + lm)))/(n*(n + lm)))))

    return result.squeeze(-1)


def ampcompD3(ell_n_tensor, NullPoints):
    ell = ell_n_tensor[:, 0].view(-1, 1)
    n = ell_n_tensor[:, 1].view(-1, 1)
    s1 = NullPoints[:, 0].view(1, -1)
    s2 = NullPoints[:, 1].view(1, -1)
    result =  (2 / (n**3 * (n + lm)**7)) * (4 * (-3 + d) * (-1 + d) * (1 + d) * (n - s1)**2 * (n - s2)**2 * (n**2 + 2 * n * lm - s2 * lm -
        s1 * (s2 + lm)) * geg(-3 + ell, (3 + d)/2, 1 + (2 * ((-n + s2) * lm + s1 * (s2 + lm)))/(n * (n + lm))) -
     2 * (-3 + d) * (-1 + d) * n * (n - s1)**2 * (n -s2)**2 * (n + lm) * geg(-2 + ell, (1 + d)/2,
       1 + (2 * ((-n + s2) * lm + s1 * (s2 + lm)))/(n * (n + lm))) -
     4 * (-3 + d) * (-1 + d) * n * (n - s1) * (n - s2) * (2 * n - s1 -s2) * (n + lm)**2 * geg(-2 + ell, (1 + d)/2,
       1 + (2 * ((-n + s2) * lm + s1 * (s2 + lm)))/(n * (n + lm))) +
     8 * (-3 + d) * (-1 + d) * n * (n - s1) * (n - s2) * (n + lm) * (n**2 +2 * n * lm - s2 * lm -
        s1 * (s2 + lm)) * geg(-2 + ell, (1 + d)/2,1 + (2 * ((-n + s2) * lm + s1 * (s2 + lm)))/(n * (n + lm))) -
     2 * (-3 + d) * n**2 * (n - s1) * (n -s2) * (n + lm)**2 * geg(-1 + ell, 1/2 * (-1 + d),
       1 + (2 * ((-n + s2) * lm + s1 * (s2 + lm)))/( n * (n + lm))) -
     2 * (-3 + d) * n**2 * (2 * n - s1 - s2) * (n + lm)**3 * geg(-1 + ell, 1/2 * (-1 + d),1 + (2 * ((-n + s2) * lm + s1 * (s2 + lm)))/(n * (n + lm))) +
     n**3 * (n + lm)**3 * geg(ell, 1/2 * (-3 + d),1 + (2 * ((-n + s2) * lm + s1 * (s2 + lm)))/( n * (n + lm))) +
     4 * n * (n + lm) * (2 * (-3 + d) * (-1 + d) * (n - s1) * (n -s2) * (n**2 + 2 * n * lm - s2 * lm -
     s1 * (s2 + lm)) * geg(-2 + ell, (1 + d)/2,1 + (2 * ((-n + s2) * lm + s1 * (s2 + lm)))/(n * (n + lm)))
     - (-3 + d) * n * (n - s1) * (n -s2) * (n + lm) * geg(-1 + ell, 1/2 * (-1 + d),
      1 + (2 * ((-n + s2) * lm + s1 * (s2 + lm)))/(n * (n + lm))) - (-3 + d) * n * (2 * n - s1 -s2) * (n + lm)**2 * geg(-1 + ell, 1/2 * (-1 + d),
      1 + (2 * ((-n + s2) * lm + s1 * (s2 + lm)))/( n * (n + lm))) +3 * (-3 + d) * n * (n + lm) * (n**2 + 2 * n * lm -
      s2 * lm - s1 * (s2 + lm)) * geg(-1 + ell,1/2 * (-1 + d),1 + (2 * ((-n + s2) * lm + s1 * (s2 + lm)))/( n * (n + lm))) -
        n**2 * (n + lm)**2 * geg(ell, 1/2 * (-3 + d),  1 + (2 * ((-n + s2) * lm + s1 * (s2 + lm)))/(n * (n + lm)))))
    return result.squeeze(-1)

#First, lets calculate cc(ell,n) for open string

def AbsOpString(n,s2):
    return (-1)**(1 + n)*gamma(-s2)/(gamma(n+1)*gamma(1 - n - s2))



def cc(ell_n_tensor):
    z = torch.linspace(-1 + 10**(-6), 1 - 10**(-6), 10**6).view(1, -1)

    # Directly use ell and n from ell_n_tensor
    ell_values = ell_n_tensor[:, 0].view(-1, 1)  # Shape (k, 1)
    n_values = ell_n_tensor[:, 1].view(-1, 1)    # Shape (k, 1)

    # Compute normfac for all combinations of ell and n
    normfac = ((np.pi * 2**(1 - 2 * alpha) * gamma(2 * alpha + ell_values)) /
                (gamma(ell_values + 1) * (ell_values + alpha) * gamma(alpha)**2))**-1

    # Compute integrand
    integrand = normfac * (1 - z**2)**(alpha - 1/2) * \
                AbsOpString(n_values, n_values * (z - 1) / 2) * \
                geg(ell_values, alpha, z)

    # Integrate across z
    return torch.trapz(integrand, z, dim=1)

NullPoints

tensor([[ 0.400000000000, 10.100000000000],
        [ 1.400000000000, 10.100000000000],
        [ 2.400000000000, 10.100000000000],
        [ 3.400000000000, 10.100000000000],
        [ 4.400000000000, 10.100000000000],
        [ 5.400000000000, 10.100000000000],
        [ 6.400000000000, 10.100000000000],
        [ 7.400000000000, 10.100000000000],
        [ 8.400000000000, 10.100000000000],
        [ 9.400000000000, 10.100000000000],
        [10.400000000000, 10.100000000000]], dtype=torch.float64)

In [ ]:
#Store all the values
coeff_vals = cc(ell_n_tensor)
w00comp_vals = w00comp(ell_n_tensor)
w01comp_vals = w01comp(ell_n_tensor)
w10comp_vals = w10comp(ell_n_tensor)

ampcomp_vals = ampcomp(ell_n_tensor, NullPoints)
ampcompD1P_vals = ampcompD1P(ell_n_tensor, NullPoints)
ampcompD1M_vals = ampcompD1M(ell_n_tensor, NullPoints)
ampcompD1_vals = ampcompD1(ell_n_tensor, NullPoints)
ampcompD2_vals = ampcompD2(ell_n_tensor, NullPoints)
ampcompD3_vals = ampcompD3(ell_n_tensor, NullPoints)

<ipython-input-31-c351bcb08155>:31: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return torch.tensor(result, dtype=z.dtype)
<ipython-input-31-c351bcb08155>:33: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return torch.tensor(scipy_hyp2f1(a,b,c,z))


In [ ]:
#CHECKS FOR OPEN STRING
def w00fullX():
    return torch.dot(coeff_vals,w00comp_vals)

def w10fullX():
    return torch.dot(coeff_vals,w10comp_vals)

def w01fullX():
    return torch.dot(coeff_vals,w01comp_vals)

def ampfullX():
    return torch.sum(coeff_vals.view(-1, 1) * ampcomp_vals, dim = 0)

def ampD1PfullX():
    return torch.sum(coeff_vals.view(-1, 1) * ampcompD1P_vals, dim = 0)

def ampD1MfullX():
    return torch.sum(coeff_vals.view(-1, 1) * ampcompD1M_vals, dim = 0)

def ampD1fullX():
    return torch.sum(coeff_vals.view(-1, 1) * ampcompD1_vals, dim = 0)

def ampD2fullX():
    return torch.sum(coeff_vals.view(-1, 1) * ampcompD2_vals, dim = 0)

def ampD3fullX():
    return torch.sum(coeff_vals.view(-1, 1) * ampcompD3_vals, dim = 0)

(w00, w00fullX().item(),w10, w10fullX().item(),w01, w01fullX().item(),ampfullX()[10].item(),
 ampD1fullX()[10].item(),(ampD1fullX()/ampfullX())[10].item())

(-1.6449340668482264,
 -1.5961632442027958,
 -1.2020569031595942,
 -1.2020569018830987,
 1.8940656589944915,
 1.8940656589876617,
 134345.68405250116,
 -2.5088225522098497,
 -1.8674381465275974e-05)

In [ ]:
# Same definitions for the PINN
def w00full():
    return torch.dot(ImPW(ell_n_tensor),w00comp_vals)

def w01full():
    return torch.dot(ImPW(ell_n_tensor),w01comp_vals)

def w10full():
    return torch.dot(ImPW(ell_n_tensor),w10comp_vals)

def ampfull():
    return torch.sum(ImPW(ell_n_tensor).view(-1, 1) * ampcomp_vals, dim = 0)

def ampD1Pfull():
    return torch.sum(ImPW(ell_n_tensor).view(-1, 1) * ampcompD1P_vals, dim = 0)

def ampD1Mfull():
    return torch.sum(ImPW(ell_n_tensor).view(-1, 1) * ampcompD1M_vals, dim = 0)

def ampD1full():
    return torch.sum(ImPW(ell_n_tensor).view(-1, 1) * ampcompD1_vals, dim = 0)

def ampD2full():
    return torch.sum(ImPW(ell_n_tensor).view(-1, 1) * ampcompD2_vals, dim = 0)

def ampD3full():
    return torch.sum(ImPW(ell_n_tensor).view(-1, 1) * ampcompD3_vals, dim = 0)

In [ ]:
# Define the neural network. We will impose multi-lambda equality. But lets first check how well
# this is satisfied with three derivatives imposed.
class SimpleNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(SimpleNN, self).__init__()
        self.input = nn.Linear(input_size, hidden_size, dtype=torch.float64)
        self.hidden1 = nn.Linear(hidden_size, hidden_size, dtype=torch.float64)
        self.hidden2 = nn.Linear(hidden_size, hidden_size, dtype=torch.float64)
        self.output = nn.Linear(hidden_size, output_size, dtype=torch.float64)

    def forward(self, x):
        x = torch.relu(self.input(x))
        x = torch.relu(self.hidden1(x))
        x = torch.relu(self.hidden2(x))
        x = F.softplus(self.output(x))
        return x.squeeze()  # Remove extra dimensions for output

# Create the neural network
input_size = 2  # Since ell_n_tensor has 2 columns
hidden_size = 64
output_size = 1  # Since coeff(ell,n) it a 1D tensor
ImPW = SimpleNN(input_size, hidden_size, output_size)
lr = 0.0001
optimizer = optim.Adam(ImPW.parameters(), lr=lr)  # Adam optimizer with learning rate of 0.0001

zero_vec = torch.zeros(len(NullPoints),dtype = torch.float64)
one_vec = torch.ones(len(NullPoints),dtype = torch.float64)

# Training loop
epoch_part = 20000
num_epochs = epoch_part
min_cons = torch.tensor([10**4], dtype = torch.float64)  # Initialize minimum cons to positive infinity
min_cons_epoch = 0
max_w00 = -torch.tensor([10**10], dtype = torch.float64)  # Track the max w00
min_cons_w00 = 0  # Save w00 for minimum loss epoch
min_cons_w10 = 0  # Save w10 difference for minimum loss epoch
min_cons_w01 = 0  # Save w01 difference for minimum loss epoch
min_cons_D1 = 0
min_cons_D2 = 0
min_cons_D3 = 0
min_cons_L6 = 0


zero_vec = torch.zeros(len(NullPoints),dtype = torch.float64)
one_vec = torch.ones(len(NullPoints),dtype = torch.float64)
for epoch in range(num_epochs+1):

    w00vals = w00full()
    w01vals = w01full()
    w10vals = w10full()


    losscomp1 = (w01- w01vals)
    losscomp2 = (w10- w10vals)
    losscomp3 =  F.mse_loss(ampD1full(),zero_vec)
    losscomp4 =  F.mse_loss(ampD2full(),zero_vec)
    losscomp5 =  F.mse_loss(ampD3full(),zero_vec)
    losscomp6 =  F.mse_loss(ampD1full()/ampfull(),zero_vec)
    beta1 = 10**2
    beta4 = 10**2


    cons1 = (losscomp1**2 + losscomp2**2)
    cons2 =  losscomp3
    cons3 =  (losscomp4 + losscomp5)
    cons4 = losscomp6
    cons = (cons1**(1/2))*(cons4**(1/2))

    loss = - w00vals + (beta1**2)*cons1 + (beta4**2)*cons4

    if cons < min_cons:
      if w00vals > max_w00:
        min_cons = cons.item()
        min_cons_epoch = epoch
        max_w00 = w00vals.item()
        min_cons_w10 = (losscomp1).item()
        min_cons_w01 = (losscomp2).item()
        min_cons_D1 = (losscomp3**(1/2)).item()
        min_cons_D2 = (losscomp4**(1/2)).item()
        min_cons_D3 = (losscomp5**(1/2)).item()
        min_cons_L6 = (losscomp6**(1/2)).item()
        min_cons_coeffs = ImPW(ell_n_tensor)

    # Print loss every 5000 epochs
    if epoch % 5000 == 0:
        print(f"Epoch {epoch}, Loss_Tot: {loss.item()}, Const: {(losscomp1).item(), (losscomp2).item(), (losscomp3**(1/2)).item(), (losscomp4**(1/2)).item(),(losscomp5**(1/2)).item(),(losscomp6**(1/2)).item()}, W00: {w00vals.item()}")
     # Print best solution every 100000 epochs
    if (epoch+1) % (epoch_part) == 0:
      print(f"\nMinimum Cons: {min_cons:.24f} at epoch {min_cons_epoch}")
      print("Values at minimum cons epoch:")
      print(f"W00: {max_w00:.10f}")
      print(f"W10 diff: {min_cons_w10:.10f}")
      print(f"W01 diff: {min_cons_w01:.10f}")
      print(f"Comp3: {min_cons_D1:.16f}")
      print(f"Comp4: {min_cons_D2:.16f}")
      print(f"Comp5: {min_cons_D3:.16f}")
      print(f"Comp6: {min_cons_L6:.16f}")
      print(f"Coeffs: {min_cons_coeffs}")

      min_cons = torch.tensor([10**4], dtype = torch.float64)  # Initialize minimum cons to positive infinity
      min_cons_epoch = 0
      max_w00 = -torch.tensor([10**10], dtype = torch.float64)  # Track the max w00
      min_cons_w00 = 0  # Save w00 for minimum loss epoch
      min_cons_w10 = 0  # Save w10 difference for minimum loss epoch
      min_cons_w01 = 0  # Save w01 difference for minimum loss epoch
      min_cons_D1 = 0
      min_cons_D2 = 0
      min_cons_D3 = 0
      min_cons_L6 = 0

    # Backward pass and optimization
    optimizer.zero_grad()  # Zero the gradients
    loss.backward()  # Backpropagation
    optimizer.step()  # Update weights

Epoch 0, Loss_Tot: 100722.53255268406, Const: (1.1221696229948912, 0.45418347064635767, 355885443.977414, 47382291.655973606, 8860561.751928331, 0.0808790677041104), W00: -86001.64543894853
Epoch 5000, Loss_Tot: 52.75628831012998, Const: (-0.00024951706145492913, -0.00042757804670245925, 59898.05772812645, 7230.663778513237, 1260.5184169425427, 0.0656769607870327), W00: -9.619205710415885
Epoch 10000, Loss_Tot: 15.440777086385776, Const: (-0.0010683769123445153, 0.0007088052042654169, 869.7901965697465, 85.17518762429057, 12.448907164212255, 0.037410532665230777), W00: -1.4288592029785412
Epoch 15000, Loss_Tot: 5.432620542939739, Const: (0.00030332542323563416, -0.00039461618402358845, 176.52262769179492, 13.841697113409205, 1.6017781977652141, 0.02019045464569537), W00: -1.3535986724901787

Minimum Cons: 0.000000335495058210502536 at epoch 13998
Values at minimum cons epoch:
W00: -1.3322674776
W10 diff: -0.0000009463
W01 diff: -0.0000148251
Comp3: 193.8522293524100348
Comp4: 15.655993

In [ ]:
# Training loop
epoch_part = 2*20000
num_epochs = epoch_part
min_cons = torch.tensor([10**4], dtype = torch.float64)  # Initialize minimum cons to positive infinity
min_cons_epoch = 0
max_w00 = -torch.tensor([10**10], dtype = torch.float64)  # Track the max w00
min_cons_w00 = 0  # Save w00 for minimum loss epoch
min_cons_w10 = 0  # Save w10 difference for minimum loss epoch
min_cons_w01 = 0  # Save w01 difference for minimum loss epoch
min_cons_D1 = 0
min_cons_D2 = 0
min_cons_D3 = 0
min_cons_L6 = 0


zero_vec = torch.zeros(len(NullPoints),dtype = torch.float64)
one_vec = torch.ones(len(NullPoints),dtype = torch.float64)
for epoch in range(num_epochs+1):

    w00vals = w00full()
    w01vals = w01full()
    w10vals = w10full()


    losscomp1 = (w01- w01vals)
    losscomp2 = (w10- w10vals)
    losscomp3 =  F.mse_loss(ampD1full(),zero_vec)
    losscomp4 =  F.mse_loss(ampD2full(),zero_vec)
    losscomp5 =  F.mse_loss(ampD3full(),zero_vec)
    losscomp6 =  F.mse_loss(ampD1full()/ampfull(),zero_vec)
    beta1 = 10**2
    beta4 = 10**2


    cons1 = (losscomp1**2 + losscomp2**2)
    cons2 =  losscomp3
    cons3 =  (losscomp4 + losscomp5)
    cons4 = losscomp6
    cons = (cons1**(1/2))*(cons4**(1/2))

    loss = - w00vals + (beta1**2)*cons1 + (beta4**2)*cons4

    if cons < min_cons:
      if w00vals > max_w00:
        min_cons = cons.item()
        min_cons_epoch = epoch
        max_w00 = w00vals.item()
        min_cons_w10 = (losscomp1).item()
        min_cons_w01 = (losscomp2).item()
        min_cons_D1 = (losscomp3**(1/2)).item()
        min_cons_D2 = (losscomp4**(1/2)).item()
        min_cons_D3 = (losscomp5**(1/2)).item()
        min_cons_L6 = (losscomp6**(1/2)).item()
        min_cons_coeffs = ImPW(ell_n_tensor)

    # Print loss every 5000 epochs
    if epoch % 5000 == 0:
        print(f"Epoch {epoch}, Loss_Tot: {loss.item()}, Const: {(losscomp1).item(), (losscomp2).item(), (losscomp3**(1/2)).item(), (losscomp4**(1/2)).item(),(losscomp5**(1/2)).item(),(losscomp6**(1/2)).item()}, W00: {w00vals.item()}")
     # Print best solution every 100000 epochs
    if (epoch+1) % (epoch_part) == 0:
      print(f"\nMinimum Cons: {min_cons:.24f} at epoch {min_cons_epoch}")
      print("Values at minimum cons epoch:")
      print(f"W00: {max_w00:.10f}")
      print(f"W10 diff: {min_cons_w10:.10f}")
      print(f"W01 diff: {min_cons_w01:.10f}")
      print(f"Comp3: {min_cons_D1:.16f}")
      print(f"Comp4: {min_cons_D2:.16f}")
      print(f"Comp5: {min_cons_D3:.16f}")
      print(f"Comp6: {min_cons_L6:.16f}")
      print(f"Coeffs: {min_cons_coeffs}")

      min_cons = torch.tensor([10**4], dtype = torch.float64)  # Initialize minimum cons to positive infinity
      min_cons_epoch = 0
      max_w00 = -torch.tensor([10**10], dtype = torch.float64)  # Track the max w00
      min_cons_w00 = 0  # Save w00 for minimum loss epoch
      min_cons_w10 = 0  # Save w10 difference for minimum loss epoch
      min_cons_w01 = 0  # Save w01 difference for minimum loss epoch
      min_cons_D1 = 0
      min_cons_D2 = 0
      min_cons_D3 = 0
      min_cons_L6 = 0

    # Backward pass and optimization
    optimizer.zero_grad()  # Zero the gradients
    loss.backward()  # Backpropagation
    optimizer.step()  # Update weights

Epoch 0, Loss_Tot: 2.2457705267539296, Const: (3.495357213689765e-05, -0.00020360923966555866, 20.221730154432247, 1.9484820372101554, 0.3241630129298996, 0.009497254106240033), W00: -1.3433653864221748
Epoch 5000, Loss_Tot: 4.186607574062043, Const: (0.0004335002647257724, 0.0006896404681322466, 74.08868530104097, 5.3113646623015285, 0.5492296984214411, 0.01664569309146374), W00: -1.4091813245619764
Epoch 10000, Loss_Tot: 3.3307880522131565, Const: (1.9938022258969568e-05, -2.884761007271308e-05, 71.08413636762131, 5.10024506726909, 0.5315961216469218, 0.01382967110748439), W00: -1.4181777257078862
Epoch 15000, Loss_Tot: 2.984441616922628, Const: (6.353915426915435e-06, -2.5099667815942084e-05, 59.46019729638826, 4.231672745798405, 0.4462672057093945, 0.012479433015115405), W00: -1.4270724294794468
Epoch 20000, Loss_Tot: 1.9806320494752037, Const: (-8.035474356860028e-05, 1.1806849907314287e-05, 24.781966790592982, 2.737006192567001, 0.48189756647567034, 0.0072737553737051075), W00: -

In [ ]:
# Define the neural network. We will impose multi-lambda equality. But lets first check how well
# this is satisfied with three derivatives imposed.
class SimpleNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(SimpleNN, self).__init__()
        self.input = nn.Linear(input_size, hidden_size, dtype=torch.float64)
        self.hidden1 = nn.Linear(hidden_size, hidden_size, dtype=torch.float64)
        self.hidden2 = nn.Linear(hidden_size, hidden_size, dtype=torch.float64)
        self.output = nn.Linear(hidden_size, output_size, dtype=torch.float64)

    def forward(self, x):
        x = torch.relu(self.input(x))
        x = torch.relu(self.hidden1(x))
        x = torch.relu(self.hidden2(x))
        x = F.softplus(self.output(x))
        return x.squeeze()  # Remove extra dimensions for output

# Create the neural network
input_size = 2  # Since ell_n_tensor has 2 columns
hidden_size = 64
output_size = 1  # Since coeff(ell,n) it a 1D tensor
ImPW = SimpleNN(input_size, hidden_size, output_size)
lr = 0.0001
optimizer = optim.Adam(ImPW.parameters(), lr=lr)  # Adam optimizer with learning rate of 0.0001

zero_vec = torch.zeros(len(NullPoints),dtype = torch.float64)
one_vec = torch.ones(len(NullPoints),dtype = torch.float64)

# Training loop
epoch_part = 20000
num_epochs = epoch_part
min_cons = torch.tensor([10**4], dtype = torch.float64)  # Initialize minimum cons to positive infinity
min_cons_epoch = 0
max_w00 = -torch.tensor([10**10], dtype = torch.float64)  # Track the max w00
min_cons_w00 = 0  # Save w00 for minimum loss epoch
min_cons_w10 = 0  # Save w10 difference for minimum loss epoch
min_cons_w01 = 0  # Save w01 difference for minimum loss epoch
min_cons_D1 = 0
min_cons_D2 = 0
min_cons_D3 = 0
min_cons_L6 = 0


zero_vec = torch.zeros(len(NullPoints),dtype = torch.float64)
one_vec = torch.ones(len(NullPoints),dtype = torch.float64)
for epoch in range(num_epochs+1):

    w00vals = w00full()
    w01vals = w01full()
    w10vals = w10full()


    losscomp1 = (w01- w01vals)
    losscomp2 = (w10- w10vals)
    losscomp3 =  F.mse_loss(ampD1full(),zero_vec)
    losscomp4 =  F.mse_loss(ampD2full(),zero_vec)
    losscomp5 =  F.mse_loss(ampD3full(),zero_vec)
    losscomp6 =  F.mse_loss(ampD1full()/ampfull(),zero_vec)
    beta1 = 10**3
    beta4 = 10**4


    cons1 = (losscomp1**2 + losscomp2**2)
    cons2 =  losscomp3
    cons3 =  (losscomp4 + losscomp5)
    cons4 = losscomp6
    cons = (cons1**(1/2))*(cons4**(1/2))

    loss = - w00vals + (beta1**2)*cons1 + (beta4**2)*cons4

    if cons < min_cons:
      if w00vals > max_w00:
        min_cons = cons.item()
        min_cons_epoch = epoch
        max_w00 = w00vals.item()
        min_cons_w10 = (losscomp1).item()
        min_cons_w01 = (losscomp2).item()
        min_cons_D1 = (losscomp3**(1/2)).item()
        min_cons_D2 = (losscomp4**(1/2)).item()
        min_cons_D3 = (losscomp5**(1/2)).item()
        min_cons_L6 = (losscomp6**(1/2)).item()
        min_cons_coeffs = ImPW(ell_n_tensor)

    # Print loss every 5000 epochs
    if epoch % 5000 == 0:
        print(f"Epoch {epoch}, Loss_Tot: {loss.item()}, Const: {(losscomp1).item(), (losscomp2).item(), (losscomp3**(1/2)).item(), (losscomp4**(1/2)).item(),(losscomp5**(1/2)).item(),(losscomp6**(1/2)).item()}, W00: {w00vals.item()}")
     # Print best solution every 100000 epochs
    if (epoch+1) % (epoch_part) == 0:
      print(f"\nMinimum Cons: {min_cons:.24f} at epoch {min_cons_epoch}")
      print("Values at minimum cons epoch:")
      print(f"W00: {max_w00:.10f}")
      print(f"W10 diff: {min_cons_w10:.10f}")
      print(f"W01 diff: {min_cons_w01:.10f}")
      print(f"Comp3: {min_cons_D1:.16f}")
      print(f"Comp4: {min_cons_D2:.16f}")
      print(f"Comp5: {min_cons_D3:.16f}")
      print(f"Comp6: {min_cons_L6:.16f}")
      print(f"Coeffs: {min_cons_coeffs}")

      min_cons = torch.tensor([10**4], dtype = torch.float64)  # Initialize minimum cons to positive infinity
      min_cons_epoch = 0
      max_w00 = -torch.tensor([10**10], dtype = torch.float64)  # Track the max w00
      min_cons_w00 = 0  # Save w00 for minimum loss epoch
      min_cons_w10 = 0  # Save w10 difference for minimum loss epoch
      min_cons_w01 = 0  # Save w01 difference for minimum loss epoch
      min_cons_D1 = 0
      min_cons_D2 = 0
      min_cons_D3 = 0
      min_cons_L6 = 0

    # Backward pass and optimization
    optimizer.zero_grad()  # Zero the gradients
    loss.backward()  # Backpropagation
    optimizer.step()  # Update weights

Epoch 0, Loss_Tot: 1967347.0108321188, Const: (1.0287391234512722, 0.274083475532225, 286517923.46272576, 37823892.469479576, 7030747.432997793, 0.08734472476811098), W00: -71010.98067129756
Epoch 5000, Loss_Tot: 41184.8615862186, Const: (0.0020879438877416945, -0.003878113911989667, 859.2354806696462, 69.53265610364494, 8.418875535376733, 0.02028880072008237), W00: -1.9188431043685934
Epoch 10000, Loss_Tot: 181.43670341216654, Const: (-1.7802646110398612e-05, -2.9465846507648052e-05, 95.81431861895061, 8.557739044879659, 1.1597152178017833, 0.0013407718846606624), W00: -1.668593572197147
Epoch 15000, Loss_Tot: 16.164049598868964, Const: (-2.0404052172295906e-06, 3.576857371401232e-06, 121.09351768636348, 11.52857317142221, 1.6660762085458154, 0.00037859956280881526), W00: -1.8302697458042487

Minimum Cons: 0.000005060087423376469576 at epoch 6623
Values at minimum cons epoch:
W00: -1.6081960128
W10 diff: -0.0016383396
W01 diff: -0.0028643917
Comp3: 84.4584318896035029
Comp4: 7.4460761

In [ ]:
# Training loop
epoch_part = 2*20000
num_epochs = epoch_part
min_cons = torch.tensor([10**4], dtype = torch.float64)  # Initialize minimum cons to positive infinity
min_cons_epoch = 0
max_w00 = -torch.tensor([10**10], dtype = torch.float64)  # Track the max w00
min_cons_w00 = 0  # Save w00 for minimum loss epoch
min_cons_w10 = 0  # Save w10 difference for minimum loss epoch
min_cons_w01 = 0  # Save w01 difference for minimum loss epoch
min_cons_D1 = 0
min_cons_D2 = 0
min_cons_D3 = 0
min_cons_L6 = 0


zero_vec = torch.zeros(len(NullPoints),dtype = torch.float64)
one_vec = torch.ones(len(NullPoints),dtype = torch.float64)
for epoch in range(num_epochs+1):

    w00vals = w00full()
    w01vals = w01full()
    w10vals = w10full()


    losscomp1 = (w01- w01vals)
    losscomp2 = (w10- w10vals)
    losscomp3 =  F.mse_loss(ampD1full(),zero_vec)
    losscomp4 =  F.mse_loss(ampD2full(),zero_vec)
    losscomp5 =  F.mse_loss(ampD3full(),zero_vec)
    losscomp6 =  F.mse_loss(ampD1full()/ampfull(),zero_vec)
    beta1 = 10**3
    beta4 = 10**4


    cons1 = (losscomp1**2 + losscomp2**2)
    cons2 =  losscomp3
    cons3 =  (losscomp4 + losscomp5)
    cons4 = losscomp6
    cons = (cons1**(1/2))*(cons4**(1/2))

    loss = - w00vals + (beta1**2)*cons1 + (beta4**2)*cons4

    if cons < min_cons:
      if w00vals > max_w00:
        min_cons = cons.item()
        min_cons_epoch = epoch
        max_w00 = w00vals.item()
        min_cons_w10 = (losscomp1).item()
        min_cons_w01 = (losscomp2).item()
        min_cons_D1 = (losscomp3**(1/2)).item()
        min_cons_D2 = (losscomp4**(1/2)).item()
        min_cons_D3 = (losscomp5**(1/2)).item()
        min_cons_L6 = (losscomp6**(1/2)).item()
        min_cons_coeffs = ImPW(ell_n_tensor)

    # Print loss every 5000 epochs
    if epoch % 5000 == 0:
        print(f"Epoch {epoch}, Loss_Tot: {loss.item()}, Const: {(losscomp1).item(), (losscomp2).item(), (losscomp3**(1/2)).item(), (losscomp4**(1/2)).item(),(losscomp5**(1/2)).item(),(losscomp6**(1/2)).item()}, W00: {w00vals.item()}")
     # Print best solution every 100000 epochs
    if (epoch+1) % (epoch_part) == 0:
      print(f"\nMinimum Cons: {min_cons:.24f} at epoch {min_cons_epoch}")
      print("Values at minimum cons epoch:")
      print(f"W00: {max_w00:.10f}")
      print(f"W10 diff: {min_cons_w10:.10f}")
      print(f"W01 diff: {min_cons_w01:.10f}")
      print(f"Comp3: {min_cons_D1:.16f}")
      print(f"Comp4: {min_cons_D2:.16f}")
      print(f"Comp5: {min_cons_D3:.16f}")
      print(f"Comp6: {min_cons_L6:.16f}")
      print(f"Coeffs: {min_cons_coeffs}")

      min_cons = torch.tensor([10**4], dtype = torch.float64)  # Initialize minimum cons to positive infinity
      min_cons_epoch = 0
      max_w00 = -torch.tensor([10**10], dtype = torch.float64)  # Track the max w00
      min_cons_w00 = 0  # Save w00 for minimum loss epoch
      min_cons_w10 = 0  # Save w10 difference for minimum loss epoch
      min_cons_w01 = 0  # Save w01 difference for minimum loss epoch
      min_cons_D1 = 0
      min_cons_D2 = 0
      min_cons_D3 = 0
      min_cons_L6 = 0

    # Backward pass and optimization
    optimizer.zero_grad()  # Zero the gradients
    loss.backward()  # Backpropagation
    optimizer.step()  # Update weights

Epoch 0, Loss_Tot: 12.18120843967089, Const: (9.425371840188213e-05, -5.6047941565662285e-05, 138.6478764152385, 13.16805954420045, 1.9008437354306413, 0.0003204299276300041), W00: -1.9016494523875926
Epoch 5000, Loss_Tot: 9.421290559669394, Const: (0.00018047860882530742, -0.00011135931838324176, 183.30516782840454, 17.249622716063175, 2.4716303619906737, 0.00027061256624149335), W00: -2.053201032854433
Epoch 10000, Loss_Tot: 7.8754679222462025, Const: (5.008359059943679e-05, -2.815724100746486e-05, 200.60975562271508, 18.738959275717114, 2.6691959838593013, 0.0002408190737893818), W00: -2.072784095900146
Epoch 15000, Loss_Tot: 7.5002155328307865, Const: (0.0005652718744237895, -0.0003455677550450176, 174.35014718483797, 16.234891945585613, 2.3066714187504544, 0.00022627762337037587), W00: -1.9411098836747802
Epoch 20000, Loss_Tot: 6.355232758210838, Const: (8.613813264890524e-07, 6.234999363474714e-07, 157.87692566377933, 14.641823042634101, 2.0731526237471427, 0.00021221287213189132

In [ ]:
# Training loop
epoch_part = 20000
num_epochs = 2*epoch_part
min_cons = torch.tensor([10**4], dtype = torch.float64)  # Initialize minimum cons to positive infinity
min_cons_epoch = 0
max_w00 = -torch.tensor([10**10], dtype = torch.float64)  # Track the max w00
min_cons_w00 = 0  # Save w00 for minimum loss epoch
min_cons_w10 = 0  # Save w10 difference for minimum loss epoch
min_cons_w01 = 0  # Save w01 difference for minimum loss epoch
min_cons_D1 = 0
min_cons_D2 = 0
min_cons_D3 = 0
min_cons_L6 = 0


zero_vec = torch.zeros(len(NullPoints),dtype = torch.float64)
one_vec = torch.ones(len(NullPoints),dtype = torch.float64)
for epoch in range(num_epochs+1):

    w00vals = w00full()
    w01vals = w01full()
    w10vals = w10full()


    losscomp1 = (w01- w01vals)
    losscomp2 = (w10- w10vals)
    losscomp3 =  F.mse_loss(ampD1full(),zero_vec)
    losscomp4 =  F.mse_loss(ampD2full(),zero_vec)
    losscomp5 =  F.mse_loss(ampD3full(),zero_vec)
    losscomp6 =  F.mse_loss(ampD1full()/ampfull(),zero_vec)
    beta1 = 10**3
    beta4 = 10**4


    cons1 = (losscomp1**2 + losscomp2**2)
    cons2 =  losscomp3
    cons3 =  (losscomp4 + losscomp5)
    cons4 = losscomp6
    cons = (cons1**(1/2))*(cons4**(1/2))

    loss = - w00vals + (beta1**2)*cons1 + (beta4**2)*cons4

    if cons < min_cons:
      if w00vals > max_w00:
        min_cons = cons.item()
        min_cons_epoch = epoch
        max_w00 = w00vals.item()
        min_cons_w10 = (losscomp1).item()
        min_cons_w01 = (losscomp2).item()
        min_cons_D1 = (losscomp3**(1/2)).item()
        min_cons_D2 = (losscomp4**(1/2)).item()
        min_cons_D3 = (losscomp5**(1/2)).item()
        min_cons_L6 = (losscomp6**(1/2)).item()
        min_cons_coeffs = ImPW(ell_n_tensor)

    # Print loss every 5000 epochs
    if epoch % 5000 == 0:
        print(f"Epoch {epoch}, Loss_Tot: {loss.item()}, Const: {(losscomp1).item(), (losscomp2).item(), (losscomp3**(1/2)).item(), (losscomp4**(1/2)).item(),(losscomp5**(1/2)).item(),(losscomp6**(1/2)).item()}, W00: {w00vals.item()}")
     # Print best solution every 100000 epochs
    if (epoch+1) % (epoch_part) == 0:
      print(f"\nMinimum Cons: {min_cons:.24f} at epoch {min_cons_epoch}")
      print("Values at minimum cons epoch:")
      print(f"W00: {max_w00:.10f}")
      print(f"W10 diff: {min_cons_w10:.10f}")
      print(f"W01 diff: {min_cons_w01:.10f}")
      print(f"Comp3: {min_cons_D1:.16f}")
      print(f"Comp4: {min_cons_D2:.16f}")
      print(f"Comp5: {min_cons_D3:.16f}")
      print(f"Comp6: {min_cons_L6:.16f}")
      print(f"Coeffs: {min_cons_coeffs}")

      min_cons = torch.tensor([10**4], dtype = torch.float64)  # Initialize minimum cons to positive infinity
      min_cons_epoch = 0
      max_w00 = -torch.tensor([10**10], dtype = torch.float64)  # Track the max w00
      min_cons_w00 = 0  # Save w00 for minimum loss epoch
      min_cons_w10 = 0  # Save w10 difference for minimum loss epoch
      min_cons_w01 = 0  # Save w01 difference for minimum loss epoch
      min_cons_D1 = 0
      min_cons_D2 = 0
      min_cons_D3 = 0
      min_cons_L6 = 0

    # Backward pass and optimization
    optimizer.zero_grad()  # Zero the gradients
    loss.backward()  # Backpropagation
    optimizer.step()  # Update weights

Epoch 0, Loss_Tot: 13.243121498108032, Const: (0.0019666562860753967, -0.001229459671541866, 8.460292600793133, 3.009772108271091, 0.8056927671425375, 0.0002488321827587895), W00: -1.6720679489499661
Epoch 5000, Loss_Tot: 1.9896089358677966, Const: (-5.794485038457253e-07, -5.558786959625195e-07, 13.599134632034012, 4.6733550340740635, 1.0712386312522908, 5.911696671246713e-05), W00: -1.6401267157778088
Epoch 10000, Loss_Tot: 1.9127887900316898, Const: (-1.6143588088546323e-07, -1.202403129507701e-06, 12.595633126243717, 4.276951826019244, 0.9765447790622456, 5.498681780125056e-05), W00: -1.6104323050060676
Epoch 15000, Loss_Tot: 1.8772867860590048, Const: (2.8437561689909074e-05, -1.9049564644380013e-05, 11.97739756344299, 4.067517470051273, 0.9278111445833946, 5.33997645731295e-05), W00: -1.5909617195844314

Minimum Cons: 0.000000000037401061169227 at epoch 4664
Values at minimum cons epoch:
W00: -1.6403033410
W10 diff: 0.0000003512
W01 diff: -0.0000005246
Comp3: 13.8490298485373913


In [ ]:
# Training loop
epoch_part = 20000
num_epochs = 2*epoch_part
min_cons = torch.tensor([10**4], dtype = torch.float64)  # Initialize minimum cons to positive infinity
min_cons_epoch = 0
max_w00 = -torch.tensor([10**10], dtype = torch.float64)  # Track the max w00
min_cons_w00 = 0  # Save w00 for minimum loss epoch
min_cons_w10 = 0  # Save w10 difference for minimum loss epoch
min_cons_w01 = 0  # Save w01 difference for minimum loss epoch
min_cons_D1 = 0
min_cons_D2 = 0
min_cons_D3 = 0
min_cons_L6 = 0


zero_vec = torch.zeros(len(NullPoints),dtype = torch.float64)
one_vec = torch.ones(len(NullPoints),dtype = torch.float64)
for epoch in range(num_epochs+1):

    w00vals = w00full()
    w01vals = w01full()
    w10vals = w10full()


    losscomp1 = (w01- w01vals)
    losscomp2 = (w10- w10vals)
    losscomp3 =  F.mse_loss(ampD1full(),zero_vec)
    losscomp4 =  F.mse_loss(ampD2full(),zero_vec)
    losscomp5 =  F.mse_loss(ampD3full(),zero_vec)
    losscomp6 =  F.mse_loss(ampD1full()/ampfull(),zero_vec)
    beta1 = 10**3
    beta4 = 10**4


    cons1 = (losscomp1**2 + losscomp2**2)
    cons2 =  losscomp3
    cons3 =  (losscomp4 + losscomp5)
    cons4 = losscomp6
    cons = (cons1**(1/2))*(cons4**(1/2))

    loss = - w00vals + (beta1**2)*cons1 + (beta4**2)*cons4

    if cons < min_cons:
      if w00vals > max_w00:
        min_cons = cons.item()
        min_cons_epoch = epoch
        max_w00 = w00vals.item()
        min_cons_w10 = (losscomp1).item()
        min_cons_w01 = (losscomp2).item()
        min_cons_D1 = (losscomp3**(1/2)).item()
        min_cons_D2 = (losscomp4**(1/2)).item()
        min_cons_D3 = (losscomp5**(1/2)).item()
        min_cons_L6 = (losscomp6**(1/2)).item()
        min_cons_coeffs = ImPW(ell_n_tensor)

    # Print loss every 5000 epochs
    if epoch % 5000 == 0:
        print(f"Epoch {epoch}, Loss_Tot: {loss.item()}, Const: {(losscomp1).item(), (losscomp2).item(), (losscomp3**(1/2)).item(), (losscomp4**(1/2)).item(),(losscomp5**(1/2)).item(),(losscomp6**(1/2)).item()}, W00: {w00vals.item()}")
     # Print best solution every 100000 epochs
    if (epoch+1) % (epoch_part) == 0:
      print(f"\nMinimum Cons: {min_cons:.24f} at epoch {min_cons_epoch}")
      print("Values at minimum cons epoch:")
      print(f"W00: {max_w00:.10f}")
      print(f"W10 diff: {min_cons_w10:.10f}")
      print(f"W01 diff: {min_cons_w01:.10f}")
      print(f"Comp3: {min_cons_D1:.16f}")
      print(f"Comp4: {min_cons_D2:.16f}")
      print(f"Comp5: {min_cons_D3:.16f}")
      print(f"Comp6: {min_cons_L6:.16f}")
      print(f"Coeffs: {min_cons_coeffs}")

      min_cons = torch.tensor([10**4], dtype = torch.float64)  # Initialize minimum cons to positive infinity
      min_cons_epoch = 0
      max_w00 = -torch.tensor([10**10], dtype = torch.float64)  # Track the max w00
      min_cons_w00 = 0  # Save w00 for minimum loss epoch
      min_cons_w10 = 0  # Save w10 difference for minimum loss epoch
      min_cons_w01 = 0  # Save w01 difference for minimum loss epoch
      min_cons_D1 = 0
      min_cons_D2 = 0
      min_cons_D3 = 0
      min_cons_L6 = 0

    # Backward pass and optimization
    optimizer.zero_grad()  # Zero the gradients
    loss.backward()  # Backpropagation
    optimizer.step()  # Update weights

Epoch 0, Loss_Tot: 1.7711369355017341, Const: (2.3665809543160776e-06, -3.0433615085811994e-06, 9.706588190317065, 3.292864795718211, 0.7462667167381815, 4.8836012293133896e-05), W00: -1.5326264630775364
Epoch 5000, Loss_Tot: 1.8001005614326475, Const: (-0.0001508017431663955, 9.883800710253077e-05, 10.005098278206713, 3.3379123838000058, 0.754673553761852, 4.9071228845135006e-05), W00: -1.526791894005463
Epoch 10000, Loss_Tot: 1.7545510600528416, Const: (1.0903582643884135e-05, -8.39680876718596e-06, 9.351710326949952, 3.1892946168078895, 0.7235254181812751, 4.858085228254207e-05), W00: -1.5183517446910795
Epoch 15000, Loss_Tot: 1659.9005552707595, Const: (-0.008089582426125785, -0.014038011054296362, 859.6852599445373, 72.21992606064113, 9.196443651624103, 0.003734979928683595), W00: -2.3859503141982534

Minimum Cons: 0.000000000039467013995111 at epoch 11186
Values at minimum cons epoch:
W00: -1.5151262769
W10 diff: -0.0000003958
W01 diff: -0.0000007195
Comp3: 9.4808155468431128
Com

In [ ]:
# Training loop
epoch_part = 20000
num_epochs = 2*epoch_part
min_cons = torch.tensor([10**4], dtype = torch.float64)  # Initialize minimum cons to positive infinity
min_cons_epoch = 0
max_w00 = -torch.tensor([10**10], dtype = torch.float64)  # Track the max w00
min_cons_w00 = 0  # Save w00 for minimum loss epoch
min_cons_w10 = 0  # Save w10 difference for minimum loss epoch
min_cons_w01 = 0  # Save w01 difference for minimum loss epoch
min_cons_D1 = 0
min_cons_D2 = 0
min_cons_D3 = 0
min_cons_L6 = 0


zero_vec = torch.zeros(len(NullPoints),dtype = torch.float64)
one_vec = torch.ones(len(NullPoints),dtype = torch.float64)
for epoch in range(num_epochs+1):

    w00vals = w00full()
    w01vals = w01full()
    w10vals = w10full()


    losscomp1 = (w01- w01vals)
    losscomp2 = (w10- w10vals)
    losscomp3 =  F.mse_loss(ampD1full(),zero_vec)
    losscomp4 =  F.mse_loss(ampD2full(),zero_vec)
    losscomp5 =  F.mse_loss(ampD3full(),zero_vec)
    losscomp6 =  F.mse_loss(ampD1full()/ampfull(),zero_vec)
    beta1 = 10**3
    beta4 = 10**4


    cons1 = (losscomp1**2 + losscomp2**2)
    cons2 =  losscomp3
    cons3 =  (losscomp4 + losscomp5)
    cons4 = losscomp6
    cons = (cons1**(1/2))*(cons4**(1/2))

    loss = - w00vals + (beta1**2)*cons1 + (beta4**2)*cons4

    if cons < min_cons:
      if w00vals > max_w00:
        min_cons = cons.item()
        min_cons_epoch = epoch
        max_w00 = w00vals.item()
        min_cons_w10 = (losscomp1).item()
        min_cons_w01 = (losscomp2).item()
        min_cons_D1 = (losscomp3**(1/2)).item()
        min_cons_D2 = (losscomp4**(1/2)).item()
        min_cons_D3 = (losscomp5**(1/2)).item()
        min_cons_L6 = (losscomp6**(1/2)).item()
        min_cons_coeffs = ImPW(ell_n_tensor)

    # Print loss every 5000 epochs
    if epoch % 5000 == 0:
        print(f"Epoch {epoch}, Loss_Tot: {loss.item()}, Const: {(losscomp1).item(), (losscomp2).item(), (losscomp3**(1/2)).item(), (losscomp4**(1/2)).item(),(losscomp5**(1/2)).item(),(losscomp6**(1/2)).item()}, W00: {w00vals.item()}")
     # Print best solution every 100000 epochs
    if (epoch+1) % (epoch_part) == 0:
      print(f"\nMinimum Cons: {min_cons:.24f} at epoch {min_cons_epoch}")
      print("Values at minimum cons epoch:")
      print(f"W00: {max_w00:.10f}")
      print(f"W10 diff: {min_cons_w10:.10f}")
      print(f"W01 diff: {min_cons_w01:.10f}")
      print(f"Comp3: {min_cons_D1:.16f}")
      print(f"Comp4: {min_cons_D2:.16f}")
      print(f"Comp5: {min_cons_D3:.16f}")
      print(f"Comp6: {min_cons_L6:.16f}")
      print(f"Coeffs: {min_cons_coeffs}")

      min_cons = torch.tensor([10**4], dtype = torch.float64)  # Initialize minimum cons to positive infinity
      min_cons_epoch = 0
      max_w00 = -torch.tensor([10**10], dtype = torch.float64)  # Track the max w00
      min_cons_w00 = 0  # Save w00 for minimum loss epoch
      min_cons_w10 = 0  # Save w10 difference for minimum loss epoch
      min_cons_w01 = 0  # Save w01 difference for minimum loss epoch
      min_cons_D1 = 0
      min_cons_D2 = 0
      min_cons_D3 = 0
      min_cons_L6 = 0

    # Backward pass and optimization
    optimizer.zero_grad()  # Zero the gradients
    loss.backward()  # Backpropagation
    optimizer.step()  # Update weights

Epoch 0, Loss_Tot: 5.362006663716068, Const: (1.7865321994881e-05, -1.3703969204970079e-05, 104.392373163185, 9.672500807431673, 1.3691394915624928, 0.00019395546904455753), W00: -1.5996272979846857
Epoch 5000, Loss_Tot: 5.130974519199414, Const: (-1.5193284335079937e-06, -9.000531679959067e-07, 99.93484506111987, 9.233312159239668, 1.3047532848596568, 0.00018833467934239152), W00: -1.5839762564446773
Epoch 10000, Loss_Tot: 1.8555514609005985, Const: (-6.537565904829457e-05, 5.714649967170082e-05, 9.979652921208997, 3.4990150064160432, 0.8014725620884626, 5.7701706068348186e-05), W00: -1.515063073360067
Epoch 15000, Loss_Tot: 1.7479094541348086, Const: (-4.3033379548695905e-07, -1.2569863292455352e-06, 9.549574046469415, 3.269502316028815, 0.747062703876203, 5.025397484934888e-05), W00: -1.4953614901171022

Minimum Cons: 0.000000000038164272748429 at epoch 8835
Values at minimum cons epoch:
W00: -1.5285157828
W10 diff: -0.0000003051
W01 diff: -0.0000005944
Comp3: 8.7382248255990316
Com

In [ ]:
# Training loop
epoch_part = 20000
num_epochs = 2*epoch_part
min_cons = torch.tensor([10**4], dtype = torch.float64)  # Initialize minimum cons to positive infinity
min_cons_epoch = 0
max_w00 = -torch.tensor([10**10], dtype = torch.float64)  # Track the max w00
min_cons_w00 = 0  # Save w00 for minimum loss epoch
min_cons_w10 = 0  # Save w10 difference for minimum loss epoch
min_cons_w01 = 0  # Save w01 difference for minimum loss epoch
min_cons_D1 = 0
min_cons_D2 = 0
min_cons_D3 = 0
min_cons_L6 = 0


zero_vec = torch.zeros(len(NullPoints),dtype = torch.float64)
one_vec = torch.ones(len(NullPoints),dtype = torch.float64)
for epoch in range(num_epochs+1):

    w00vals = w00full()
    w01vals = w01full()
    w10vals = w10full()


    losscomp1 = (w01- w01vals)
    losscomp2 = (w10- w10vals)
    losscomp3 =  F.mse_loss(ampD1full(),zero_vec)
    losscomp4 =  F.mse_loss(ampD2full(),zero_vec)
    losscomp5 =  F.mse_loss(ampD3full(),zero_vec)
    losscomp6 =  F.mse_loss(ampD1full()/ampfull(),zero_vec)
    beta1 = 10**3
    beta4 = 10**4


    cons1 = (losscomp1**2 + losscomp2**2)
    cons2 =  losscomp3
    cons3 =  (losscomp4 + losscomp5)
    cons4 = losscomp6
    cons = (cons1**(1/2))*(cons4**(1/2))

    loss = - w00vals + (beta1**2)*cons1 + (beta4**2)*cons4

    if cons < min_cons:
      if w00vals > max_w00:
        min_cons = cons.item()
        min_cons_epoch = epoch
        max_w00 = w00vals.item()
        min_cons_w10 = (losscomp1).item()
        min_cons_w01 = (losscomp2).item()
        min_cons_D1 = (losscomp3**(1/2)).item()
        min_cons_D2 = (losscomp4**(1/2)).item()
        min_cons_D3 = (losscomp5**(1/2)).item()
        min_cons_L6 = (losscomp6**(1/2)).item()
        min_cons_coeffs = ImPW(ell_n_tensor)

    # Print loss every 5000 epochs
    if epoch % 5000 == 0:
        print(f"Epoch {epoch}, Loss_Tot: {loss.item()}, Const: {(losscomp1).item(), (losscomp2).item(), (losscomp3**(1/2)).item(), (losscomp4**(1/2)).item(),(losscomp5**(1/2)).item(),(losscomp6**(1/2)).item()}, W00: {w00vals.item()}")
     # Print best solution every 100000 epochs
    if (epoch) % (epoch_part) == 0:
      print(f"\nMinimum Cons: {min_cons:.24f} at epoch {min_cons_epoch}")
      print("Values at minimum cons epoch:")
      print(f"W00: {max_w00:.10f}")
      print(f"W10 diff: {min_cons_w10:.10f}")
      print(f"W01 diff: {min_cons_w01:.10f}")
      print(f"Comp3: {min_cons_D1:.16f}")
      print(f"Comp4: {min_cons_D2:.16f}")
      print(f"Comp5: {min_cons_D3:.16f}")
      print(f"Comp6: {min_cons_L6:.16f}")
      print(f"Coeffs: {min_cons_coeffs}")

      min_cons = torch.tensor([10**4], dtype = torch.float64)  # Initialize minimum cons to positive infinity
      min_cons_epoch = 0
      max_w00 = -torch.tensor([10**10], dtype = torch.float64)  # Track the max w00
      min_cons_w00 = 0  # Save w00 for minimum loss epoch
      min_cons_w10 = 0  # Save w10 difference for minimum loss epoch
      min_cons_w01 = 0  # Save w01 difference for minimum loss epoch
      min_cons_D1 = 0
      min_cons_D2 = 0
      min_cons_D3 = 0
      min_cons_L6 = 0

    # Backward pass and optimization
    optimizer.zero_grad()  # Zero the gradients
    loss.backward()  # Backpropagation
    optimizer.step()  # Update weights

Epoch 0, Loss_Tot: 1.7318231714612402, Const: (-3.5910245963499676e-06, -1.1946161795339805e-06, 8.578402752734767, 3.0675049444598796, 0.7042482868444416, 5.002646306903402e-05), W00: -1.4815441481760296

Minimum Cons: 0.000000000189325977097350 at epoch 0
Values at minimum cons epoch:
W00: -1.4815441482
W10 diff: -0.0000035910
W01 diff: -0.0000011946
Comp3: 8.5784027527347675
Comp4: 3.0675049444598796
Comp5: 0.7042482868444416
Comp6: 0.0000500264630690
Coeffs: tensor([9.918321507130e-01, 6.220454278521e-02, 5.568322436444e-03,
        6.328984339690e-04, 1.165227146282e-04, 2.349581785052e-05,
        4.018726353680e-06, 6.600286889364e-07, 1.103256930660e-07,
        1.892833417128e-08, 5.963842333198e-02, 8.592698456581e-03,
        1.236379131991e-03, 1.896790949828e-04, 3.480831696014e-05,
        7.212375580935e-06, 1.404577089772e-06, 2.551631754386e-07,
        4.539497289747e-08, 8.076021875555e-09, 6.909284104650e-03,
        1.202348686742e-03, 2.393669725925e-04, 5.5590413

In [ ]:
# Training loop
epoch_part = 20000
num_epochs = 2*epoch_part
min_cons = torch.tensor([10**4], dtype = torch.float64)  # Initialize minimum cons to positive infinity
min_cons_epoch = 0
max_w00 = -torch.tensor([10**10], dtype = torch.float64)  # Track the max w00
min_cons_w00 = 0  # Save w00 for minimum loss epoch
min_cons_w10 = 0  # Save w10 difference for minimum loss epoch
min_cons_w01 = 0  # Save w01 difference for minimum loss epoch
min_cons_D1 = 0
min_cons_D2 = 0
min_cons_D3 = 0
min_cons_L6 = 0


zero_vec = torch.zeros(len(NullPoints),dtype = torch.float64)
one_vec = torch.ones(len(NullPoints),dtype = torch.float64)
for epoch in range(num_epochs+1):

    w00vals = w00full()
    w01vals = w01full()
    w10vals = w10full()


    losscomp1 = (w01- w01vals)
    losscomp2 = (w10- w10vals)
    losscomp3 =  F.mse_loss(ampD1full(),zero_vec)
    losscomp4 =  F.mse_loss(ampD2full(),zero_vec)
    losscomp5 =  F.mse_loss(ampD3full(),zero_vec)
    losscomp6 =  F.mse_loss(ampD1full()/ampfull(),zero_vec)
    beta1 = 10**3
    beta4 = 10**4


    cons1 = (losscomp1**2 + losscomp2**2)
    cons2 =  losscomp3
    cons3 =  (losscomp4 + losscomp5)
    cons4 = losscomp6
    cons = (cons1**(1/2))*(cons4**(1/2))

    loss = - w00vals + (beta1**2)*cons1 + (beta4**2)*cons4

    if cons < min_cons:
      if w00vals > max_w00:
        min_cons = cons.item()
        min_cons_epoch = epoch
        max_w00 = w00vals.item()
        min_cons_w10 = (losscomp1).item()
        min_cons_w01 = (losscomp2).item()
        min_cons_D1 = (losscomp3**(1/2)).item()
        min_cons_D2 = (losscomp4**(1/2)).item()
        min_cons_D3 = (losscomp5**(1/2)).item()
        min_cons_L6 = (losscomp6**(1/2)).item()
        min_cons_coeffs = ImPW(ell_n_tensor)

    # Print loss every 5000 epochs
    if epoch % 5000 == 0:
        print(f"Epoch {epoch}, Loss_Tot: {loss.item()}, Const: {(losscomp1).item(), (losscomp2).item(), (losscomp3**(1/2)).item(), (losscomp4**(1/2)).item(),(losscomp5**(1/2)).item(),(losscomp6**(1/2)).item()}, W00: {w00vals.item()}")
     # Print best solution every 100000 epochs
    if (epoch) % (epoch_part) == 0:
      print(f"\nMinimum Cons: {min_cons:.24f} at epoch {min_cons_epoch}")
      print("Values at minimum cons epoch:")
      print(f"W00: {max_w00:.10f}")
      print(f"W10 diff: {min_cons_w10:.10f}")
      print(f"W01 diff: {min_cons_w01:.10f}")
      print(f"Comp3: {min_cons_D1:.16f}")
      print(f"Comp4: {min_cons_D2:.16f}")
      print(f"Comp5: {min_cons_D3:.16f}")
      print(f"Comp6: {min_cons_L6:.16f}")
      print(f"Coeffs: {min_cons_coeffs}")

      min_cons = torch.tensor([10**4], dtype = torch.float64)  # Initialize minimum cons to positive infinity
      min_cons_epoch = 0
      max_w00 = -torch.tensor([10**10], dtype = torch.float64)  # Track the max w00
      min_cons_w00 = 0  # Save w00 for minimum loss epoch
      min_cons_w10 = 0  # Save w10 difference for minimum loss epoch
      min_cons_w01 = 0  # Save w01 difference for minimum loss epoch
      min_cons_D1 = 0
      min_cons_D2 = 0
      min_cons_D3 = 0
      min_cons_L6 = 0

    # Backward pass and optimization
    optimizer.zero_grad()  # Zero the gradients
    loss.backward()  # Backpropagation
    optimizer.step()  # Update weights

Epoch 0, Loss_Tot: 1.9854058743447487, Const: (-4.157440558216763e-05, 2.5137017697351993e-05, 10.098027117483936, 4.505412351524974, 1.1399776923252793, 6.821352602829763e-05), W00: -1.5177370601651972

Minimum Cons: 0.000000003314012799210211 at epoch 0
Values at minimum cons epoch:
W00: -1.5177370602
W10 diff: -0.0000415744
W01 diff: 0.0000251370
Comp3: 10.0980271174839356
Comp4: 4.5054123515249742
Comp5: 1.1399776923252793
Comp6: 0.0000682135260283
Coeffs: tensor([9.923845245287e-01, 5.866007483048e-02, 5.052034857374e-03,
        5.614320401154e-04, 7.991808625847e-05, 9.982154815417e-06,
        1.234896649416e-06, 1.528248371804e-07, 1.874479021877e-08,
        2.239210755565e-09, 6.053164980146e-02, 8.369631809228e-03,
        1.123639186016e-03, 1.678729395622e-04, 2.970906846680e-05,
        3.847429109400e-06, 5.204492438867e-07, 6.837731892620e-08,
        8.681609493676e-09, 1.107486458501e-09, 7.494581027023e-03,
        1.170570946296e-03, 2.334195186890e-04, 5.115306816

In [ ]:
# Training loop
epoch_part = 20000
num_epochs = 2*epoch_part
min_cons = torch.tensor([10**4], dtype = torch.float64)  # Initialize minimum cons to positive infinity
min_cons_epoch = 0
max_w00 = -torch.tensor([10**10], dtype = torch.float64)  # Track the max w00
min_cons_w00 = 0  # Save w00 for minimum loss epoch
min_cons_w10 = 0  # Save w10 difference for minimum loss epoch
min_cons_w01 = 0  # Save w01 difference for minimum loss epoch
min_cons_D1 = 0
min_cons_D2 = 0
min_cons_D3 = 0
min_cons_L6 = 0


zero_vec = torch.zeros(len(NullPoints),dtype = torch.float64)
one_vec = torch.ones(len(NullPoints),dtype = torch.float64)
for epoch in range(num_epochs+1):

    w00vals = w00full()
    w01vals = w01full()
    w10vals = w10full()


    losscomp1 = (w01- w01vals)
    losscomp2 = (w10- w10vals)
    losscomp3 =  F.mse_loss(ampD1full(),zero_vec)
    losscomp4 =  F.mse_loss(ampD2full(),zero_vec)
    losscomp5 =  F.mse_loss(ampD3full(),zero_vec)
    losscomp6 =  F.mse_loss(ampD1full()/ampfull(),zero_vec)
    beta1 = 10**3
    beta4 = 10**4


    cons1 = (losscomp1**2 + losscomp2**2)
    cons2 =  losscomp3
    cons3 =  (losscomp4 + losscomp5)
    cons4 = losscomp6
    cons = (cons1**(1/2))*(cons4**(1/2))

    loss = - w00vals + (beta1**2)*cons1 + (beta4**2)*cons4

    if cons < min_cons:
      if w00vals > max_w00:
        min_cons = cons.item()
        min_cons_epoch = epoch
        max_w00 = w00vals.item()
        min_cons_w10 = (losscomp1).item()
        min_cons_w01 = (losscomp2).item()
        min_cons_D1 = (losscomp3**(1/2)).item()
        min_cons_D2 = (losscomp4**(1/2)).item()
        min_cons_D3 = (losscomp5**(1/2)).item()
        min_cons_L6 = (losscomp6**(1/2)).item()
        min_cons_coeffs = ImPW(ell_n_tensor)

    # Print loss every 5000 epochs
    if epoch % 5000 == 0:
        print(f"Epoch {epoch}, Loss_Tot: {loss.item()}, Const: {(losscomp1).item(), (losscomp2).item(), (losscomp3**(1/2)).item(), (losscomp4**(1/2)).item(),(losscomp5**(1/2)).item(),(losscomp6**(1/2)).item()}, W00: {w00vals.item()}")
     # Print best solution every 100000 epochs
    if (epoch) % (epoch_part) == 0:
      print(f"\nMinimum Cons: {min_cons:.24f} at epoch {min_cons_epoch}")
      print("Values at minimum cons epoch:")
      print(f"W00: {max_w00:.10f}")
      print(f"W10 diff: {min_cons_w10:.10f}")
      print(f"W01 diff: {min_cons_w01:.10f}")
      print(f"Comp3: {min_cons_D1:.16f}")
      print(f"Comp4: {min_cons_D2:.16f}")
      print(f"Comp5: {min_cons_D3:.16f}")
      print(f"Comp6: {min_cons_L6:.16f}")
      print(f"Coeffs: {min_cons_coeffs}")

      min_cons = torch.tensor([10**4], dtype = torch.float64)  # Initialize minimum cons to positive infinity
      min_cons_epoch = 0
      max_w00 = -torch.tensor([10**10], dtype = torch.float64)  # Track the max w00
      min_cons_w00 = 0  # Save w00 for minimum loss epoch
      min_cons_w10 = 0  # Save w10 difference for minimum loss epoch
      min_cons_w01 = 0  # Save w01 difference for minimum loss epoch
      min_cons_D1 = 0
      min_cons_D2 = 0
      min_cons_D3 = 0
      min_cons_L6 = 0

    # Backward pass and optimization
    optimizer.zero_grad()  # Zero the gradients
    loss.backward()  # Backpropagation
    optimizer.step()  # Update weights

Epoch 0, Loss_Tot: 1.8493465837068388, Const: (0.00011530217064303905, -7.660062091896513e-05, 9.333890743516745, 3.69611444869329, 0.9153508486277312, 6.089747210184725e-05), W00: -1.459334127187145

Minimum Cons: 0.000000008429900859825647 at epoch 0
Values at minimum cons epoch:
W00: -1.4593341272
W10 diff: 0.0001153022
W01 diff: -0.0000766006
Comp3: 9.3338907435167453
Comp4: 3.6961144486932902
Comp5: 0.9153508486277312
Comp6: 0.0000608974721018
Coeffs: tensor([9.914129375297e-01, 5.876343826770e-02, 5.024982965718e-03,
        5.626323926488e-04, 8.553573001048e-05, 1.038763589811e-05,
        1.259199858311e-06, 1.582264209059e-07, 2.012798431731e-08,
        2.560480897834e-09, 5.858827703854e-02, 8.441192731330e-03,
        1.144367134300e-03, 1.726460640883e-04, 3.183083299413e-05,
        4.402745163801e-06, 5.729440526590e-07, 7.455901454006e-08,
        9.752738050106e-09, 1.299619747249e-09, 6.297411161765e-03,
        1.102136968072e-03, 2.214529457719e-04, 5.492523394614e

In [ ]:
# Training loop
epoch_part = 20000
num_epochs = 2*epoch_part
min_cons = torch.tensor([10**4], dtype = torch.float64)  # Initialize minimum cons to positive infinity
min_cons_epoch = 0
max_w00 = -torch.tensor([10**10], dtype = torch.float64)  # Track the max w00
min_cons_w00 = 0  # Save w00 for minimum loss epoch
min_cons_w10 = 0  # Save w10 difference for minimum loss epoch
min_cons_w01 = 0  # Save w01 difference for minimum loss epoch
min_cons_D1 = 0
min_cons_D2 = 0
min_cons_D3 = 0
min_cons_L6 = 0


zero_vec = torch.zeros(len(NullPoints),dtype = torch.float64)
one_vec = torch.ones(len(NullPoints),dtype = torch.float64)
for epoch in range(num_epochs+1):

    w00vals = w00full()
    w01vals = w01full()
    w10vals = w10full()


    losscomp1 = (w01- w01vals)
    losscomp2 = (w10- w10vals)
    losscomp3 =  F.mse_loss(ampD1full(),zero_vec)
    losscomp4 =  F.mse_loss(ampD2full(),zero_vec)
    losscomp5 =  F.mse_loss(ampD3full(),zero_vec)
    losscomp6 =  F.mse_loss(ampD1full()/ampfull(),zero_vec)
    beta1 = 10**3
    beta4 = 10**4


    cons1 = (losscomp1**2 + losscomp2**2)
    cons2 =  losscomp3
    cons3 =  (losscomp4 + losscomp5)
    cons4 = losscomp6
    cons = (cons1**(1/2))*(cons4**(1/2))

    loss = - w00vals + (beta1**2)*cons1 + (beta4**2)*cons4

    if cons < min_cons:
      if w00vals > max_w00:
        min_cons = cons.item()
        min_cons_epoch = epoch
        max_w00 = w00vals.item()
        min_cons_w10 = (losscomp1).item()
        min_cons_w01 = (losscomp2).item()
        min_cons_D1 = (losscomp3**(1/2)).item()
        min_cons_D2 = (losscomp4**(1/2)).item()
        min_cons_D3 = (losscomp5**(1/2)).item()
        min_cons_L6 = (losscomp6**(1/2)).item()
        min_cons_coeffs = ImPW(ell_n_tensor)

    # Print loss every 5000 epochs
    if epoch % 5000 == 0:
        print(f"Epoch {epoch}, Loss_Tot: {loss.item()}, Const: {(losscomp1).item(), (losscomp2).item(), (losscomp3**(1/2)).item(), (losscomp4**(1/2)).item(),(losscomp5**(1/2)).item(),(losscomp6**(1/2)).item()}, W00: {w00vals.item()}")
     # Print best solution every 100000 epochs
    if (epoch) % (epoch_part) == 0:
      print(f"\nMinimum Cons: {min_cons:.24f} at epoch {min_cons_epoch}")
      print("Values at minimum cons epoch:")
      print(f"W00: {max_w00:.10f}")
      print(f"W10 diff: {min_cons_w10:.10f}")
      print(f"W01 diff: {min_cons_w01:.10f}")
      print(f"Comp3: {min_cons_D1:.16f}")
      print(f"Comp4: {min_cons_D2:.16f}")
      print(f"Comp5: {min_cons_D3:.16f}")
      print(f"Comp6: {min_cons_L6:.16f}")
      print(f"Coeffs: {min_cons_coeffs}")

      min_cons = torch.tensor([10**4], dtype = torch.float64)  # Initialize minimum cons to positive infinity
      min_cons_epoch = 0
      max_w00 = -torch.tensor([10**10], dtype = torch.float64)  # Track the max w00
      min_cons_w00 = 0  # Save w00 for minimum loss epoch
      min_cons_w10 = 0  # Save w10 difference for minimum loss epoch
      min_cons_w01 = 0  # Save w01 difference for minimum loss epoch
      min_cons_D1 = 0
      min_cons_D2 = 0
      min_cons_D3 = 0
      min_cons_L6 = 0

    # Backward pass and optimization
    optimizer.zero_grad()  # Zero the gradients
    loss.backward()  # Backpropagation
    optimizer.step()  # Update weights

Epoch 0, Loss_Tot: 1.6572151158741937, Const: (5.705541755229859e-05, -3.704934656423298e-05, 7.007735442422155, 2.4369632115892825, 0.5551893291149875, 4.630557325872445e-05), W00: -1.4381665296393802

Minimum Cons: 0.000000003150132023329291 at epoch 0
Values at minimum cons epoch:
W00: -1.4381665296
W10 diff: 0.0000570554
W01 diff: -0.0000370493
Comp3: 7.0077354424221552
Comp4: 2.4369632115892825
Comp5: 0.5551893291149875
Comp6: 0.0000463055732587
Coeffs: tensor([9.909807028743e-01, 6.056333055026e-02, 5.170909291525e-03,
        5.907293325524e-04, 8.270973389535e-05, 1.020816899092e-05,
        1.300411886647e-06, 1.656579651154e-07, 2.110296599189e-08,
        2.688280912163e-09, 5.777064526825e-02, 8.800058365869e-03,
        1.181625161954e-03, 1.834371514988e-04, 3.488581575113e-05,
        4.701920113252e-06, 6.099132984801e-07, 7.976571175673e-08,
        1.076172818198e-08, 1.451937022256e-09, 5.912616591234e-03,
        9.934467683782e-04, 2.422067335393e-04, 6.01398256585

In [ ]:
# Training loop
epoch_part = 20000
num_epochs = 2*epoch_part
min_cons = torch.tensor([10**4], dtype = torch.float64)  # Initialize minimum cons to positive infinity
min_cons_epoch = 0
max_w00 = -torch.tensor([10**10], dtype = torch.float64)  # Track the max w00
min_cons_w00 = 0  # Save w00 for minimum loss epoch
min_cons_w10 = 0  # Save w10 difference for minimum loss epoch
min_cons_w01 = 0  # Save w01 difference for minimum loss epoch
min_cons_D1 = 0
min_cons_D2 = 0
min_cons_D3 = 0
min_cons_L6 = 0


zero_vec = torch.zeros(len(NullPoints),dtype = torch.float64)
one_vec = torch.ones(len(NullPoints),dtype = torch.float64)
for epoch in range(num_epochs+1):

    w00vals = w00full()
    w01vals = w01full()
    w10vals = w10full()


    losscomp1 = (w01- w01vals)
    losscomp2 = (w10- w10vals)
    losscomp3 =  F.mse_loss(ampD1full(),zero_vec)
    losscomp4 =  F.mse_loss(ampD2full(),zero_vec)
    losscomp5 =  F.mse_loss(ampD3full(),zero_vec)
    losscomp6 =  F.mse_loss(ampD1full()/ampfull(),zero_vec)
    beta1 = 10**3
    beta4 = 10**4


    cons1 = (losscomp1**2 + losscomp2**2)
    cons2 =  losscomp3
    cons3 =  (losscomp4 + losscomp5)
    cons4 = losscomp6
    cons = (cons1**(1/2))*(cons4**(1/2))

    loss = - w00vals + (beta1**2)*cons1 + (beta4**2)*cons4

    if cons < min_cons:
      if w00vals > max_w00:
        min_cons = cons.item()
        min_cons_epoch = epoch
        max_w00 = w00vals.item()
        min_cons_w10 = (losscomp1).item()
        min_cons_w01 = (losscomp2).item()
        min_cons_D1 = (losscomp3**(1/2)).item()
        min_cons_D2 = (losscomp4**(1/2)).item()
        min_cons_D3 = (losscomp5**(1/2)).item()
        min_cons_L6 = (losscomp6**(1/2)).item()
        min_cons_coeffs = ImPW(ell_n_tensor)

    # Print loss every 5000 epochs
    if epoch % 5000 == 0:
        print(f"Epoch {epoch}, Loss_Tot: {loss.item()}, Const: {(losscomp1).item(), (losscomp2).item(), (losscomp3**(1/2)).item(), (losscomp4**(1/2)).item(),(losscomp5**(1/2)).item(),(losscomp6**(1/2)).item()}, W00: {w00vals.item()}")
     # Print best solution every 100000 epochs
    if (epoch) % (epoch_part) == 0:
      print(f"\nMinimum Cons: {min_cons:.24f} at epoch {min_cons_epoch}")
      print("Values at minimum cons epoch:")
      print(f"W00: {max_w00:.10f}")
      print(f"W10 diff: {min_cons_w10:.10f}")
      print(f"W01 diff: {min_cons_w01:.10f}")
      print(f"Comp3: {min_cons_D1:.16f}")
      print(f"Comp4: {min_cons_D2:.16f}")
      print(f"Comp5: {min_cons_D3:.16f}")
      print(f"Comp6: {min_cons_L6:.16f}")
      print(f"Coeffs: {min_cons_coeffs}")

      min_cons = torch.tensor([10**4], dtype = torch.float64)  # Initialize minimum cons to positive infinity
      min_cons_epoch = 0
      max_w00 = -torch.tensor([10**10], dtype = torch.float64)  # Track the max w00
      min_cons_w00 = 0  # Save w00 for minimum loss epoch
      min_cons_w10 = 0  # Save w10 difference for minimum loss epoch
      min_cons_w01 = 0  # Save w01 difference for minimum loss epoch
      min_cons_D1 = 0
      min_cons_D2 = 0
      min_cons_D3 = 0
      min_cons_L6 = 0

    # Backward pass and optimization
    optimizer.zero_grad()  # Zero the gradients
    loss.backward()  # Backpropagation
    optimizer.step()  # Update weights

Epoch 0, Loss_Tot: 1.6125134896286974, Const: (-1.969648344646302e-06, -1.344197457120444e-06, 6.075127663629711, 2.151925230868283, 0.48809781588118467, 4.2698390861210656e-05), W00: -1.4301925450336204

Minimum Cons: 0.000000000101819158030704 at epoch 0
Values at minimum cons epoch:
W00: -1.4301925450
W10 diff: -0.0000019696
W01 diff: -0.0000013442
Comp3: 6.0751276636297113
Comp4: 2.1519252308682830
Comp5: 0.4880978158811847
Comp6: 0.0000426983908612
Coeffs: tensor([9.912837042025e-01, 5.693513205136e-02, 4.666114757029e-03,
        5.105432498970e-04, 6.806108410160e-05, 8.123119470860e-06,
        9.826936189586e-07, 1.188808956445e-07, 1.394350681328e-08,
        1.607299681898e-09, 5.797680195961e-02, 8.433485072339e-03,
        1.103528725062e-03, 1.633001707048e-04, 3.048572445697e-05,
        3.871535267489e-06, 4.767872522993e-07, 6.001449399152e-08,
        7.616071402708e-09, 9.683526877160e-10, 5.870073273538e-03,
        9.738814722497e-04, 2.285596886865e-04, 5.69608090

In [ ]:
Minimum Cons: 0.000000000035135990593339 at epoch 29234
Values at minimum cons epoch:
W00: -1.4263267074
W10 diff: -0.0000006722
W01 diff: -0.0000004850
Comp3: 6.3288020616735725
Comp4: 2.1690481153664658
Comp5: 0.4911645418862397
Comp6: 0.0000423898468000

solTensor = torch.tensor([9.909786424221e-01, 5.831245027849e-02, 4.889267298079e-03,
        5.459525523058e-04, 7.593072707354e-05, 9.321459187367e-06,
        1.162991750703e-06, 1.451001346493e-07, 1.810334396960e-08,
        2.258654311105e-09, 5.748642405114e-02, 8.750967027870e-03,
        1.158325725456e-03, 1.749280456759e-04, 3.291607892181e-05,
        4.487746789229e-06, 5.696339346754e-07, 7.366809646699e-08,
        9.632722699769e-09, 1.270821138893e-09, 5.703666499225e-03,
        9.540975587035e-04, 2.418818952620e-04, 6.024208301060e-05,
        1.333964006948e-05, 2.597150479530e-06, 3.689154389894e-07,
        4.868078105349e-08, 6.423743395211e-09, 1.313225904534e-03,
        1.540323646400e-04, 5.426515731104e-05, 2.541478009370e-05,
        5.616198396680e-06, 1.208046373108e-06, 2.138869438717e-07,
        3.244855750223e-08, 4.418152339657e-09, 4.730936446912e-04,
        3.776847782864e-05, 1.289456172509e-05, 6.744676298659e-06,
        2.662608200681e-06, 5.671549439221e-07, 1.196524558326e-07,
        1.768406323756e-08, 2.772577289964e-04, 1.322165457429e-05,
        4.372001715032e-06, 1.696296704276e-06, 8.939518637113e-07,
        2.783543735535e-07, 5.995514075858e-08, 1.119374591912e-08,
        6.287479707099e-05, 8.509516405496e-06, 1.531947401763e-06,
        5.902393227702e-07, 2.261391939946e-07, 1.220450112634e-07,
        3.040949521787e-08, 1.081829773687e-05, 5.708273765410e-06,
        6.644348473947e-07, 2.128987262080e-07, 7.672168633527e-08,
        3.080026857889e-08, 1.582001832512e-08, 4.702724541761e-06,
        2.153756712558e-06, 4.152004024705e-07, 7.704335521876e-08,
        2.821722233626e-08, 1.028183663892e-08, 2.230778728978e-05,
        1.468627752813e-06, 2.454751627146e-07, 3.768817771549e-08,
        1.039114552266e-08, 3.952251440213e-09, 9.840051601179e-09,
        1.210724652591e-06, 3.198412851901e-07, 2.378291075923e-08,
        4.005741604013e-09, 7.469748308654e-12, 1.037408681475e-06,
        3.064699554966e-07, 2.533652662752e-08, 1.736214293074e-09,
        7.229245369600e-15, 8.889029861131e-07, 2.525247522921e-07,
        3.779821458993e-08, 8.507648481868e-18, 8.017381095941e-07,
        2.163756654229e-07, 5.084937638661e-08, 1.116005001209e-20,
        1.984647858315e-06, 1.854013439914e-07, 1.862558250892e-23,
        8.278591615765e-06, 1.588610170669e-07, 2.650675309404e-26,
        8.276486607088e-09, 5.124776563364e-29, 3.951016788873e-13,
        9.920129856583e-32, 1.929130555388e-34], dtype=torch.float64)

def CoeffFuncSol(solTensor, ell, n):
    solTensor = solTensor.unsqueeze(1)  # Shape [49, 1]
    combined_tensor = torch.cat((ell_n_tensor, solTensor), dim=1)
    mask = (combined_tensor[:, 0] == ell) & (combined_tensor[:, 1] == n)
    # Check if there is an entry
    if mask.any():
        return combined_tensor[mask, 2].item()
    else:
        return None

CoeffFuncSol(solTensor,0,1),CoeffFuncSol(solTensor,1,2),CoeffFuncSol(solTensor,2,3),CoeffFuncSol(solTensor,3,4),CoeffFuncSol(solTensor,4,5),CoeffFuncSol(solTensor,5,6)

(0.9909786424221,
 0.05748642405114,
 0.005703666499225,
 0.001313225904534,
 0.0004730936446912,
 0.0002772577289964)